# パッケージ

In [ ]:
for i in range(1):
    import pandas as pd
    import numpy as np
    import datetime
    from tqdm.notebook import tqdm
    from sklearn.metrics import roc_auc_score
    from sklearn.preprocessing import LabelEncoder
    import lightgbm as lgb
    import requests
    from bs4 import BeautifulSoup
    import time
    from tqdm.notebook import tqdm
    import re
    from urllib.request import urlopen
    import optuna.integration.lightgbm as lgb_o
    from itertools import combinations, permutations
    import matplotlib.pyplot as plt
    import pickle
    from tkinter import messagebox
    from datetime import datetime
    from datetime import timedelta
    import os
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.support.ui import Select
    from selenium.webdriver.chrome import service as fs
    from selenium.webdriver.common.by import By
    import time
    from webdriver_manager.chrome import ChromeDriverManager
    from dateutil.relativedelta import relativedelta
    import calendar
    import glob
    import schedule
    from selenium.webdriver.common.alert import Alert
    from IPython.display import clear_output
    from selenium.common.exceptions import NoSuchElementException, NoAlertPresentException

# 定義

In [ ]:
for i in range(1):
    #クラス
    class DataProcessor:
        """    
        Attributes:
        ----------
        data : rawデータ
        data_i : merge_horse_infomerge_horse_info後のデータ
        data_j : merge_jockey_info後のデータ
        data_p : preprocessing後のデータ
        data_h : merge_horse_results後のデータ
        data_cp : merge_horse_results後のデータ        
        data_pe : concat_past_data後のデータ
        data_c : process_categorical後のデータ
        no_peds: Numpy.array
            merge_pedsを実行した時に、血統データが存在しなかった馬のhorse_id一覧
        """

        def __init__(self):
            self.data = pd.DataFrame()
            self.data_i = pd.DataFrame()
            self.data_j = pd.DataFrame()
            self.data_p = pd.DataFrame()
            self.data_h = pd.DataFrame()
            self.data_cp = pd.DataFrame()
            self.data_pe = pd.DataFrame()
            self.data_c = pd.DataFrame()

        def merge_horse_info(self, hi):
            not_exist = list(set(self.data['horse_id']) - set(hi.horse_info.index))
            if len(not_exist)>0:
                print('not_exist_horse_info\n',not_exist)
                h_info = HorseResults.scrape(not_exist,False)
                # h_result.to_pickle('pickle/syutuba/new_h_info/h_result_'+datetime.now().strftime('%m_%d')+'.pickle')
                h_info.to_pickle('pickle/syutuba/new_h_info/h_info_'+datetime.now().strftime('%m_%d')+'.pickle')
                #read_pickle後と一致させる
                horse_info = h_info[['生年月日','調教師_id','馬主_id','生産者_id','近親馬_id1','近親馬_id2']]
                condf = pd.concat([hi.horse_info,horse_info])
                condf.to_pickle('pickle/syutuba/latest/h_info_latest.pickle')
                self.data_i = pd.merge(self.data,condf,how='left',left_on='horse_id',right_index=True)
            else:
                self.data_i = pd.merge(self.data,hi.horse_info,how='left',left_on='horse_id',right_index=True)

        def merge_jockey_info(self,jr,past_num):
            not_exist = list(set(self.data['jockey_id']) - set(jr.jockey_results.index))

            if len(not_exist)>0:
                print('not_exist_jockey_id\n',not_exist)
                j_id = jockey_results.scrape(not_exist)
                j_id.to_pickle('pickle/syutuba/new_j_id/j_id_'+datetime.now().strftime('%m_%d')+'.pickle')
                j_id_df = j_id[['年度', '順位', '1着', '2着', '3着', '着外', '重賞_出走', '重賞_勝利', '特別_出走', '特別_勝利',\
                                  '平場_出走', '平場_勝利', '芝_出走', '芝_勝利', 'ダート_出走', 'ダート_勝利', '勝率', '連対率',\
                                  '複勝率', '収得賞金(万円)']]
                condf = pd.concat([jr.jockey_results,j_id_df])
                condf.to_pickle('pickle/syutuba/latest/jockey_id_latest.pickle')
                jockey_df = condf.copy()

            else:
                jockey_df = jr.jockey_results.copy()

            jockey_id_list = jockey_df.index.unique()
            jockey_df.reset_index(inplace=True)
            jockey_df.rename(columns={'index': 'jockey_id'},inplace=True)
            jockey_df['年度'].replace('累計',0,inplace=True)
            jockey_df['年度'] = jockey_df['年度'].astype("int16")

            results = self.data_i.copy()
            results['年度']=results['date'].str[:4].astype("int16")
            max_nen = results['年度'].unique().max()
            min_nen = results['年度'].unique().min()
            results.reset_index(inplace=True)
            results.rename(columns={'index': 'race_id'},inplace=True)
            results.index.name = None

            #前年から順に抜き出し、指定した年数分のデータをくっつける
            for i in range(max_nen,min_nen-1,-1):
                # print(i)
                condf = results[results['年度']==i]

                for past in range(1,past_num+1):
                    # print('past',i-past)
                    tardf = jockey_df[jockey_df['年度']==i-past]
                    tardf = tardf.add_suffix('_{}'.format(past))
                    condf = pd.merge(condf,tardf,how='left',left_on=['jockey_id'],right_on =['jockey_id_{}'.format(past)])
                    condf.drop('jockey_id_{}'.format(past), axis=1, inplace=True)   
                #初回は結合先が無いため、代入
                if i ==max_nen:
                    concatdf=condf
                else:
                    concatdf = pd.concat([concatdf,condf])

            concatdf.set_index('race_id',inplace=True)
            concatdf.index.name = None

            nendo_col=[]
            for i in range(past_num):
                nendo_col.append('年度_{}'.format(i+1))

            concatdf.drop(nendo_col, axis=1, inplace=True)

            self.data_j = concatdf

            # del self.data_i
            # del self.data
        def merge_horse_results(self, hr, n_samples_list=[5, 9, 'all'],rt=pd.DataFrame()):
            """
            馬の過去成績データから、
            n_samples_listで指定されたレース分の着順と賞金の平均を追加してdata_hに返す
            Parameters:
            ----------
            hr : HorseResults
                馬の過去成績データ
            n_samples_list : list, default [5, 9, 'all']
                過去何レース分追加するか
            rt：results_p
                出馬テーブルを作成する際に過去resultsをくっつけてSPIを算出するために必要となる

            """
            """
            日付を取らないとdata_pに含まれるレース全レースを対象にしている
            リークが起きていないかは確認すること
            #ここでresultデータをhrクラスへ渡すために一度入れる
            #SPI算出のために出馬表をマージする際にはresultsテーブルを必要とする
            """

            if len(rt) != 0:
                self.data_p.reset_index(inplace = True)
                r_data_p = rt.copy()
                r_data_p.reset_index(inplace = True)
                #出馬テーブルに過去resultsをくっつける
                #出馬テーブルと重複しているレースがある場合は、削除
                r_data_p = r_data_p[~r_data_p['index'].isin(self.data_p['index'].unique())]
                self.data_h = pd.concat([self.data_p,r_data_p],axis = 0)
                self.data_h.set_index('index',inplace = True)
                self.data_p.set_index('index',inplace = True)
                # print(self.data_h[self.data_h.index=='202207040410'])
                #rtを出馬テーブルに置き換え
                rt = self
            else:
                self.data_h = self.data_p.copy()
                rt=None

            for n_samples in n_samples_list:
                self.data_h = hr.merge_all(self.data_h, n_samples=n_samples,rt=rt)
                # self.data_h = hr.merge_all(r_data_p, n_samples=n_samples,rt=rt)


            #6/6追加： 馬の出走間隔追加
            # self.data_h['interval'] = (self.data_h['date'] - self.data_h['latest']).dt.days
            self.data_h.insert(21, 'interval', (self.data_h['date'] - self.data_h['latest']).dt.days)

            #class変更 whereはnanだったものをnanに戻す
            self.data_h.insert(21, 'change_class', self.data_h['class']!=self.data_h['past_class'])
            self.data_h['change_class'].where(~self.data_h['past_class'].isna(),np.nan, inplace=True)
            self.data_h.insert(21, 'Past_class',self.data_h['past_class'])#順序変更のため

            #開催を削除
            # self.data_h.drop(['latest','騎手','past_jokey','past_race_type','len_cange','past_course_len','past_class'], axis=1, inplace=True)
            self.data_h.drop(['latest','騎手','past_class'], axis=1, inplace=True)

            # del self.data_j

        def concat_past_data(self,hr,past_num):

            def merge_past_data(df,myhr,day):

                #１日分のresultsとhorse_id
                # Aday_results = df.query('date == @day')
                Aday_results = df[df['date'] == day]
                Aday_hid_list = Aday_results['horse_id'].unique()

                day_hr = myhr[myhr['date'] < day]

                speedlist=[] 
                for target_hid in Aday_hid_list:

                    tardf = day_hr[day_hr['horse_id'] == target_hid].head(past_num).values

                    #arrayの一次元配列に変換
                    data_np = np.ravel(tardf)

                    data_len = len(tardf)
                    if data_len==0: #過去走データがない場合
                        conarray=empty_nan
                    else:
                        empty=np.array([np.nan] * col_num * (past_num-data_len))
                        conarray= np.concatenate([data_np,empty])

                    speedlist.append(conarray)

                afterdf = pd.DataFrame(speedlist,columns=suffix_list)
                afterdf['horse_id_1p'] = afterdf['horse_id_1p'].astype(str)#nanのときにmergeエラー吐くため
                mergdf = pd.merge(Aday_results,afterdf,how='left',left_on='horse_id',right_on = 'horse_id_1p')

                return mergdf

            def create_list(col_list):
                suffix_list , all_col = [],[]

                def suffix(n,m):
                    return n+'_{}p'.format(m)

                for num in range(1,past_num+1):
                    m = [str(num)] * len(col_list)
                    all_col.append(list(map(suffix, col_list,m)))

                return np.ravel(all_col)        

            df = self.data_h.copy()
            # df.reset_index(inplace=True)
            # df = df.rename(columns={'index': 'race_id'})

            myhr = hr.horse_results_p.copy()
            myhr.reset_index(inplace=True)
            myhr = myhr.rename(columns={'index': 'horse_id'})
            #category型にすることで抽出処理を高速化する
            myhr['horse_id'] = myhr['horse_id'].astype('category')

            col_list=myhr.columns

            col_num=len(col_list)
            # print(col_list)

            suffix_list=create_list(col_list)

            #resultsからすべての日付を取得
            day_list = df['date'].unique()
            empty_nan = np.array([np.nan] * col_num * past_num).astype('object')

            merged_df = pd.concat([merge_past_data(df,myhr,day) for day in tqdm(day_list)])

            self.data_cp = merged_df

            #複数list宣言方法
            horse_id_col, jockey_name_col , date_col = [],[],[]
            for i in range(past_num):
                horse_id_col.append('horse_id_{}p'.format(i+1))
                jockey_name_col.append('騎手_{}p'.format(i+1))
                date_col.append('date_{}p'.format(i+1))

            self.data_cp.drop(horse_id_col + jockey_name_col + date_col, axis=1, inplace=True)

            # del self.data_h

        def merge_peds(self, peds):
            """
            5世代分血統データを追加してdata_peに返す
            Parameters:
            ----------
            peds : Peds.peds_e
                Pedsクラスで加工された血統データ。
            """

            self.data_pe = self.data_cp.merge(peds, left_on='horse_id',right_index=True,how='left')

        def process_categorical(self, le_horse, le_jockey,le_class, le_place, results_m\
                                ,le_trainer,le_horseowner,le_producer,le_relative1,le_relative2,le_kai,le_day,past_race_num):

            """
            カテゴリ変数を処理してdata_cに返す
            Parameters:
            ----------
            le_horse : sklearn.preprocessing.LabelEncoder
                horse_idを0始まりの整数に変換するLabelEncoderオブジェクト
            le_jockey : sklearn.preprocessing.LabelEncoder
                jockey_idを0始まりの整数に変換するLabelEncoderオブジェクト。
            results_m : Results.data_pe
                ダミー変数化のとき、ResultsクラスとShutubaTableクラスで列を合わせるためのもの
            le_class : sklearn.preprocessing.LabelEncoder
                classを0始まりの整数に変換するLabelEncoderオブジェクト。
            """
            df = self.data_pe.copy()

            #存在していないidをmask関数で抽出してclassesに追加し、エンコードする。
            #末尾に追加するための処理
            def masking(df,le_encoder,colname,colname2):
                mask_horse = df[colname].isin(le_encoder.classes_)
                new_horse_id = df[colname].mask(mask_horse).dropna().unique()
                le_encoder.classes_ = np.concatenate([le_encoder.classes_, new_horse_id])
                #全体がnanの場合にfloat型になりtransformでエラーになるためobject型に変換
                df[colname2] = df[colname2].astype('object')
                df[colname2] = le_encoder.transform(df[colname2])
                df[colname2] = df[colname2].astype('category')
                return le_encoder

            for num in range(past_race_num):
                masking(df,le_jockey,'jockey_id_{}p'.format(num+1),'jockey_id_{}p'.format(num+1))
                masking(df,le_place,'place_{}p'.format(num+1),'place_{}p'.format(num+1))
                masking(df,le_kai,'kai_{}p'.format(num+1),'kai_{}p'.format(num+1))
                masking(df,le_day,'day_{}p'.format(num+1),'day_{}p'.format(num+1))

            self.le_horse = masking(df,le_horse,'horse_id','horse_id')
            self.le_jockey = masking(df,le_jockey,'jockey_id','jockey_id')            
            # masking(df,le_jockey,'jockey_id','past_jockey')
            self.le_class = masking(df,le_class,'class','class')
            self.le_class = masking(df,le_class,'class','Past_class')
            self.le_place = masking(df,le_place,'place','place')
            self.le_trainer = masking(df,le_trainer,'調教師_id','調教師_id')
            self.le_horseowner = masking(df,le_horseowner,'馬主_id','馬主_id')
            self.le_producer = masking(df,le_producer,'生産者_id','生産者_id')
            self.le_relative1 = masking(df,le_relative1,'近親馬_id1','近親馬_id1')
            self.le_relative2 = masking(df,le_relative2,'近親馬_id2','近親馬_id2')

            #------------------------------------------------------------------------------------------------------        
            #そのほかのカテゴリ変数をpandasのcategory型に変換してからダミー変数化
            #列を一定にするため
            #nullを含んでいる場合は変換できないため"Na"埋め
            def change_categorical(df,results_m,colname):
                results_m[colname].fillna('Na',inplace =True)
                unique_value = results_m[colname].unique()
                df[colname] = pd.Categorical(df[colname], unique_value)
                return df

            weather_past = []
            race_type_past = []
            ground_state_past = []
            for num in range(past_race_num):
                weather_past.append('weather_{}p'.format(num+1))
                race_type_past.append('race_type_{}p'.format(num+1))
                ground_state_past.append('ground_state_{}p'.format(num+1))

            columns_name=['weather', 'race_type', 'ground_state', '性','race_round','change_class']      
            columns_name = columns_name+weather_past+race_type_past+ground_state_past

            for colname in columns_name:
                df = change_categorical(df,results_m,colname)

            df = pd.get_dummies(df, columns=columns_name)

            self.data_c = df
    class Results(DataProcessor):
        def __init__(self, results):
            super(Results, self).__init__()
            self.data = results

        @classmethod
        def read_pickle(cls, path_list):
            df = pd.read_pickle(path_list[0])
            for path in path_list[1:]:
                df = update_data(df, pd.read_pickle(path))
            return cls(df)

        @staticmethod
        def scrape(race_id_list):
            """
            レース結果データをスクレイピングする関数
            Parameters:
            ----------
            race_id_list : list
                レースIDのリスト
            Returns:
            ----------
            race_results_df : pandas.DataFrame
                全レース結果データをまとめてDataFrame型にしたもの
            """

            #race_idをkeyにしてDataFrame型を格納
            race_results = {}
            for race_id in tqdm(race_id_list):
                try:
                    time.sleep(1)
                    url = "https://db.netkeiba.com/race/" + race_id

                    #メインとなるテーブルデータを取得
                    df = pd.read_html(url)[0]

                    html = requests.get(url)
                    html.encoding = "EUC-JP"
                    soup = BeautifulSoup(html.text, "html.parser")

                    #天候、レースの種類、コースの長さ、馬場の状態、日付をスクレイピング
                    texts = (
                        soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                        + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
                    )
                    info = re.findall(r'\w+', texts)
                    # print(info)
                    for text in info:
                        if text in ["芝", "ダート"]:
                            df["race_type"] = [text] * len(df)
                        if "障" in text:
                            df["race_type"] = ["障害"] * len(df)
                        if "m" in text:
                            df['course_len'] = [int(re.findall(r"(\d+)(?=m)", text)[0])] * len(df)
                            #回り情報の取得追加
                            procestext = re.findall(r"(\D+)(?=\d)", text)[0]
                            procestext = re.sub(r"[ダ,芝]", "", procestext)
                            df["race_round"] = [procestext] * len(df)
                        if text in ["良", "稍重", "重", "不良"]:
                            df["ground_state"] = [text] * len(df)
                        if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                            df["weather"] = [text] * len(df)
                        if "年" in text:
                            df["date"] = [text] * len(df)
    #独自追加--------------------------------------------------------
                    #クラス名取得を追加
                        if any(anytext in text for anytext in ("新馬","未出走","勝","オープン","以上","以下","万下")):
                            df["class"] = [text] * len(df)

                    #レース名取得を追加
                    race_name = soup.find("div", attrs={"class": "data_intro"}).find_all("h1")[0].text
                    df["race_name"] = race_name
    #--------------------------------------------------------   
                    #馬ID、騎手IDをスクレイピング
                    horse_id_list = []
                    horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                        "a", attrs={"href": re.compile("^/horse")}
                    )
                    for a in horse_a_list:
                        horse_id = re.findall(r"\d+", a["href"])
                        horse_id_list.append(horse_id[0])
                    jockey_id_list = []
                    jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                        "a", attrs={"href": re.compile("^/jockey")}
                    )
                    for a in jockey_a_list:
                        jockey_id = re.findall(r"recent/(\w+)", a["href"])
                        jockey_id_list.append(jockey_id[0])
                    df["horse_id"] = horse_id_list
                    df["jockey_id"] = jockey_id_list

                    #インデックスをrace_idにする
                    df.index = [race_id] * len(df)

                    race_results[race_id] = df



                #存在しないrace_idを飛ばす
                except IndexError:
                    continue
                #wifiの接続が切れた時などでも途中までのデータを返せるようにする
                except Exception as e:
                    print(e)
                    break
                #Jupyterで停止ボタンを押した時の対処
                except:
                    break

            #pd.DataFrame型にして一つのデータにまとめる
            race_results_df = pd.concat([race_results[key] for key in race_results])

            return race_results_df

        #前処理    
        def preprocessing(self):
            df = self.data_j.copy()

            #障害レースを削除
            df = df[~(df['race_type']=='障害')]

            # 着順に数字以外の文字列が含まれているものを取り除く
            df['着順'] = pd.to_numeric(df['着順'], errors='coerce')
            df.dropna(subset=['着順'], inplace=True)
            df['着順'] = df['着順'].astype(int)

            # #二値分類パターン(標準3位以内)
            # df['rank'] = df['着順'].map(lambda x:1 if x<4 else 0)

            # # # 人気よりも着順が3以上良い馬かつ3位以内
            # df['人気-着順']  = df['人気']-df['着順'] 
            # df.loc[(df['人気-着順'] >2)&(df['着順']<4),'着順'] = 1
            # df.loc[(df['人気-着順'] <=2)&(df['着順']<4),'着順'] = 0
            # df.loc[(df['人気-着順'] >0)&(df['着順']>=4),'着順'] = 0
            # df.loc[(df['人気-着順'] <=0),'着順'] = 0
            # df['rank'] = df['着順']
            # df.drop(['人気-着順'],axis=1, inplace=True)

            #多クラス分類パターン(rankをグループで順位付けする)
            # df['rank'] = df['着順'].map(lambda x:1 if x<4 else (2 if x>3 and x<7 else (3 if x>6 and x<10 else (4 if x>9 and x<14 else 5))))

            # # 多クラス分類パターン(着順そのまま)
            # df['rank'] = df['着順']

            # 性齢を性と年齢に分ける
            df["性"] = df["性齢"].map(lambda x: str(x)[0])
            df["年齢"] = df["性齢"].map(lambda x: str(x)[1:]).astype(int)

            # 馬体重を体重と体重変化に分ける
            df["体重"] = df["馬体重"].str.split("(", expand=True)[0]
            df["体重変化"] = df["馬体重"].str.split("(", expand=True)[1].str[:-1]

            #errors='coerce'で、"計不"など変換できない時に欠損値にする
            df['体重'] = pd.to_numeric(df['体重'], errors='coerce')
            df['体重変化'] = pd.to_numeric(df['体重変化'], errors='coerce')

            # 単勝をfloatに変換
            df["単勝"] = df["単勝"].astype(float)
            # 距離は10の位を切り捨てる
            df["course_len"] = df["course_len"].astype(float) // 100

    #独自追加--------------------------------------------------------
            # タイムは着順が漢字(中,除,取)のときにNaNになるため上記でDrop済みだが念のため
            df.dropna(subset=['タイム'], inplace=True)
            #独自追加--------------------------------------------------------
            #タイムデータ(秒表示に変換)
            def race_time(x):
                try:
                    if type(x) != str:
                        return x
                    elif  x.count(':') == 2 :
                        m, s, l = str(x).split(':')
                        return (int(m) * 60 + float(s) + float(l))
                    elif  x.count('.') == 2 :
                        m, s, l = str(x).split('.')
                        return (int(m) * 60 + float(s) + float(l))
                    else:
                        m, s = str(x).split(':')
                        return ((int(m) * 60 + float(s)))
                except Exception as e:
                    print(x)
                    print(type(x))
                    print(e)

            df['time'] = df['タイム'].map(lambda x: race_time(x))
            df.drop(['タイム'], axis=1, inplace=True)
            #--------------------------------------------------------
    #独自追加--------------------------------------------------------       

            # 不要な列を削除
            #'騎手'を保持に変更、hr側でjokey_idのmergeに使用するため
            # df.drop(["着差", "調教師", "性齢", "馬体重", '馬名', '人気', '着順', 'race_name'],axis=1, inplace=True)
            df.drop(["着差", "調教師", "性齢", "馬体重", '馬名','race_name'],axis=1, inplace=True)

            df["date"] = pd.to_datetime(df["date"], format="%Y年%m月%d日")

            #月齢変換
            def chenge_day(x):
                Y=x[:4]
                M=x[5:7]
                D=x[8:10]
                return ((((int(Y)-2009)%19)*11+int(M)+int(D))%30)
            df['geturei'] = df['date'].map(lambda x: chenge_day(str(x)))

            #日付の周期化
            df["生年月日"] = pd.to_datetime(df["生年月日"], format="%Y年%m月%d日")

            def trigonometric(x,T):
                if T == 'M':
                    num=int(x[5:7])
                    sin = np.sin(2 * np.pi * num/12)
                    cos = np.cos(2 * np.pi * num/12)
                elif T == 'D':
                    num=int(x[8:10])
                    sin = np.sin(2 * np.pi * num/31)
                    cos = np.cos(2 * np.pi * num/31)
                else:
                    num = int(x.strftime ( '%j' ))
                    sin = np.sin(2 * np.pi * num/366)
                    cos = np.cos(2 * np.pi * num/366)
                return sin,cos

            df['sin_birth']= df['生年月日'].map(lambda x: trigonometric((x),'T')).str[0]
            df['cos_birth']= df['生年月日'].map(lambda x: trigonometric((x),'T')).str[1]
            df['sin_date']= df["date"].map(lambda x: trigonometric((x),'T')).str[0]
            df['cos_date']= df["date"].map(lambda x: trigonometric((x),'T')).str[1]

            #開催場所
            df['place'] = df.index.map(lambda x:str(x)[4:6])

        #6/6出走数追加
            df['n_horses'] = df.index.map(df.index.value_counts())

            self.data_p = df

        #カテゴリ変数の処理
        def process_categorical(self,past_race_num):
            #複数列の値でカテゴリ変数化する場合は、concatする
            # place_= kai_ = day_ = jockey_ = []
            place_, kai_, day_, jockey_ = [], [], [], []
            for i in range(past_race_num):
                place_.append('self.data_pe["place_{}p"]'.format(i+1))
                kai_.append('self.data_pe["kai_{}p"]'.format(i+1))
                day_.append('self.data_pe["day_{}p"]'.format(i+1))
                jockey_.append('self.data_pe["jockey_id_{}p"]'.format(i+1))

            place_.append('self.data_pe["place"]')
            self.le_place = eval('LabelEncoder().fit(pd.concat(['+','.join(place_)+']).unique())')
            self.le_kai = eval('LabelEncoder().fit(pd.concat(['+','.join(kai_)+']).unique())')
            self.le_day = eval('LabelEncoder().fit(pd.concat(['+','.join(day_)+']).unique())')
            self.le_jockey = eval('LabelEncoder().fit(pd.concat(['+','.join(jockey_)+']).unique())')

            self.le_horse = LabelEncoder().fit(self.data_pe['horse_id'].unique())
            self.le_class = LabelEncoder().fit(pd.concat([self.data_pe['class'],self.data_pe['Past_class']]).unique())
            # self.le_jockey = LabelEncoder().fit(pd.concat([self.data_pe['jockey_id'],self.data_pe['past_jockey']]).unique())
            # self.le_jockey = LabelEncoder().fit(self.data_pe['jockey_id'].unique())

            self.le_trainer = LabelEncoder().fit(self.data_pe['調教師_id'].unique())
            self.le_horseowner = LabelEncoder().fit(self.data_pe['馬主_id'].unique())
            self.le_producer = LabelEncoder().fit(self.data_pe['生産者_id'].unique())
            self.le_relative1 = LabelEncoder().fit(self.data_pe['近親馬_id1'].unique())
            self.le_relative2 = LabelEncoder().fit(self.data_pe['近親馬_id2'].unique())

            super().process_categorical(self.le_horse, self.le_jockey, self.le_class, self.le_place, self.data_pe\
                                       ,self.le_trainer,self.le_horseowner,self.le_producer,self.le_relative1,\
                                        self.le_relative2,self.le_kai,self.le_day,past_race_num)
    class HorseInfo:
        def __init__(self, horse_info):
            self.horse_info = horse_info[['生年月日','調教師_id','馬主_id','生産者_id','近親馬_id1','近親馬_id2']]
            self.horse_info_p = pd.DataFrame()

        @classmethod
        def read_pickle(cls, path_list):
            df = pd.read_pickle(path_list[0])
            for path in path_list[1:]:
                df = update_data(df, pd.read_pickle(path))
            return cls(df)
    class HorseResults:
        def __init__(self, horse_results):
            #self.horse_results = horse_results[['日付', '着順', '賞金', '着差', '通過', '開催', '距離']]
            # self.horse_results = horse_results[['日付', '着順', '賞金', '着差', '通過', '開催', '距離','タイム','上り','馬場','斤量','騎手']]
            self.horse_results = horse_results[['日付', '着順', '賞金', '着差', '通過', '開催',\
                                                '距離','タイム','上り','馬場','斤量','騎手','天気','R','頭数',\
                                                '枠番','馬番','オッズ','人気','ペース','馬体重']]
            # self.preprocessing()
            self.horse_results_p = pd.DataFrame()

        @classmethod
        def read_pickle(cls, path_list):
            df = pd.read_pickle(path_list[0])
            for path in path_list[1:]:
                df = update_data(df, pd.read_pickle(path))
            return cls(df)

        @staticmethod
        def scrape(horse_id_list,flg):
            print("if you want get resuts data, True. else info_data False")
            # inpt = input("if you want get resuts data, True. else info_data False(y/n)\n")
            # # if inpt != 'y':
            # #     return print('Please choise', file=sys.stderr)

            """
            馬の過去成績データをスクレイピングする関数
            Parameters:
            ----------
            horse_id_list : list
                馬IDのリスト
            Returns:
            ----------
            horse_results_df : pandas.DataFrame
                全馬の過去成績データをまとめてDataFrame型にしたもの
            """

            #horse_idをkeyにしてDataFrame型を格納
            horse_results = {}
            horse_info ={}
            for horse_id in tqdm(horse_id_list):
                try:
                    time.sleep(1)
                    url = 'https://db.netkeiba.com/horse/' + horse_id

                    if flg:

                        df = pd.read_html(url)[3]
                        #受賞歴がある馬の場合、3番目に受賞歴テーブルが来るため、4番目のデータを取得する
                        if df.columns[0]=='受賞歴':
                            df = pd.read_html(url)[4]
                        df.index = [horse_id] * len(df)
                        horse_results[horse_id] = df

                    else:
                        #bace_info
                        idf = pd.read_html(url)[1]
                        Tidf = idf.T
                        headers = Tidf.iloc[0]
                        info_df  = pd.DataFrame(Tidf.values[1:], columns=headers)
                        info_df.index =  [horse_id]
                        #id henkan
                        html = requests.get(url)
                        html.encoding = "EUC-JP"
                        soup = BeautifulSoup(html.text, "html.parser")
                        all_id = soup.find("table", attrs={"summary": "のプロフィール"}).find_all(
                        "a", attrs={"href": re.compile("/\w")})

                        inf_horse_id1=inf_horse_id2=inf_owner_id=""
                        for a in all_id:
                            if "trainer" in str(a):
                                inf_trainer_id = re.findall(r"\w+", a["href"])
                                if len(inf_trainer_id) ==1:
                                     info_df['調教師_id']=np.nan
                                else:
                                    info_df['調教師_id']=inf_trainer_id[1]
                            elif "owner" in str(a):
                                if inf_owner_id=="":
                                    inf_owner_id = re.findall(r"\w+", a["href"])
                                    if len(inf_owner_id) ==1:
                                        info_df['馬主_id']=np.nan
                                    else:
                                        info_df['馬主_id']=inf_owner_id[1]
                            elif "breeder" in str(a):
                                inf_breeder_id = re.findall(r"\w+", a["href"])
                                if len(inf_breeder_id) ==1:
                                    info_df['生産者_id']=np.nan
                                else:
                                    info_df['生産者_id']=inf_breeder_id[1]
                            elif "horse/result" in str(a):
                                inf_horse_result_id = re.findall(r"\w+", a["href"])
                                if len(inf_horse_result_id) ==2:
                                    info_df['通算成績_id']=np.nan
                                else:
                                    info_df['通算成績_id']=inf_horse_result_id[2]
                            elif "race" in str(a):
                                inf_race_id = re.findall(r"\w+", a["href"])
                                if len(inf_race_id) ==1:
                                    info_df['主な勝鞍_id']=np.nan
                                else:
                                    info_df['主な勝鞍_id']=inf_race_id[1]
                            elif "horse" in str(a):
                                if inf_horse_id1=="":
                                    inf_horse_id1=re.findall(r"\w+", a["href"])
                                    if len(inf_horse_id1) ==1:
                                        info_df['近親馬_id1']=np.nan
                                    else:
                                        info_df['近親馬_id1']=inf_horse_id1[1]
                                else:
                                    inf_horse_id2=re.findall(r"\w+", a["href"])
                                    if len(inf_horse_id2) ==1:
                                        info_df['近親馬_id2']=np.nan
                                    else:
                                        info_df['近親馬_id2']=inf_horse_id2[1]
                        horse_info[horse_id] = info_df 

                except IndexError:
                    continue
                except Exception as e:
                    print(e)
                    break
                except:
                    break

            #pd.DataFrame型にして一つのデータにまとめる
            if flg:
                horse_results_df = pd.concat([horse_results[key] for key in horse_results])
                return horse_results_df
            else:
                horse_info_df = pd.concat([horse_info[key] for key in horse_info])
                return horse_info_df


        def preprocessing(self):
            df = self.horse_results.copy()

            # 開催場所をidに変換するための辞書型
            place_dict = {
                '札幌':'01',  '函館':'02',  '福島':'03',  '新潟':'04',  '東京':'05', 
                '中山':'06',  '中京':'07',  '京都':'08',  '阪神':'09',  '小倉':'10'
            }

            # place_dict = {
            #     '札幌':'01',  '函館':'02',  '福島':'03',  '新潟':'04',  '東京':'05', 
            #     '中山':'06',  '中京':'07',  '京都':'08',  '阪神':'09',  '小倉':'10',
            #     '大井':'25',  '浦和':'12',  '笠松':'13',  '名古屋':'14',  '園田':'15',
            #     '高知':'16',  '佐賀':'17',  '川崎':'18',  '盛岡':'19',  '水沢':'20',
            #     '船橋':'21',  '姫路':'22',  '門別':'23',  '金沢':'24'
            # }

            #レースタイプをレース結果データと整合させるための辞書型
            race_type_dict = {
                '芝': '芝', 'ダ': 'ダート', '障': '障害'
            }

            # 着順に数字以外の文字列が含まれているものを取り除く
            df['着順'] = pd.to_numeric(df['着順'], errors='coerce')
            df.dropna(subset=['着順'], inplace=True)
            df['着順'] = df['着順'].astype(int)

            df["date"] = pd.to_datetime(df['日付'])
            df.drop(['日付'], axis=1, inplace=True)

            #月齢変換
            def chenge_day(x):
                Y=x[:4]
                M=x[5:7]
                D=x[8:10]
                return ((((int(Y)-2009)%19)*11+int(M)+int(D))%30)

            df['geturei'] = df['date'].map(lambda x: chenge_day(str(x)))

            def trigonometric(x,T):
                if T == 'M':
                    num=int(x[5:7])
                    sin = np.sin(2 * np.pi * num/12)
                    cos = np.cos(2 * np.pi * num/12)
                elif T == 'D':
                    num=int(x[8:10])
                    sin = np.sin(2 * np.pi * num/31)
                    cos = np.cos(2 * np.pi * num/31)
                else:
                    num = int(x.strftime ( '%j' ))
                    sin = np.sin(2 * np.pi * num/366)
                    cos = np.cos(2 * np.pi * num/366)
                return sin,cos

            df['sin_date']= df["date"].map(lambda x: trigonometric((x),'T')).str[0]
            df['cos_date']= df["date"].map(lambda x: trigonometric((x),'T')).str[1]

            #賞金のNaNを0で埋める
            df['賞金'].fillna(0, inplace=True)

            #1着の着差を0にする
            df['着差'] = df['着差'].map(lambda x: 0 if x<0 else x)

            #レース展開データ
            #n=1: 最初のコーナー位置, n=4: 最終コーナー位置
            def corner(x, n):
                if type(x) != str:
                    return x
                elif n==4:
                    return int(re.findall(r'\d+', x)[-1])
                elif n==1:
                    return int(re.findall(r'\d+', x)[0])
            df['first_corner'] = df['通過'].map(lambda x: corner(x, 1))
            df['final_corner'] = df['通過'].map(lambda x: corner(x, 4))

            df['final_to_rank'] = df['final_corner'] - df['着順']
            df['first_to_rank'] = df['first_corner'] - df['着順']
            df['first_to_final'] = df['first_corner'] - df['final_corner']

    #独自追加--------------------------------------------------------
            def create_kai_day(x,n):
                try:
    #                 if x is np.nan:
    #                     return x
                    #任意の数字を含むかつ前方の数値を取得する場合
                    if bool(re.search(r'\d', x)) and n==1:
                        return str(re.findall(r'^[0-9]+',x)[0]).zfill(2)
                    elif bool(re.search(r'\d', x)) and n==2:
                        return  str(re.findall(r'[0-9]+$',x)[0]).zfill(2)
                except Exception as e:
                    return x

            #開催場所から開催回と開催日を抽出
            df['kai'] = df['開催'].map(lambda x: create_kai_day(x,1))
            df['day'] = df['開催'].map(lambda x: create_kai_day(x,2))        

            #カラム名の変更(スピード指数のマージのため)
            df.rename(columns={"馬場": "ground_state"},inplace=True)

    #--------------------------------------------------------

            #開催場所(地方や海外データは11に変換)
            df['place'] = df['開催'].str.extract(r'(\D+)')[0].map(place_dict).fillna('11')
            #race_type
            df['race_type'] = df['距離'].str.extract(r'(\D+)')[0].map(race_type_dict)
            #距離は10の位を切り捨てる
            df['course_len'] = df['距離'].str.extract(r'(\d+)').astype(int) // 100
            #インデックス名を与える
            df.index.name = 'horse_id'

    #独自追加--------------------------------------------------------
            #タイムデータ
            def race_time(x):
                try:
                    if type(x) != str:
                        return x
                    elif  x.count(':') == 2 :
                        m, s, l = str(x).split(':')
                        return (int(m) * 60 + float(s) + float(l))
                    elif  x.count('.') == 2 :
                        m, s, l = str(x).split('.')
                        return (int(m) * 60 + float(s) + float(l))
                    else:
                        m, s = str(x).split(':')
                        return ((int(m) * 60 + float(s)))
                except Exception as e:
                    print(x)
                    print(type(x))
                    print(e)

            df['time'] = df['タイム'].map(lambda x: race_time(x))

            df["weather"]=df["天気"]

            # 馬体重を体重と体重変化に分ける
            df["体重"] = df["馬体重"].str.split("(", expand=True)[0]
            df["体重変化"] = df["馬体重"].str.split("(", expand=True)[1].str[:-1]

            #errors='coerce'で、"計不"など変換できない時に欠損値にする
            df['体重'] = pd.to_numeric(df['体重'], errors='coerce')
            df['体重変化'] = pd.to_numeric(df['体重変化'], errors='coerce')

            #ペース
            df['first_pace'] = df["ペース"].str.split("-", expand=True)[0]
            df['latter_pace'] = df["ペース"].str.split("-", expand=True)[1]
            df['first_pace'] = df['first_pace'].astype('float')
            df['latter_pace'] = df['latter_pace'].astype('float')

            df.drop(['開催','天気','馬体重','通過','距離','タイム','ペース'], axis=1, inplace=True)
    #--------------------------------------------------------
            # 障害および地方レースを削除
            # df =  df[~(df['place']=='11')]

            self.horse_results_p = df

            '''
            #target_list(horse_idごとに平均値を求めたい列を指定する)
            #addition_list(horse_idごとに平均値を求める和条件とする列を指定する)
            #multiplication_list(horse_idごとに平均値を求める共通条件とする列を指定する)

            '''
            self.target_list = ['着順', '賞金', '着差', 'first_corner', 'final_corner',
                                'first_to_rank', 'first_to_final','final_to_rank','上り','time']
            self.addition_list = ['course_len', 'race_type', 'place']
            self.multiplication_list = ['horse_id','course_len', 'race_type', 'place']

        #n_samplesレース分馬ごとに平均する
        def average(self,horse_id_list, date, n_samples='all'):

            #resultから得た日付をもとにhorse_idで成績を取り出す
            target_df = self.horse_results_p.query('index in @horse_id_list')
            # print('average_date:',date)

            #過去何走分取り出すか指定
            if n_samples == 'all':
                filtered_df = target_df[target_df['date'] < date]
            elif n_samples > 0:
                filtered_df = target_df[target_df['date'] < date].\
                    sort_values('date', ascending=False).groupby(level=0).head(n_samples)
            else:
                raise Exception('n_samples must be >0')

            '''        
            #集計して辞書型に入れる
            #ここで平均されるデータはtarget_listで指定したもの
            #non_categoryとしているのは、この馬の個別要素によらない平均のため
            '''
            self.average_dict = {}
            self.average_dict['non_category'] = filtered_df.groupby(level=0)[self.target_list].mean()\
                .add_suffix('_{}R'.format(n_samples))

            '''
            #ここでaverage_dictにカテゴリ別に集計した値が代入される
            #意味はある馬のcolumn(cource_lenなど)ごとにのtarget_listで指定した5R平均
            #細分化したい場合はcolumnに追加する。(着順_cource_len_5Rなどになる)
            #columnに指定した列ごとに平均を出しaverage_dictに新たに追加する
            #指定できるのはhorse_resultsにある列だけ
            #今はないけどjokey_id別も出しておきたい
            '''
            for column in self.addition_list:
                self.average_dict[column] = filtered_df.groupby(['horse_id', column])\
                    [self.target_list].mean().add_suffix('_{}_{}R'.format(column, n_samples))

            #組み合わせを自分で設定
            self.average_dict['my_category'] = filtered_df.groupby(self.multiplication_list)\
                    [self.target_list].mean().add_suffix('_my_{}R'.format(n_samples))

            #6/6追加: 馬の出走間隔追加のために、全レースの日付を変数latestに格納
            if n_samples == 5:
                self.latest = filtered_df.groupby('horse_id')['date'].max().rename('latest')

        def merge(self, results, date, n_samples='all'):

            '''
            リークについて
                resultの全日程から一意の日付でhorse_resultsを抽出する
                抽出したから過去走をn_samples分さかのぼり平均する
                その値をconcatでresultsを一から再構築している
                リークした情報ではなく過去走の平均をとれている
            '''
            #日付でresultsを絞り込み、出走馬のhorse_idを取り出し関数に入れる
            df = results[results['date']==date]
            horse_id_list = df['horse_id']
            self.average(horse_id_list, date, n_samples)
            # print('merge_date:',date)

            if n_samples == 5:
            #merge処理する際にindex振りなおしてしまうので避難
                df.reset_index(inplace= True)  
                df = df.rename(columns={'index': 'race_id'})

                '''
                スピード指数を追加
                指定した日付から過去全体の平均タイムを算出し、各resultsごとに計算する
                '''
                # r_past_df = results.query('date < @date')
                r_past_df = results[results['date']<date]

                # print('margelen:',len(r_past_df))

                r_past_df1 = pd.DataFrame(r_past_df.groupby\
                                                (['place','race_type','course_len','ground_state'])['time'].mean())

                r_past_df1['Distance_index'] = 1/r_past_df1['time']*100

                r_past_df1.rename(columns={'time': 'mean_time'},inplace=True)

                r_past_df2=pd.merge(r_past_df,r_past_df1, how='left',on= \
                                               ["place","race_type","ground_state","course_len"])
                #スピード指数計算
                r_past_df2['Speed_index'] = (r_past_df2['mean_time']-r_past_df2['time'])\
                                    *r_past_df2['Distance_index']+(r_past_df2['斤量']-55)*0.2 

                #過去走のみになっているため、ここで平均値をとり指定した日付のスピード指数とする
                horse_id_SPI = r_past_df2.groupby(['horse_id'])['Speed_index'].mean()
                # print(horse_id_SPI)

                merged_df = pd.merge(df,horse_id_SPI, on=['horse_id'], how='left')

                #個別要素によらない平均値をマージ
                merged_df = merged_df.merge(self.average_dict['non_category'], left_on='horse_id',
                                     right_index=True, how='left')
                #複数個別要素のand条件による平均値をマージ
                merged_df = merged_df.merge(self.average_dict['my_category'],
                                            on=self.multiplication_list, how='left')       
                #個別要素別の平均値をマージ
                for column in self.addition_list:
                        merged_df = merged_df.merge(self.average_dict[column], left_on=['horse_id', column],
                                                    right_index=True, how='left')

                #6/6追加：馬の出走間隔追加のために、前レースの日付を変数latestに格納
                merged_df = merged_df.merge(self.latest, left_on='horse_id',
                                            right_index=True, how='left')

                #前走class
                past_class = r_past_df.sort_values('date', ascending=False).groupby('horse_id').head(1)
                past_class.rename(columns={'class': 'past_class'},inplace=True)
                merged_df = merged_df.merge(past_class[['horse_id','past_class']], on='horse_id', how='left')
            else:
                #個別要素によらない平均値をマージ
                merged_df = df.merge(self.average_dict['non_category'], left_on='horse_id',
                                     right_index=True, how='left')
                #複数個別要素のand条件による平均値をマージ
                merged_df = merged_df.merge(self.average_dict['my_category'],
                                            on=self.multiplication_list, how='left')       
                #個別要素別の平均値をマージ
                for column in self.addition_list:
                        merged_df = merged_df.merge(self.average_dict[column], left_on=['horse_id', column],
                                                    right_index=True, how='left')

            return merged_df

        def merge_all(self, results, n_samples='all',rt=None):

            if rt == None:
                date_list = results['date'].unique()

            else:
                #ここでのdata_pは出馬テーブルのみのデータ
                #resultsについては、出馬テーブルと過去resultsがくっついたデータになっている
                date_list = rt.data_p['date'].unique()
                # date_list = rt.data_p.sort_values('date',ascending = True)['date'].unique()
                # print('date_list_date:',date_list)
                # print('results過去全レース:',results.index.nunique())

            # jockey_idをresultから取得。hrのprossesingの時のみ実行する        
            if n_samples == 5:
                self.horse_results_p = self.horse_results_p.reset_index()
                jockey_df = results[['騎手','jockey_id']].drop_duplicates()
                # print(jockey_df[jockey_df.index=='202207040410'])
                self.horse_results_p = self.horse_results_p.merge(jockey_df, how='left', on = '騎手')
                self.horse_results_p = self.horse_results_p.set_index('horse_id')
                # print(set(self.horse_results_p.columns.tolist()))

            merged_df = pd.concat([self.merge(results, date, n_samples) for date in tqdm(date_list)])

            return merged_df
    class jockey_results:
        def __init__(self, jockey_results):
            self.jockey_results = jockey_results\
            [['年度', '順位', '1着', '2着', '3着', '着外', '重賞_出走', '重賞_勝利', '特別_出走', '特別_勝利',\
              '平場_出走', '平場_勝利', '芝_出走', '芝_勝利', 'ダート_出走', 'ダート_勝利', '勝率', '連対率',\
              '複勝率', '収得賞金(万円)']]
            self.jockey_results_p = pd.DataFrame()

        @classmethod
        def read_pickle(cls, path_list):
            df = pd.read_pickle(path_list[0])
            for path in path_list[1:]:
                df = update_data(df, pd.read_pickle(path))
            return cls(df)

        def converted_multi_columns(df):
            return [col[0]  if col[0] == col[1] else col[0]+'_'+col[1] for col in df.columns.values]

        @classmethod
        def scrape(self,jockey_id_list):
            jockey_results ={}

            for jockey_id in tqdm(jockey_id_list):
                try:

                    time.sleep(1)
                    url = 'https://db.netkeiba.com/jockey/result/'+ jockey_id
                    df = pd.read_html(url)[0]
                    #multi＿columを変換
                    df.columns = self.converted_multi_columns(df)
                    df.index = [jockey_id]* len(df)

                    #一旦辞書型に保存する
                    jockey_results[jockey_id] = df 

                except IndexError:
                    continue
                except ValueError:
                    print('No tables found')
                    df=pd.DataFrame(index=[jockey_id],columns=\
                    ['年度', '順位', '1着', '2着', '3着', '着外', '重賞_出走', '重賞_勝利', '特別_出走', '特別_勝利',\
                     '平場_出走', '平場_勝利', '芝_出走', '芝_勝利', 'ダート_出走', 'ダート_勝利', '勝率', '連対率',\
                     '複勝率', '収得賞金(万円)', '代表馬'])
                    df.fillna(0, inplace=True)
                    jockey_results[jockey_id]=df
                except Exception as e:
                    print(e)
                    break
                except:
                    break

            #pd.DataFrame型にして一つのデータにまとめる
            jockey_results_df = pd.concat([jockey_results[key] for key in jockey_results])

            return jockey_results_df
    class ShutubaTable(DataProcessor):
        def __init__(self, shutuba_tables):
            super(ShutubaTable, self).__init__()
            self.data = shutuba_tables

        @classmethod
        def read_pickle(cls, path_list):
            df = pd.read_pickle(path_list[0])
            for path in path_list[1:]:
                df = update_data(df, pd.read_pickle(path))
            return cls(df)

        @classmethod
        # def scrape(cls, race_id_list, date):
        def scrape(cls, race_id_day_dict,checkflg=True):

            data = pd.DataFrame()
            today = datetime.now().strftime('%Y年%m月%d日')
            # for race_id in tqdm(race_id_list):
            for race_id,date in tqdm(race_id_day_dict.items()):
                try:
                    # 今日よりも未来の日付の場合、レース情報が出そろっていないためスキップ
                    if checkflg:
                        if datetime.strptime(date, '%Y年%m月%d日')>datetime.strptime(today, '%Y年%m月%d日'):
                            print('skip_date_id',date,race_id)
                            continue
                    # print(race_id,date)
                    url = 'https://race.netkeiba.com/race/shutuba.html?race_id=' + race_id
                    df = pd.read_html(url)[0]
                    df = df.T.reset_index(level=0, drop=True).T

                    if checkflg:
                    #馬体重の発表がまたの場合
                        if df['馬体重(増減)'].isnull().any():
                            print('skip race id for not wight',date,race_id)
                            continue

                    html = requests.get(url)
                    html.encoding = "EUC-JP"
                    soup = BeautifulSoup(html.text, "html.parser")

                    texts = soup.find('div', attrs={'class': 'RaceData01'}).text
                    texts = re.findall(r'\w+', texts)
                    texts2 = soup.find('div', attrs={'class': 'RaceData02'}).text
                    texts2 = re.findall(r'\w+', texts2)
                    for text in texts:
                        if 'm' in text:
                            df['course_len'] = [int(re.findall(r'\d+', text)[0])] * len(df)
                        #回り情報の取得追加_resultsとは異なる   
                        if text in ['左', '右', '外', '内', '直', '直線']:
                            df["race_round"] = [text] * len(df)
                        if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                            df["weather"] = [text] * len(df)
                        if text in ["良", "稍重", "重"]:
                            df["ground_state"] = [text] * len(df)
                        if '不' in text:
                            df["ground_state"] = ['不良'] * len(df)
                        # 2020/12/13追加
                        if '稍' in text:
                            df["ground_state"] = ['稍重'] * len(df)
                        if '芝' in text:
                            df['race_type'] = ['芝'] * len(df)
                        if '障' in text:
                            df['race_type'] = ['障害'] * len(df)
                        if 'ダ' in text:
                            df['race_type'] = ['ダート'] * len(df)
                    df['date'] = [date] * len(df)
            #独自追加--------------------------------------------------------
                    bftext = aftext =np.nan
                    anytext =np.nan
                    for text2 in texts2:
                        if any(anytext in text2 for anytext in ("歳")):
                            bftext = text2
                        if any(anytext in text2 for anytext in ("新馬","未出走","勝","オープン","以上","以下","万下")):
                            aftext = text2 
                    classtext = bftext+aftext
                    df["class"] = [classtext] * len(df)

                    #レース名取得を追加
                    race_name = soup.find('div', attrs={'class': 'RaceName'}).text
                    race_name = re.findall(r'\w+', race_name)
                    # df["race_name"] = [race_name][0] * len(df)
                    #変更　レース名に"-"がある場合indexエラーとなるため
                    race_name = "-".join(race_name)
                    df["race_name"] = [race_name] * len(df)
            #--------------------------------------------------------

                    # horse_id
                    horse_id_list = []
                    horse_td_list = soup.find_all("td", attrs={'class': 'HorseInfo'})
                    for td in horse_td_list:
                        horse_id = re.findall(r'\d+', td.find('a')['href'])[0]
                        horse_id_list.append(horse_id)
                    # jockey_id
                    jockey_id_list = []
                    jockey_td_list = soup.find_all("td", attrs={'class': 'Jockey'})
                    for td in jockey_td_list:
                        jockey_id = re.findall(r"recent/(\w+)", td.find('a')['href'])[0]
                        # print(jockey_id)
                        jockey_id_list.append(jockey_id)
                    df['horse_id'] = horse_id_list
                    df['jockey_id'] = jockey_id_list

                    df.index = [race_id] * len(df)
                    # data = data.append(df)
                    data = pd.concat([data, df])
                    time.sleep(1)

                #存在しないrace_idを飛ばす
                except IndexError:
                    continue
                #wifiの接続が切れた時などでも途中までのデータを返せるようにする
                except Exception as e:
                    print(e)
                    break
                #Jupyterで停止ボタンを押した時の対処
                except:
                    break

            return cls(data)

        #前処理            
        def preprocessing(self):
            df = self.data_j.copy()

            df["性"] = df["性齢"].map(lambda x: str(x)[0])
            df["年齢"] = df["性齢"].map(lambda x: str(x)[1:]).astype(int)

            # 馬体重を体重と体重変化に分ける
            # 馬体重未発表の場合の処理
            if any(df['馬体重(増減)'].isnull()):
                df["体重"] = np.nan
                df["体重変化"] = np.nan
                print('not found ''horse wight''')
            else:
                df = df[df["馬体重(増減)"] != '--']
                df["体重"] = df["馬体重(増減)"].str.split("(", expand=True)[0].astype(int)
                # df["体重"] = df["馬体重(増減)"].str.split("(", expand=True)[0]
                df["体重変化"] = df["馬体重(増減)"].str.split("(", expand=True)[1].str[:-1]
                # 2020/12/13追加：増減が「前計不」などのとき欠損値にする
                # df['体重'] = pd.to_numeric(df['体重'], errors='coerce')
                df['体重変化'] = pd.to_numeric(df['体重変化'], errors='coerce')

            df["date"] = pd.to_datetime(df["date"], format="%Y年%m月%d日")

            # 枠未決定の場合の処理(重賞レースは決定が早いのでanyで判定)
            if any(df['枠'].isnull()):
                df['枠'] = np.nan
                df['馬番'] = np.nan
                print('not found ''waku'' and ''horse number''')
            else:
                df['枠'] = df['枠'].astype(int)
                df['馬番'] = df['馬番'].astype(int)
            df['斤量'] = df['斤量'].astype(int)

            df['place'] = df.index.map(lambda x:str(x)[4:6])

            #6/6出走数追加
            df['n_horses'] = df.index.map(df.index.value_counts())
            # 距離は10の位を切り捨てる
            df["course_len"] = df["course_len"].astype(float) // 100

    #独自追加--------------------------------------------------------        
            #classの数字を半角にして頭の文字を取り除く
            #st前日取得の際にclassが未確定のため追加
            df['class'].fillna('9未確定',inplace=True)
            df = df[~df['class'].str.contains('障害')]
            trans_table = str.maketrans({"１":"1", "２":"2", "３":"3", "４":"4", "５":"5"})
            df['class'] = df['class'].map(lambda x: x.translate(trans_table))
            df['class'] = df['class'].map(lambda x: x[x.index(re.findall(r'\d',x)[0]):])

            df["date"] = pd.to_datetime(df["date"], format="%Y年%m月%d日")

            #月齢変換
            def chenge_day(x):
                Y=x[:4]
                M=x[5:7]
                D=x[8:10]
                return ((((int(Y)-2009)%19)*11+int(M)+int(D))%30)
            df['geturei'] = df['date'].map(lambda x: chenge_day(str(x)))

            #日付の周期化
            df["生年月日"] = pd.to_datetime(df["生年月日"], format="%Y年%m月%d日")

            def trigonometric(x,T):
                if T == 'M':
                    num=int(x[5:7])
                    sin = np.sin(2 * np.pi * num/12)
                    cos = np.cos(2 * np.pi * num/12)
                elif T == 'D':
                    num=int(x[8:10])
                    sin = np.sin(2 * np.pi * num/31)
                    cos = np.cos(2 * np.pi * num/31)
                else:
                    num = int(x.strftime ( '%j' ))
                    sin = np.sin(2 * np.pi * num/366)
                    cos = np.cos(2 * np.pi * num/366)
                return sin,cos

            df['sin_birth']= df['生年月日'].map(lambda x: trigonometric((x),'T')).str[0]
            df['cos_birth']= df['生年月日'].map(lambda x: trigonometric((x),'T')).str[1]
            df['sin_date']= df["date"].map(lambda x: trigonometric((x),'T')).str[0]
            df['cos_date']= df["date"].map(lambda x: trigonometric((x),'T')).str[1]

            #発走中止や除外を除く
            df = df[df['印'].isnull()]
            df.drop(['Unnamed: 9_level_1', '厩舎', '印', 'メモ', '登録','race_name', '馬体重(増減)', '馬名', '性齢'], axis=1, inplace=True)

            self.data_p = df.rename(columns={'枠': '枠番'})
    class Peds:
        def __init__(self, peds):
            self.peds = peds
            self.peds_e = pd.DataFrame() #after label encoding and transforming into category

        @classmethod
        def read_pickle(cls, path_list):
            df = pd.read_pickle(path_list[0])
            for path in path_list[1:]:
                df = update_data(df, pd.read_pickle(path))
            return cls(df)

        @staticmethod
        def scrape(horse_id_list):

            # inpt = input("Have you set two receiving variables?(y/n)\n")
            # if inpt != 'y':
            #     return print('Please set two receiving variables', file=sys.stderr)

            p_name_df = pd.DataFrame()
            p_id_df = pd.DataFrame()
            for horse_id in tqdm(horse_id_list):
                try:
                    url = "https://db.netkeiba.com/horse/ped/" + horse_id
                    #メインとなるテーブルデータを取得
                    peds_df = pd.read_html(url)[0]

                    html = requests.get(url)
                    html.encoding = "EUC-JP"
                    soup = BeautifulSoup(html.text, "html.parser")

                    horse_id_list = []
                    idlist = []
                    peds_list = soup.find("table", attrs={"summary": "5代血統表"}).find_all(
                    "a", attrs={"href": re.compile("^/horse/\d")})
                    peds_id_df = peds_df

                    for a in peds_list:
                        horse_param = re.findall(r"\w+", a["href"])
                        #改行箇所で切り取り任意の文字列を取得する、改行コードがない場合-1が返され末尾一文字が消える
                        #horse,id,horsenameがそれぞれindexに入る
                        horse_param.extend(re.findall(r".*\w", a.text[:a.text.find('\n')]))
                        #dfから部分一致でidに置換
                        peds_id_df = peds_id_df.replace(".*"+horse_param[2]+".*", horse_param[1], regex=True)
                        idlist.append(horse_param)

                    def generater(peds_df):
                        get_peds_list = []
                        #dfから1~5代目のidと取得する
                        for i in range(2):
                            get_peds_list.append(peds_df.iloc[i*16][0])
                        for i in range(4):
                            get_peds_list.append(peds_df.iloc[i*8][1])
                        for i in range(8):
                            get_peds_list.append(peds_df.iloc[i*4][2])
                        for i in range(16):
                            get_peds_list.append(peds_df.iloc[i*2][3])
                        for i in range(32):
                            get_peds_list.append(peds_df.iloc[i][4])

                        return get_peds_list

                    peds_name_list = generater(peds_df)
                    peds_list = generater(peds_id_df)

                    #英数字以外の文字列の場合はnanに変換
                    for index, value in enumerate(peds_list):
                        if type(value) == str:
                            if re.compile('\W').search(value):
                                peds_list[index] = np.nan

                    pedsnamedf = pd.DataFrame(peds_name_list,columns=[horse_id]).T.add_prefix('peds_')
                    pedsdf = pd.DataFrame(peds_list,columns=[horse_id]).T.add_prefix('peds_')

                    p_name_df = pd.concat([p_name_df,pedsnamedf])
                    p_id_df = pd.concat([p_id_df,pedsdf])

                    time.sleep(1)
                except IndexError:
                    continue
                except Exception as e:
                    print(e)
                    break
                except:
                    break

            return p_name_df,p_id_df

        def encode(self,st=None):
            df = self.peds.copy()

            if st!=None:
                no_peds = st.data_cp[~st.data_cp['horse_id'].isin(df.index)]['horse_id']
                if len(no_peds)>0:
                    print('scrape peds at horse_id_list "no_peds"')
                    print(no_peds)
                    no_peds_name,no_peds_id = self.scrape(no_peds)
                    no_peds_name.to_pickle('pickle/syutuba/new_peds/peds_name_'+datetime.now().strftime('%m_%d')+'.pickle')
                    no_peds_id.to_pickle('pickle/syutuba/new_peds/peds_id_'+datetime.now().strftime('%m_%d')+'.pickle')
                    df = pd.concat([df,no_peds_id])
                    df.to_pickle('pickle/syutuba/latest/peds_id_latest.pickle')

            for column in df.columns:
                df[column] = LabelEncoder().fit_transform(df[column].fillna('Na'))
            self.peds_e = df.astype('category')
    class Return:
        def __init__(self, return_tables):
            self.return_tables = return_tables

        @classmethod
        def read_pickle(cls, path_list):
            df = pd.read_pickle(path_list[0])
            for path in path_list[1:]:
                df = update_data(df, pd.read_pickle(path))
            return cls(df)

        @staticmethod
        def scrape(race_id_day_dict):
        # def scrape(race_id_list):
            return_tables = {}
            # for race_id in tqdm(race_id_list):

            data = pd.DataFrame()
            today = datetime.now().strftime('%Y年%m月%d日')
            # for race_id in tqdm(race_id_list):
            for race_id,date in tqdm(race_id_day_dict.items()):
                try:
                    # 今日よりも未来の日付の場合、レース情報が出そろっていないためスキップ
                    #月曜には更新はいると考えて-2を指定
                    if datetime.strptime(date, '%Y年%m月%d日')\
                    >datetime.strptime(today, '%Y年%m月%d日')-timedelta(days=2):
                        print('skip_date_id',date,race_id)
                        continue

                    url = "https://db.netkeiba.com/race/" + race_id

                    #普通にスクレイピングすると複勝やワイドなどが区切られないで繋がってしまう。
                    #そのため、改行コードを文字列brに変換して後でsplitする
                    f = urlopen(url)
                    html = f.read()
                    html = html.replace(b'<br />', b'br')
                    dfs = pd.read_html(html)

                    #dfsの1番目に単勝〜馬連、2番目にワイド〜三連単がある
                    df = pd.concat([dfs[1], dfs[2]])

                    df.index = [race_id] * len(df)
                    return_tables[race_id] = df
                    time.sleep(1)
                except IndexError:
                    print(race_id)
                    continue
                except Exception as e:
                    print(e)
                    print(race_id)
                    break
                except:
                    print(race_id)
                    break

            #pd.DataFrame型にして一つのデータにまとめる
            return_tables_df = pd.concat([return_tables[key] for key in return_tables])
            return return_tables_df

        @property
        def fukusho(self):
            fukusho = self.return_tables[self.return_tables[0]=='複勝'][[1,2]]
            wins = fukusho[1].str.split('br', expand=True)[[0,1,2]]

            wins.columns = ['win_0', 'win_1', 'win_2']
            returns = fukusho[2].str.split('br', expand=True)[[0,1,2]]
            returns.columns = ['return_0', 'return_1', 'return_2']

            df = pd.concat([wins, returns], axis=1)
            for column in df.columns:
                df[column] = df[column].str.replace(',', '')
            return df.fillna(0).astype(int)

        @property
        def tansho(self):
            tansho = self.return_tables[self.return_tables[0]=='単勝'][[1,2]]

            # if tansho[1].str.contains('br').bool():
            #     splitdf = tansho[1].str.split('br', expand=True)[[0,1]]

            tansho.columns = ['win', 'return']

            for column in tansho.columns:
                tansho[column] = pd.to_numeric(tansho[column], errors='coerce')

            return tansho

        @property
        def umaren(self):
            umaren = self.return_tables[self.return_tables[0]=='馬連'][[1,2]]
            wins = umaren[1].str.split('-', expand=True)[[0,1]].add_prefix('win_')
            return_ = umaren[2].rename('return')  
            df = pd.concat([wins, return_], axis=1)        
            return df.apply(lambda x: pd.to_numeric(x, errors='coerce'))

        @property
        def umatan(self):
            umatan = self.return_tables[self.return_tables[0]=='馬単'][[1,2]]
            wins = umatan[1].str.split('→', expand=True)[[0,1]].add_prefix('win_')
            return_ = umatan[2].rename('return')  
            df = pd.concat([wins, return_], axis=1)        
            return df.apply(lambda x: pd.to_numeric(x, errors='coerce'))

        @property
        def wide(self):
            wide = self.return_tables[self.return_tables[0]=='ワイド'][[1,2]]
            wins = wide[1].str.split('br', expand=True)[[0,1,2]]
            wins = wins.stack().str.split('-', expand=True).add_prefix('win_')
            return_ = wide[2].str.split('br', expand=True)[[0,1,2]]
            return_ = return_.stack().rename('return')
            df = pd.concat([wins, return_], axis=1)
            return df.apply(lambda x: pd.to_numeric(x.str.replace(',',''), errors='coerce'))

        @property
        def sanrentan(self):
            rentan = self.return_tables[self.return_tables[0]=='三連単'][[1,2]]
            wins = rentan[1].str.split('→', expand=True)[[0,1,2]].add_prefix('win_')
            return_ = rentan[2].rename('return')
            df = pd.concat([wins, return_], axis=1) 
            return df.apply(lambda x: pd.to_numeric(x, errors='coerce'))

        @property
        def sanrenpuku(self):
            renpuku = self.return_tables[self.return_tables[0]=='三連複'][[1,2]]
            wins = renpuku[1].str.split('-', expand=True)[[0,1,2]].add_prefix('win_')
            return_ = renpuku[2].rename('return')
            df = pd.concat([wins, return_], axis=1) 
            return df.apply(lambda x: pd.to_numeric(x, errors='coerce'))
    class ModelEvaluator:
        def __init__(self, model, return_tables_path_list):
            self.model = model
            self.rt = Return.read_pickle(return_tables_path_list)
            self.fukusho = self.rt.fukusho
            self.tansho = self.rt.tansho
            self.umaren = self.rt.umaren
            self.umatan = self.rt.umatan
            self.wide = self.rt.wide
            self.sanrentan = self.rt.sanrentan
            self.sanrenpuku = self.rt.sanrenpuku
            #処理速度改善のために追加
            self.y_pred = pd.DataFrame()

        #3着以内に入る確率を予測
        def predict_proba(self, X, train=True, std=True, minmax=False):
            if train:
                proba = pd.Series(
                    self.model.predict_proba(X.drop(['単勝'], axis=1))[:, 1], index=X.index
                )
            else:
                proba = pd.Series(
                    self.model.predict_proba(X)[:, 1], index=X.index
                )
            if std:
                #レース内で標準化して、相対評価する。「レース内偏差値」みたいなもの。
                standard_scaler = lambda x: (x - x.mean()) / x.std()
                proba = proba.groupby(level=0).transform(standard_scaler)
            if minmax:
                #データ全体を0~1にする
                proba = (proba - proba.min()) / (proba.max() - proba.min())
            return proba

        #0か1かを予測
        def predict(self, X, threshold=0.5):
            #一度probaを求めたらあとは実行しない
            if len(self.y_pred)==0:
                print("y_pred_none")
                y_pred = self.predict_proba(X)
                self.y_pred = y_pred
                self.proba = self.y_pred
            else:
                # print("y_pred_exist")
                self.proba = self.y_pred
                y_pred = self.y_pred
            return [0 if p<threshold else 1 for p in y_pred]

        def score(self, y_true, X, train=True):
            return roc_auc_score(y_true, self.predict_proba(X,train))

        def feature_importance(self, X, n_display=20):
            importances = pd.DataFrame({"features": X.columns, 
                                        "importance": self.model.feature_importances_})
            return importances.sort_values("importance", ascending=False)[:n_display]

        def pred_table(self, X, threshold=0.5, bet_only=True):
            pred_table = X.copy()[['馬番', '単勝']]
            pred_table['pred'] = self.predict(X, threshold)
            pred_table['score'] = self.proba
            if bet_only:
                # return pred_table[pred_table['pred']==1][['馬番', '単勝', 'score']]
                #馬連馬単のシミュレーション時のエラー回避
                return pred_table[pred_table['pred']==1]
            else:
                return pred_table[['馬番', '単勝', 'score', 'pred']]

        def bet(self, race_id, kind, umaban, amount):
            if kind == 'fukusho':
                rt_1R = self.fukusho.loc[race_id]
                return_ = (rt_1R[['win_0', 'win_1', 'win_2']]==umaban).values * \
                    rt_1R[['return_0', 'return_1', 'return_2']].values * amount/100
                return_ = np.sum(return_)
            if kind == 'tansho':
                rt_1R = self.tansho.loc[race_id]
                return_ = (rt_1R['win']==umaban) * rt_1R['return'] * amount/100
            if kind == 'umaren':
                rt_1R = self.umaren.loc[race_id]
                return_ = (set(rt_1R[['win_0', 'win_1']]) == set(umaban)) \
                    * rt_1R['return']/100 * amount
            if kind == 'umatan':
                rt_1R = self.umatan.loc[race_id]
                return_ = (list(rt_1R[['win_0', 'win_1']]) == list(umaban))\
                    * rt_1R['return']/100 * amount
            if kind == 'wide':
                rt_1R = self.wide.loc[race_id]
                return_ = (rt_1R[['win_0', 'win_1']].\
                               apply(lambda x: set(x)==set(umaban), axis=1)) \
                    * rt_1R['return']/100 * amount
                return_ = return_.sum()
            if kind == 'sanrentan':
                rt_1R = self.sanrentan.loc[race_id]
                return_ = (list(rt_1R[['win_0', 'win_1', 'win_2']]) == list(umaban)) * \
                    rt_1R['return']/100 * amount
            if kind == 'sanrenpuku':
                rt_1R = self.sanrenpuku.loc[race_id]
                return_ = (set(rt_1R[['win_0', 'win_1', 'win_2']]) == set(umaban)) \
                    * rt_1R['return']/100 * amount
            if not (return_ >= 0):
                    return_ = amount
            return return_

        def fukusho_return(self, X, threshold=0.5):
            pred_table = self.pred_table(X, threshold)
            n_bets = len(pred_table)

            return_list = []
            for race_id, preds in pred_table.groupby(level=0):
                return_list.append(np.sum([
                    self.bet(race_id, 'fukusho', umaban, 1) for umaban in preds['馬番']
                ]))
            return_rate = np.sum(return_list) / n_bets
            std = np.std(return_list) * np.sqrt(len(return_list)) / n_bets
            n_hits = np.sum([x>0 for x in return_list])
            return n_bets, return_rate, n_hits, std

        def tansho_return(self, X, threshold=0.5):
            pred_table = self.pred_table(X, threshold)
            # self.sample = pred_table
            n_bets = len(pred_table)

            return_list = []
            for race_id, preds in pred_table.groupby(level=0):
                return_list.append(
                    np.sum([self.bet(race_id, 'tansho', umaban, 1) for umaban in preds['馬番']])
                )

            std = np.std(return_list) * np.sqrt(len(return_list)) / n_bets

            n_hits = np.sum([x>0 for x in return_list])
            return_rate = np.sum(return_list) / n_bets
            return n_bets, return_rate, n_hits, std

        def tansho_return_proper(self, X, threshold=0.5):
            pred_table = self.pred_table(X, threshold)
            n_bets = len(pred_table)

            return_list = []
            for race_id, preds in pred_table.groupby(level=0):
                return_list.append(
                    np.sum(preds.apply(lambda x: self.bet(
                        race_id, 'tansho', x['馬番'], 1/x['単勝']), axis=1)))

            bet_money = (1 / pred_table['単勝']).sum()

            std = np.std(return_list) * np.sqrt(len(return_list)) / bet_money

            n_hits = np.sum([x>0 for x in return_list])
            return_rate = np.sum(return_list) / bet_money
            return n_bets, return_rate, n_hits, std

        def umaren_box(self, X, threshold=0.5, n_aite=5):
            pred_table = self.pred_table(X, threshold)
            n_bets = 0

            return_list = []
            for race_id, preds in pred_table.groupby(level=0):
                return_ = 0
                preds_jiku = preds.query('pred == 1')
                if len(preds_jiku) == 1:
                    continue
                elif len(preds_jiku) >= 2:
                    for umaban in combinations(preds_jiku['馬番'], 2):
                        return_ += self.bet(race_id, 'umaren', umaban, 1)
                        n_bets += 1
                    return_list.append(return_)

            std = np.std(return_list) * np.sqrt(len(return_list)) / n_bets

            n_hits = np.sum([x>0 for x in return_list])
            return_rate = np.sum(return_list) / n_bets
            return n_bets, return_rate, n_hits, std

        def umatan_box(self, X, threshold=0.5, n_aite=5):
            pred_table = self.pred_table(X, threshold, bet_only = False)
            n_bets = 0

            return_list = []
            for race_id, preds in pred_table.groupby(level=0):
                return_ = 0
                preds_jiku = preds.query('pred == 1')
                if len(preds_jiku) == 1:
                    continue   
                elif len(preds_jiku) >= 2:
                    for umaban in permutations(preds_jiku['馬番'], 2):
                        return_ += self.bet(race_id, 'umatan', umaban, 1)
                        n_bets += 1
                return_list.append(return_)

            std = np.std(return_list) * np.sqrt(len(return_list)) / n_bets

            n_hits = np.sum([x>0 for x in return_list])
            return_rate = np.sum(return_list) / n_bets
            return n_bets, return_rate, n_hits, std

        def wide_box(self, X, threshold=0.5, n_aite=5):
            pred_table = self.pred_table(X, threshold, bet_only = False)
            n_bets = 0

            return_list = []
            for race_id, preds in pred_table.groupby(level=0):
                return_ = 0
                #予想結果が１である馬が2頭以上いる場合に成り立つ
                preds_jiku = preds.query('pred == 1')
                if len(preds_jiku) == 1:
                    continue
                elif len(preds_jiku) >= 2:
                    for umaban in combinations(preds_jiku['馬番'], 2):
                        return_ += self.bet(race_id, 'wide', umaban, 1)
                        n_bets += 1
                    return_list.append(return_)

            std = np.std(return_list) * np.sqrt(len(return_list)) / n_bets

            n_hits = np.sum([x>0 for x in return_list])
            return_rate = np.sum(return_list) / n_bets
            return n_bets, return_rate, n_hits, std  

        def sanrentan_box(self, X, threshold=0.5):
            pred_table = self.pred_table(X, threshold)
            n_bets = 0

            return_list = []
            for race_id, preds in pred_table.groupby(level=0):
                return_ = 0
                if len(preds)<3:
                    continue
                else:
                    for umaban in permutations(preds['馬番'], 3):
                        return_ += self.bet(race_id, 'sanrentan', umaban, 1)
                        n_bets += 1
                    return_list.append(return_)

            std = np.std(return_list) * np.sqrt(len(return_list)) / n_bets

            n_hits = np.sum([x>0 for x in return_list])
            return_rate = np.sum(return_list) / n_bets
            return n_bets, return_rate, n_hits, std

        def sanrenpuku_box(self, X, threshold=0.5):
            pred_table = self.pred_table(X, threshold)
            n_bets = 0

            return_list = []
            for race_id, preds in pred_table.groupby(level=0):
                return_ = 0
                if len(preds)<3:
                    continue
                else:
                    for umaban in combinations(preds['馬番'], 3):
                        return_ += self.bet(race_id, 'sanrenpuku', umaban, 1)
                        n_bets += 1
                    return_list.append(return_)

            std = np.std(return_list) * np.sqrt(len(return_list)) / n_bets

            n_hits = np.sum([x>0 for x in return_list])
            return_rate = np.sum(return_list) / n_bets
            return n_bets, return_rate, n_hits, std

        def umaren_nagashi(self, X, threshold=0.5, n_aite=5):
            pred_table = self.pred_table(X, threshold, bet_only = False)
            n_bets = 0

            return_list = []
            for race_id, preds in pred_table.groupby(level=0):
                return_ = 0
                preds_jiku = preds.query('pred == 1')
                if len(preds_jiku) == 1:
                    preds_aite = preds.sort_values('score', ascending = False)\
                        .iloc[1:(n_aite+1)]['馬番']
                    return_ = preds_aite.map(
                        lambda x: self.bet(
                            race_id, 'umaren', [preds_jiku['馬番'].values[0], x], 1
                        )
                    ).sum()
                    n_bets += n_aite
                    return_list.append(return_)
                elif len(preds_jiku) >= 2:
                    for umaban in combinations(preds_jiku['馬番'], 2):
                        return_ += self.bet(race_id, 'umaren', umaban, 1)
                        n_bets += 1
                    return_list.append(return_)

            std = np.std(return_list) * np.sqrt(len(return_list)) / n_bets

            n_hits = np.sum([x>0 for x in return_list])
            return_rate = np.sum(return_list) / n_bets
            return n_bets, return_rate, n_hits, std

        def umatan_nagashi(self, X, threshold=0.5, n_aite=5):
            pred_table = self.pred_table(X, threshold, bet_only = False)
            n_bets = 0

            return_list = []
            for race_id, preds in pred_table.groupby(level=0):
                return_ = 0
                preds_jiku = preds.query('pred == 1')
                if len(preds_jiku) == 1:
                    preds_aite = preds.sort_values('score', ascending = False)\
                        .iloc[1:(n_aite+1)]['馬番']
                    return_ = preds_aite.map(
                        lambda x: self.bet(
                            race_id, 'umatan', [preds_jiku['馬番'].values[0], x], 1
                        )
                    ).sum()
                    n_bets += n_aite

                elif len(preds_jiku) >= 2:
                    for umaban in permutations(preds_jiku['馬番'], 2):
                        return_ += self.bet(race_id, 'umatan', umaban, 1)
                        n_bets += 1
                return_list.append(return_)

            std = np.std(return_list) * np.sqrt(len(return_list)) / n_bets

            n_hits = np.sum([x>0 for x in return_list])
            return_rate = np.sum(return_list) / n_bets
            return n_bets, return_rate, n_hits, std

        def wide_nagashi(self, X, threshold=0.5, n_aite=5):
            pred_table = self.pred_table(X, threshold, bet_only = False)
            n_bets = 0

            return_list = []
            for race_id, preds in pred_table.groupby(level=0):
                return_ = 0
                preds_jiku = preds.query('pred == 1')
                if len(preds_jiku) == 1:
                    preds_aite = preds.sort_values('score', ascending = False)\
                        .iloc[1:(n_aite+1)]['馬番']
                    return_ = preds_aite.map(
                        lambda x: self.bet(
                            race_id, 'wide', [preds_jiku['馬番'].values[0], x], 1
                        )
                    ).sum()
                    n_bets += len(preds_aite)
                    return_list.append(return_)
                elif len(preds_jiku) >= 2:
                    for umaban in combinations(preds_jiku['馬番'], 2):
                        return_ += self.bet(race_id, 'wide', umaban, 1)
                        n_bets += 1
                    return_list.append(return_)

            std = np.std(return_list) * np.sqrt(len(return_list)) / n_bets

            n_hits = np.sum([x>0 for x in return_list])
            return_rate = np.sum(return_list) / n_bets
            return n_bets, return_rate, n_hits, std

        def sanrentan_nagashi(self, X, threshold = 1.5, n_aite=7):
            pred_table = self.pred_table(X, threshold, bet_only = False)
            n_bets = 0
            return_list = []
            for race_id, preds in pred_table.groupby(level=0):
                preds_jiku = preds.query('pred == 1')
                if len(preds_jiku) == 1:
                    continue
                elif len(preds_jiku) == 2:
                    preds_aite = preds.sort_values('score', ascending = False)\
                        .iloc[2:(n_aite+2)]['馬番']
                    return_ = preds_aite.map(
                        lambda x: self.bet(
                            race_id, 'sanrentan',
                            np.append(preds_jiku['馬番'].values, x),
                            1
                        )
                    ).sum()
                    n_bets += len(preds_aite)
                    return_list.append(return_)
                elif len(preds_jiku) >= 3:
                    return_ = 0
                    for umaban in permutations(preds_jiku['馬番'], 3):
                        return_ += self.bet(race_id, 'sanrentan', umaban, 1)
                        n_bets += 1
                    return_list.append(return_)

            std = np.std(return_list) * np.sqrt(len(return_list)) / n_bets

            n_hits = np.sum([x>0 for x in return_list])
            return_rate = np.sum(return_list) / n_bets
            return n_bets, return_rate, n_hits, std
    class ST_results:
        def __init__(self,STresults):
            self.data = STresults

        @classmethod
        def read_pickle(self, path_list):
            df = pd.read_pickle(path_list[0])
            for path in path_list[1:]:
                df = update_data(df, pd.read_pickle(path),False)
            return self(df)

        @classmethod
        def scrape(self,race_id_list):
            data = pd.DataFrame()
            for race_id in tqdm(race_id_list):
                time.sleep(1)
                url = 'https://race.netkeiba.com/race/result.html?race_id=' + race_id
                # print(url)
                df = pd.read_html(url)[0]
                df['race_id'] = [race_id] * len(df)
                df = df[['race_id','着順','馬番','単勝オッズ']]
                data = pd.concat([data, df])
            return self(data)
    class ST_odds:
        def __init__(self,STresults):
            self.data = STresults

        @classmethod
        def read_pickle(self, path_list):
            df = pd.read_pickle(path_list[0])
            for path in path_list[1:]:
                df = update_data(df, pd.read_pickle(path),True)
            return self(df)

        @classmethod
        def scrape(self,race_id_list):
            df = pd.DataFrame()
            data = pd.DataFrame()

            #ブラウザのオプション設定
            #IPとポート番号を指定することで、既に開かれているブラウザをつかむキャッチする
            #seleniumの保存先　C:\Windows\System32
            options = Options()
            # options.add_argument('--headless')
            
            driver = webdriver.Chrome(options=options)
            #バージョンをその都度確認して更新する場合
            # driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)

            for race_id in tqdm(race_id_list):
                driver.get('https://race.netkeiba.com/odds/index.html?type=b1&race_id=' + race_id + '&rf=shutuba_submenu')
                time.sleep(0.5)
                url = driver.current_url
                table= pd.read_html(url, header = 0)

                odds = []
                for getelement in driver.find_elements_by_css_selector(".Odds.transition-color"):
                    text = getelement.text
                    odds.append(text)

                i=0
                df = table[0]

                if len(df[~df['選択'].isna()])==0:#取消なしの場合
                    for num in df.index:
                        df.loc[num,'オッズ']=odds[i]
                        i+=1
                    for num in df.index:
                        df.loc[num,'複勝オッズ']=odds[i]
                        i+=1
                else:
                    for num in df.index:
                        if df.loc[num,'選択'] is np.nan:
                            df.loc[num,'オッズ']=odds[i]
                            i+=1
                        else:
                            continue
                    for num in df.index:
                        if df.loc[num,'選択'] is np.nan:
                            df.loc[num,'複勝オッズ']=odds[i]
                            i+=1
                        else:
                            continue

                df['複勝オッズ_min'] =df['複勝オッズ'].str.split('-', expand=True)[0]
                df['複勝オッズ_max'] = df['複勝オッズ'].str.split('-', expand=True)[1]
                df.drop(['枠','印','選択','複勝オッズ'], axis=1, inplace=True)
                df.rename(columns={'オッズ': '単勝オッズ'}, inplace=True)
                df.index = [race_id] * len(df)

                data = pd.concat([data, df])
            driver.quit()
            return self(data)

    #関数
    def update_data(old, new,index=True):
        """
        Parameters:
        ----------
        old : pandas.DataFrame
            古いデータ
        new : pandas.DataFrame
            新しいデータ
        """
        if index:
            filtered_old = old[~old.index.isin(new.index)]
        else:
            #更新されたことになっていないのでは？
            filtered_old = old[~old['race_id'].isin(new['race_id'])]

        return pd.concat([filtered_old, new])
    def split_data(df, test_size=0.3):
        sorted_id_list = df.sort_values("date").index.unique()
        train_id_list = sorted_id_list[: round(len(sorted_id_list) * (1 - test_size))]
        test_id_list = sorted_id_list[round(len(sorted_id_list) * (1 - test_size)) :]
        train = df.loc[train_id_list]
        test = df.loc[test_id_list]
        return train, test
    def gain(return_func, X, n_samples=100, t_range=[0.5, 2.5]):
        gain = {}
        for i in tqdm(range(n_samples)):
        # for i in range(n_samples):
            #min_thresholdから1まで、n_samples等分して、thresholdをfor分で回す
            threshold = t_range[1] * i / n_samples + t_range[0] * (1-(i/n_samples))
            n_bets, return_rate, n_hits, std = return_func(X, threshold)
            if n_bets > 2:
                gain[threshold] = {'return_rate': return_rate, 
                                'n_hits': n_hits,
                                'std': std,
                                'n_bets': n_bets}
        return pd.DataFrame(gain).T
    def plot(df, label=' '):
        #標準偏差で幅をつけて薄くプロット
        plt.fill_between(df.index, y1=df['return_rate']-df['std'],
                     y2=df['return_rate']+df['std'],
                     alpha=0.3) #alphaで透明度を設定
        #回収率を実線でプロット
        plt.plot(df.index, df['return_rate'], label=label)
        plt.legend() #labelで設定した凡例を表示させる
        plt.grid(True) #グリッドをつける
    def time_protcol():
        #使用する際は事前に下記を実行
        #t = time.time()
        global t
        s = time.time()
        print(s-t)
        t=time.time()

In [ ]:
# pdの行列表示数の設定
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',850)

# 前日作業

## データ更新済み確認

In [ ]:
#SPI算出用にresultsの先週分まで更新されているか
#ShutubaTableの更新確認
#LabelEncorderの更新確認
st = ShutubaTable.read_pickle(['pickle/syutuba/race/syutuba_latest.pickle'])
print('st.data\n',pd.to_datetime(st.data['date'], format='%Y年%m月%d日').sort_values(ascending=False).head(1))
r22_data_p= pd.read_pickle('pickle/r_data19_21/r_data_p22.pickle')
print('\nr22_data_p\n',r22_data_p.sort_values(by=['date'],ascending=False)['date'].head(1))

## 各種スクレイピング作業を事前に行っておく

In [ ]:
""" 
#その日に開催されるレースIDと日付けを指定する
#日付は予測値に影響を与えるので注意(hrのマージの日付になる)
"""
date = '2022年12月4日'
#開催場所に合わせて設定
racenum = 12
race_id_list1 = ['2022060502{}'.format(str(i).zfill(2)) for i in range(1, racenum+1, 1)]
race_id_list2 = ['2022090602{}'.format(str(i).zfill(2)) for i in range(1, racenum+1, 1)]
race_id_list3 = ['2022070602{}'.format(str(i).zfill(2)) for i in range(1, racenum+1, 1)]
race_id_list = race_id_list1 + race_id_list2 + race_id_list3
# race_id_list = race_id_list1 + race_id_list2

print(len(race_id_list))
date_list = [date]*len(race_id_list)
race_id_day_dict = dict(zip(race_id_list,date_list))
race_id_day_dict

In [ ]:
# #前日オッズを取得
# odds = ST_odds.scrape(race_id_day_dict.keys())
# odds.data.to_pickle('pickle/syutuba/race/odds/the_day_befor_odds/odds_{}.pickle'.format(date))

In [ ]:
#スクレイピングして保存(馬体重なしになる)(前日情報として別で保存)
st = ShutubaTable.scrape(race_id_day_dict,False)
st.data.to_pickle('pickle/syutuba/race/day_before/syutuba_{}.pickle'.format(date))

In [ ]:
st = ShutubaTable.read_pickle(['pickle/syutuba/race/day_before/syutuba_{}.pickle'.format(date)])

In [ ]:
jr = jockey_results.read_pickle(['pickle/syutuba/latest/jockey_id_latest.pickle'])
hi = HorseInfo.read_pickle(['pickle/syutuba/latest/h_info_latest.pickle'])

In [ ]:
#horse_info,jockey_idを取得する
st.merge_horse_info(hi)
st.merge_jockey_info(jr,5)
st.preprocessing()

In [ ]:
#海外jockeyなどの新しいジョッキーが出てきた場合にデータが存在しなく、
#エラーとなった場合に手動でデータを追加する手順
#no text parsed from document (line 0)
# test = pd.read_pickle('pickle/syutuba/latest/jockey_id_latest.pickle')
# test.loc['05628'] = 0#更新
# test[test.index.isin(['a0436','05628','05464'])]#確認
# test.to_pickle('pickle/syutuba/latest/jockey_id_latest.pickle')#保存

In [ ]:
#出走馬のhorse_resulsを取得する
horse_id_list = st.data[st.data.index.isin(list(race_id_day_dict))]['horse_id'].unique()
h_results = HorseResults.scrape(horse_id_list,True)
h_results.to_pickle('pickle/syutuba/new_h_results/h_results_'+datetime.now().strftime('%m_%d')+'.pickle')
hr = HorseResults.read_pickle(['pickle/syutuba/latest/horse_results_latest.pickle',\
                              'pickle/syutuba/new_h_results/h_results_'+datetime.now().strftime('%m_%d')+'.pickle'])
hr.horse_results.to_pickle('pickle/syutuba/latest/horse_results_latest.pickle')

In [ ]:
#マージ用にrの作成
r = Results.read_pickle(['pickle/syutuba/r_data/r_data_latest.pickle'])
jr = jockey_results.read_pickle(['pickle/syutuba/latest/jockey_id_latest.pickle'])
hr = HorseResults.read_pickle(['pickle/syutuba/latest/horse_results_latest.pickle'])
hi = HorseInfo.read_pickle(['pickle/syutuba/latest/h_info_latest.pickle'])
p = Peds.read_pickle(['pickle/syutuba/latest/peds_id_latest.pickle'])

past_race_num=10
t = time.time()
r.merge_horse_info(hi)
r.merge_jockey_info(jr,5)
time_protcol()
r.preprocessing()

In [ ]:
hr.preprocessing()
past_race_num=10
st.merge_horse_results(hr, n_samples_list=[5,9,'all'] ,rt = r.data_p)
st.concat_past_data(hr,past_race_num)

In [ ]:
#pedsが存在しない場合は自動更新
p = Peds.read_pickle(['pickle/syutuba/latest/peds_id_latest.pickle'])
p.encode(st)
st.merge_peds(p.peds_e)
st.data_pe.head()

# 当日自動実行プログラム

## 関数定義

In [ ]:
#1_予測値の保存先ディレクトリの作成
def making_dir(date):
    os.makedirs('pickle/syutuba/race/'+date)
    os.makedirs('pickle/syutuba/race/'+date+'/std')
    os.makedirs('pickle/syutuba/race/'+date+'/first_result')
    os.makedirs('pickle/syutuba/race/'+date+'/latest_odds')
    os.makedirs('pickle/syutuba/生予測値/stdata_c/'+date)
#2_JRAからレース時刻一覧を取得してrace_scheduleを作成する
def get_racetime(date):
    #レースページから１R目の開始時間を取得する
    options = Options()
    # options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
    driver = webdriver.Chrome(options=options)
    
    year,month,day,tmp = re.split('\D+', date)
    # year,month,day = date[:4],date[5:7],date[8:10]
    # 取得するWEBページのURL(urlの作成を自動にする)
    url="https://www.jra.go.jp/keiba/calendar"\
    +year+"/"+year+"/"+month.zfill(2)+"/"+month.zfill(2)+day.zfill(2)+".html"
    driver.get(url)

    # HTTPリクエストヘッダにユーザーエージェントを設定
    headers = {'User-agent': 'Mozilla/5.0'}

    # WEBページデータを取得
    response = requests.get(url=url, headers=headers)
    response.encoding = response.apparent_encoding

    #開催場の取得
    place = []
    soup = BeautifulSoup(response.content,"html.parser")
    texts = soup.find_all(class_="inner")
    for text in texts:
        if '回' in text.text and '日' in text.text:
            # print(text.text)
            place.append(text.text.replace('\n', ''))

    # タイムスケジュールを取得
    df = pd.read_html(response.text,header = 0)
    #タイムスケジュールに開催場を追加
    race_timedf = pd.DataFrame()
    df = pd.read_html(response.text,header = 0)
    for i in range(0,len(df)):
        df[i].insert(0, '開催', place[i])
        race_timedf = pd.concat([race_timedf,df[i]])
    driver.quit()
    return race_timedf
#3_race_scheduleの加工
def create_day_schedule(race_id_list,race_schedulen,num=3,bet_time=2):
    maindf = pd.DataFrame()
    #レースIDと開催場の対応dataframe作成
    def create_raceid_df(race_id_list):
        place_list =[]
        place_dict = {
            '札幌':'01',  '函館':'02',  '福島':'03',  '新潟':'04',  '東京':'05', 
            '中山':'06',  '中京':'07',  '京都':'08',  '阪神':'09',  '小倉':'10'
        }
        place_dict = { place_dict[ i ] : i for i in place_dict.keys() }#keyとvalueの転換

        for race_id in race_id_list:
            place_list.append(place_dict[race_id[4:6]])
        return pd.DataFrame([race_id_list,place_list],index=['race_id','place']).T
    race_id_df = create_raceid_df(race_id_list)

    #タイムスケジュールにrace_idを追加
    def con_race_id_df(race_schedule,race_id_df,place):
        temp1 = race_schedule[race_schedule['開催'].str.contains(place)]
        temp2 = race_id_df[race_id_df['place'].str.contains(place)]
        temp2.reset_index(inplace=True, drop=True)
        concatdf = pd.concat([temp1,temp2],axis=1)
        return concatdf
    maindf = pd.concat([con_race_id_df(race_schedule,race_id_df,place) for place in race_id_df['place'].unique()],axis=0)
    maindf['R'] = maindf['レース番号'].map(lambda x :re.sub(r"\D", "", x))#レース番号を番号のみに変換
    maindf["time"] = pd.to_datetime(race_schedule["発走時刻"],format="%H時%M分")#時刻をdatetime型に変換
    maindf['get_odds_time']  = maindf['time'].map(lambda x: str(x + timedelta(minutes=-bet_time))[11:16])

    #各レース回の初回レース10分前に設定
    scrape_times,scrape_time = [],[]
    for i in range(1,13):
        one_rtime = maindf[maindf['レース番号']=='{}レース'.format(i)]['time'].min()
        scrape_times.append(one_rtime + timedelta(minutes=-10))

    #各レース回のレイピングタイミング
    for i in range(0,12):
        scrape_time.append(str(scrape_times[i])[11:16])
    maindf['get_race_time'] = scrape_time*num
    return maindf.iloc[:,4:]
#4_予測値の算出
def create_race_pred(race_id_day_dict,date,hr_p_backup,r):
    
    start_text = 'Start scraping ShutubaTable.',list(race_id_day_dict.keys())
    with open('pickle/syutuba/race/'+date+'/console_log.txt', 'a', encoding='sjis') as f:
        print(start_text,file=f)
    
    now = datetime.now()
    current_time = now.strftime("%Y%m%d%H%M%S")
    st = ShutubaTable.scrape(race_id_day_dict,False)
    for rid,date in race_id_day_dict.items():
        st.data[st.data.index==rid].to_pickle('pickle/syutuba/race/{0}/syutuba_{1}.pickle'.format(date,rid))
    
    #不足データがある場合、スクレイピングされ常に更新されていく
    hi = HorseInfo.read_pickle(['pickle/syutuba/latest/h_info_latest.pickle'])
    st.merge_horse_info(hi)
    jr = jockey_results.read_pickle(['pickle/syutuba/latest/jockey_id_latest.pickle'])
    st.merge_jockey_info(jr,5)
    st.preprocessing()
    hr.horse_results_p = hr_p_backup.copy()
    past_race_num=10
    st.merge_horse_results(hr, n_samples_list=[5,9,'all'] ,rt = r.data_p)
    st.concat_past_data(hr,past_race_num)
    p = Peds.read_pickle(['pickle/syutuba/latest/peds_id_latest.pickle'])
    p.encode(st)
    st.merge_peds(p.peds_e)
    
    #最新ラベルエンコーダーの読み込み
    def Read_Label_Encorder(r):
        with open('pickle/LabelEncoder/le_horse.pickle', 'rb') as f:
            r.le_horse = pickle.load(f)
        with open('pickle/LabelEncoder/le_jockey.pickle', 'rb') as f:
            r.le_jockey = pickle.load(f)
        with open('pickle/LabelEncoder/le_class.pickle', 'rb') as f:
            r.le_class = pickle.load(f)
        with open('pickle/LabelEncoder/le_place.pickle', 'rb') as f:
            r.le_place = pickle.load(f)
        with open('pickle/LabelEncoder/le_trainer.pickle', 'rb') as f:
            r.le_trainer = pickle.load(f)
        with open('pickle/LabelEncoder/le_horseowner.pickle', 'rb') as f:
            r.le_horseowner = pickle.load(f)
        with open('pickle/LabelEncoder/le_producer.pickle', 'rb') as f:
            r.le_producer = pickle.load(f)
        with open('pickle/LabelEncoder/le_relative1.pickle', 'rb') as f:
            r.le_relative1 = pickle.load(f)
        with open('pickle/LabelEncoder/le_relative2.pickle', 'rb') as f:
            r.le_relative2 = pickle.load(f)
        with open('pickle/LabelEncoder/le_kai.pickle', 'rb') as f:
            r.le_kai = pickle.load(f)
        with open('pickle/LabelEncoder/le_day.pickle', 'rb') as f:
            r.le_day = pickle.load(f)
    Read_Label_Encorder(r)
    
    #カテゴリデータへ変換
    st.process_categorical\
    (r.le_horse,r.le_jockey,r.le_class,r.le_place,r.data_pe,r.le_trainer,r.le_horseowner,\
     r.le_producer,r.le_relative1,r.le_relative2,r.le_kai,r.le_day,past_race_num)

    #カテゴリデータの順番整理
    # latest_colcategory = pd.read_pickle('pickle/st_data/category_std/st_data20220827.pickle')
    latest_colcategory = pd.read_pickle('pickle/st_data/st_data_c_latest_categories.pickle')
    for index ,name in latest_colcategory.head(0).dtypes.iteritems():
        if name=='category':
            # print(index)
            new = list(st.data_c[index].cat.categories)
            moto = list(latest_colcategory[index].cat.categories)
            number = 0
            befor = 0
            changeFLG=False
            A,B,C,D=[],[],[],[]
            D = list(set(new)-set(moto))#完全に新しく出てきたもの
            # if len(D)>0:
            #     print('new_label_encode',index,D)
            for i in moto:
                if befor <= i:
                    befor = i
                    number+=1
                else:
                    changeFLG = True
                    A = moto[:number]#前半 直前まで出たもの
                    B = moto[number:]#後半 2021まで出ていたもの
                    C = list(set(B)&set(new))#今回新たに出ているが過去にも出たことのあるもの
                    # for s in C:
                    #     B.remove(s)
                    # B = [s for s in B if s not in C]
                    list(map(lambda x: B.remove(x), C))
            if changeFLG==False:
                A = moto[:number]#前半 直前まで出たもの
                B = moto[number:]#後半 2021まで出ていたもの
                C = list(set(B)&set(new))#今回新たに出ているが過去にも出たことのあるもの
                list(map(lambda x: B.remove(x), C))
    
            setlist = (sorted(A+C)+sorted(D)+(B))
            # time_protcol()

            if len(setlist)==0:#何も更新がなかった場合
                # print('更新なし',index)
                st.data_c[index] =st.data_c[index].cat.set_categories((moto))
            else:
                #前半部分に新しいリストを合わせて昇順にする
                st.data_c[index] =st.data_c[index].cat.set_categories((setlist))

    #標準偏差列を追加して特徴量を合わせる
    def std_data_proccesing(st):
        #並び替えとindexのsetとタイトル削除
        st.data_c=st.data_c.sort_values(['race_id','馬番'])
        st.data_c.set_index('race_id',inplace = True)
        st.data_c =st.data_c.rename_axis(index=None)

        #特徴量を削除
        # st.data_c.drop(columns =['time','final_to_rank_my_allR','jockey_id'],inplace = True)
        st.data_c.drop(columns =['time'],inplace = True)

        #レースIDごとに標準偏差に変更する
        std_tar_col =['枠番', 'first_to_final_course_len_5R', 'first_corner_race_type_9R', 'final_to_rank_8p', '体重_8p']
        # std_tar_col =['枠番', 'first_to_final_course_len_5R', 'first_corner_race_type_9R']
        standard_scaler = lambda x: (x - x.mean()) / x.std()
        st.data_c[list(map(lambda x:  x+'_std', std_tar_col))] = st.data_c[std_tar_col].groupby(level=0).transform(standard_scaler)

        #学習モデルの特徴量を合わせるために、保存したデータを読み込み
        col_df = pd.read_pickle('pickle/syutuba/columns/r_columns.pickle')
        if len(set(col_df.columns.tolist())-set(st.data_c.columns.tolist())) != 0:
            print(set(col_df.columns.tolist())-set(st.data_c.columns.tolist()))
        elif len(set(st.data_c.columns.tolist())-set(col_df.columns.tolist())) != 0:
            print(set(st.data_c.columns.tolist())-set(col_df.columns.tolist()))

        #不足分をリスト化して追加する
        col = set(col_df.columns)-set(st.data_c.columns)
        st.data_c[list(col)]=0
    std_data_proccesing(st)
    
    #予測前データの保存
    for rid,date in race_id_day_dict.items():
        st.data_c[st.data_c.index==rid].to_pickle('pickle/syutuba/生予測値/stdata_c/{0}/stdata_c_std{1}.pickle'.format(date,rid))
    
    #カテゴリデータのnullチェック
    def check_null_category(stdata_c):
        nullcheck = False
        for index ,name in stdata_c.head(0).dtypes.iteritems():
            if name=='category':
                if len(stdata_c[stdata_c[index].isna()==True])>0:
                    nullcheck = True
                    print(index,len(stdata_c[stdata_c[index].isna()==True]))
        if nullcheck==False:
            print('not exist null in category data.')
        else:
            print('exist null in category data.')   
    check_null_category(st.data_c)
    
    #学習モデルに予測させる
    def Created_predscore(st, r, returnModel, top4Model):
        stX_test = st.data_c.drop(['着順', '人気','date','生年月日'], axis=1)
        # stX_test = test.drop(['着順', '人気','date','生年月日'], axis=1)

        #学習モデルの読み込み
        lgb_clf_return = pickle.load(open('./pickle/syutuba/学習モデル/'+returnModel+'.pkl', 'rb'))
        lgb_clf_top4 = pickle.load(open('./pickle/syutuba/学習モデル/'+top4Model+'.pkl', 'rb'))

        #ModelEvaluatorクラスのオブジェクトを作成
        #予想時にはreturn情報はないので、適当によみこむ
        me_st_return = ModelEvaluator(lgb_clf_return, ['pickle/syutuba/latest/return_table_latest.pickle'])
        me_st_top4 = ModelEvaluator(lgb_clf_top4, ['pickle/syutuba/latest/return_table_latest.pickle'])

        #predの算出
        scores = me_st_return.predict_proba(stX_test)
        pred = stX_test[['馬番','horse_id','class','place']].copy()
        #ラベルデコード
        pred['le_horse_id']=r.le_horse.inverse_transform(pred['horse_id'])
        pred['le_class']=r.le_class.inverse_transform(pred['class'])
        pred['le_place']=r.le_place.inverse_transform(pred['place'])
        pred['score'] = scores
        pred_return = pred.copy()
        #predの算出
        scores = me_st_top4.predict_proba(stX_test)
        pred = stX_test[['馬番','horse_id','class','place']].copy()
        #ラベルデコード
        pred['le_horse_id']=r.le_horse.inverse_transform(pred['horse_id'])
        pred['le_class']=r.le_class.inverse_transform(pred['class'])
        pred['le_place']=r.le_place.inverse_transform(pred['place'])
        pred['score'] = scores
        pred_top4 = pred.copy()

        #mergeするためにindexをカラムにして、merge後元に戻す
        st.data.reset_index(inplace=True)
        st.data.rename(columns={'index': 'race_id'},inplace=True)
        pred_return.reset_index(inplace=True)
        pred_return.rename(columns={'index': 'race_id'},inplace=True)
        pred_top4.reset_index(inplace=True)
        pred_top4.rename(columns={'index': 'race_id'},inplace=True)

        AI_return = pred_return.merge(st.data[['horse_id','馬名']].drop_duplicates(),how='left',left_on=['le_horse_id'],right_on=['horse_id'])
        AI_top4 = pred_top4.merge(st.data[['horse_id','馬名']].drop_duplicates(),how='left',left_on=['le_horse_id'],right_on=['horse_id'])
        AI_top4 = AI_top4.merge(st.data[['race_id','date']].drop_duplicates(),how='left',left_on=['race_id'],right_on=['race_id'])

        st.data.set_index('race_id',inplace = True)
        st.data.index.name = None
        pred_return.set_index('race_id',inplace = True)
        pred_return.index.name = None
        pred_top4.set_index('race_id',inplace = True)
        pred_top4.index.name = None

        AI_return.drop(columns=['horse_id_x','class','le_horse_id','horse_id_y'],inplace=True)
        AI_return = AI_return.reindex(columns=['race_id','le_place','馬番','馬名','le_class','score'])

        AI_top4.drop(columns=['horse_id_x','class','le_horse_id','horse_id_y'],inplace=True)
        AI_top4 = AI_top4.reindex(columns=['race_id','date','le_place','馬番','馬名','le_class','score'])

        place_dict = {
        '01':'札幌',  '02':'函館',  '03':'福島',  '04':'新潟',  '05':'東京', 
        '06':'中山',  '07':'中京',  '08':'京都',  '09':'阪神',  '10':'小倉'
        }
        AI_return = AI_return.replace(place_dict)
        AI_top4 = AI_top4.replace(place_dict)

        AI = AI_top4.merge(AI_return[['race_id','馬名','score']],how='left',left_on=['race_id','馬名'],right_on=['race_id','馬名'])
        AI.rename(columns={'score_x': 'score_top4','score_y': 'score_return',},inplace=True)
        AI.insert(loc=3,column='R',value=AI['race_id'].str[-2:])
        return AI
    returnModel = '19-21学習モデル_750'
    top4Model = '19-21的中型上位4位モデル'
    AI = Created_predscore(st, r, returnModel, top4Model)
    
    #オッズを結合する
    def concate_odds(race_id_day_dict,current_time,AI):
        st_odds = ST_odds.scrape(race_id_day_dict.keys())
        st_odds.data.to_pickle('pickle/syutuba/race/odds/st_odds_{}.pickle'.format(current_time))
        STR=ST_odds.read_pickle(['pickle/syutuba/race/odds/st_odds_latest.pickle','pickle/syutuba/race/odds/st_odds_{}.pickle'.format(current_time)])
        STR.data.to_pickle('pickle/syutuba/race/odds/st_odds_latest.pickle')
        STR.data.reset_index(inplace=True)
        STR.data.rename(columns={'index': 'race_id'},inplace=True)
        add_odds = AI.merge(STR.data,how='left',left_on=['race_id','馬番'],right_on=['race_id','馬番'])
        add_odds.drop(columns = '馬名_y' ,inplace = True)
        rankdf = add_odds.copy()
        rankdf['score_rank'] = add_odds.groupby(by = 'race_id')['score_top4'].rank(ascending=False)
        rankdf['return_rank'] = add_odds.groupby(by = 'race_id')['score_return'].rank(ascending=False)
        first_result = rankdf.sort_values(['le_place','R','馬番'], ascending=True).copy()
        return first_result
    first_result = concate_odds(race_id_day_dict, current_time, AI)
    
    #予測値の保存
    for rid,date in race_id_day_dict.items():
        first_result[first_result['race_id']==rid].to_pickle('pickle/syutuba/race/{0}/first_result/first_result_{1}.pickle'.format(date,rid))
    
    clear_output(True)#出力の削除
    
    end_text = 'create_race_pred finished.'
    print(end_text)
    with open('pickle/syutuba/race/'+date+'/console_log.txt', 'a', encoding='sjis') as f:
        print(end_text,file=f)

    return first_result
#5_1ネット投票ログイン
def login_netkeiba(JRA_type):
    options = Options()
    driver = webdriver.Chrome(options=options)
    if JRA_type == "tyuuou":
        url="https://www.ipat.jra.go.jp/sp/"#中央競馬
        btnname = "btnGreen"
    else:
        url="https://n.ipat.jra.go.jp/sp/"#地方競馬
        btnname = "btnBrown"
    driver.get(url)
    driver.find_element(By.ID,"userid").send_keys("")
    driver.find_element(By.ID,"password").send_keys("")
    driver.find_element(By.ID,"pars").send_keys("")
    time.sleep(2)
    # driver.find_element(By.CLASS_NAME, 'btnGreen').click()#中央競馬
    driver.find_element(By.CLASS_NAME, btnname).click()#地方競馬
    time.sleep(2)
    if (len(driver.find_elements(By.CLASS_NAME, 'ico_regular'))) ==0:
        driver.find_element(By.CLASS_NAME, btnname).click()#OKボタンが挟まれるとき必要
    # print(driver.window_handles[-1])
    return driver
#5_2ネット投票サイトから最新オッズの取得
def get_latest_odds(kaisai,kai,driver,date="hoge",race_id="test"):
    sec=2#ブラウザ遷移の待機時間
    time.sleep(sec)
    driver.find_element(By.CLASS_NAME, 'ico_regular').click()#通常投票
    driver.find_element(By.PARTIAL_LINK_TEXT,kaisai).click()#競馬場
    time.sleep(sec)
    driver.find_element(By.PARTIAL_LINK_TEXT,str(kai)+'R').click()#何Rか
    time.sleep(sec)
    driver.find_element(By.PARTIAL_LINK_TEXT,'単勝').click()
    time.sleep(sec)
    b=driver.find_element(By.CLASS_NAME,'selectHorse').find_elements(By.CLASS_NAME,'ui-link')
    time.sleep(sec)
    
    odds_df = pd.DataFrame()
    horseNumber=[]#馬番
    horseName=[]#馬名
    odds=[]#オッズ
    torikeshi=False#出走取り消しが存在するか

    for n in range(len(b)):
        x=b[n].text.split('\n')
        horseNumber.append(x[0])
        horseName.append(x[1])
        if x[2]!='取消':
            odds.append(x[2])
        else:
            odds.append('-')
            torikeshi=True
    
    # driver.find_element(By.LINK_TEXT,'トップメニュー').click()#通常投票画面に返る
    
    odds_df = pd.DataFrame([horseNumber,horseName,odds],index=["馬番", "馬名", "単勝オッズ"] ).T
    odds_df.to_pickle('pickle/syutuba/race/{0}/latest_odds/{1}_{2}_{3}.pickle'.format(date,race_id,kaisai,kai))
    
    return odds_df
#5_取得した最新オッズで購入対象馬を選定して購入する
def Forecast_calculation(race_id,kaisai,R,date):
    start_text = race_id,kaisai,R+'R','Get the latest odds and decide which horses to buy.'
    print(start_text)
    with open('pickle/syutuba/race/'+date+'/console_log.txt', 'a', encoding='sjis') as f:
        print(start_text,file=f)
    sec=2#ブラウザ遷移の待機時間
    #race_idによる予測値表の読み込み
    pred_df=pd.DataFrame()
    path_list=[]
    path = 'pickle/syutuba/race/'+date+'/first_result/first_result_{}.pickle'.format(race_id)
    pred_df = pd.read_pickle(path)
    
    if len(pred_df) != 0:#障害レース以外の時
        
        #開催とRによる最新オッズの取得
        driver = login_netkeiba('tyuuou')#tyuuouに変更
        odds = get_latest_odds(kaisai,R,driver,date,race_id)#kaisaiとRを使用する

        #オッズの入れ替え
        pred_df.loc[:,'単勝オッズ']=odds[odds['単勝オッズ']!='-']['単勝オッズ'].values
        pred_df.loc[:,'単勝オッズ'] = pred_df['単勝オッズ'].astype(str)

        #各モデルでの予測値の算出
        range_exp = 0 #期待値のレンジ幅
        return_money = 90000 #払い戻し金額設定
        return_pred = return_model(pred_df, range_exp, return_money)#期待値範囲０にすること
        top_pred = top_model(pred_df, range_exp, return_money)
        hybrid = hybrid_model(pred_df,return_pred,top_pred,range_exp)
        index = index_model(pred_df)
        hybrid2 = hybrid2_model(pred_df)
        
        #購入対象の決定
        buyList = []#購入する馬番リストを作る
        buyList.extend(return_pred['馬番'].values)
        with open('pickle/syutuba/race/'+date+'/console_log.txt', 'a', encoding='sjis') as f:
            print('return_pred',buyList,file=f)

        buyList.extend(hybrid['馬番'].values)#index対象馬の追加
        with open('pickle/syutuba/race/'+date+'/console_log.txt', 'a', encoding='sjis') as f:
            print('hybrid',hybrid['馬番'].values,file=f)

        buyList.extend(index['馬番'].values)#index対象馬の追加
        with open('pickle/syutuba/race/'+date+'/console_log.txt', 'a', encoding='sjis') as f:
            print('index',index['馬番'].values,file=f)
        
        # buyList.extend(hybrid2['馬番'].values)#index対象馬の追加
        with open('pickle/syutuba/race/'+date+'/console_log.txt', 'a', encoding='sjis') as f:
            print('hybrid2',hybrid2['馬番'].values,file=f)
        
        buyList = list(set(buyList))#購入馬番の重複を除く
        
        #取消が存在する場合の処理はどうなっていたかサイト確認
        if len(buyList) == 0:
            output_text = kaisai + R + 'R'+"は購入しない"
            print(output_text)
            with open('pickle/syutuba/race/'+date+'/console_log.txt', 'a', encoding='sjis') as f:
                print(output_text,file=f)
        else:
            for n in range(len(buyList)):#buyListの中身を購入する
                umaban = buyList[n]
                output_text = kaisai + R + 'R '+str(umaban) + "番、" +str(odds['馬名'][umaban-1]) + "、オッズ" + str(odds['単勝オッズ'][umaban-1])
                print(output_text)
                with open('pickle/syutuba/race/'+date+'/console_log.txt', 'a', encoding='sjis') as f:
                    print(output_text,file=f)
                driver.find_element(By.CLASS_NAME,'selectHorse').find_elements(By.CLASS_NAME,'ui-link')[umaban-1].click()
                time.sleep(sec)
                driver.find_element(By.CLASS_NAME, 'ui-input-text').send_keys("10")
                time.sleep(sec)
                driver.find_element(By.LINK_TEXT,'セット').click()
                time.sleep(sec)
                driver.find_element(By.PARTIAL_LINK_TEXT,'番から').click()
                time.sleep(sec)
            driver.find_element(By.LINK_TEXT,'取消').click()
            time.sleep(sec)
            driver.find_element(By.LINK_TEXT,'入力終了').click()
            time.sleep(sec)
            driver.find_element(By.ID,'sum').send_keys(str(len(buyList)*1000))
            time.sleep(sec)
            driver.find_element(By.LINK_TEXT,'投票').click()
            time.sleep(sec)
            Alert(driver).accept()
            time.sleep(sec)
            driver.find_element(By.LINK_TEXT,'続けて通常投票').click()#通常投票画面に返る
            time.sleep(sec)
        driver.quit()
            
    else:#障害レースの時(スクレイピングファイルはあるが、中身がない)
        syogai_text = race_id,kaisai,R+'R','This race is Obstacle race.'
        print(syogai_text)
        with open('pickle/syutuba/race/'+date+'/console_log.txt', 'a', encoding='sjis') as f:
            print(syogai_text,file=f)
    
    end_text = race_id,kaisai,R+'R','Forecast_calculation finished.'
    with open('pickle/syutuba/race/'+date+'/console_log.txt', 'a', encoding='sjis') as f:
        print(end_text,file=f)
        
#5_3穴馬モデル
def return_model(first_result, range_exp, return_money):
    top3 = first_result[first_result['return_rank'] <= 3].sort_values(by = ['race_id','return_rank'], ascending=False)
    top3 = top3[~(top3['単勝オッズ'].str.contains('-', na=False))]
    top3 = top3.astype({'単勝オッズ': float})
    top3['期待値'] = top3['score_return']*top3['単勝オッズ']
    top3 = top3.sort_values('期待値', ascending=True)
    # ['course_len', 'race_type', 'race_round', 'weather','ground_state']
    dropcol = ['score_rank','return_rank','複勝オッズ_min','複勝オッズ_max']
    top3.drop(columns=dropcol,inplace=True)

    #開催場別に期待値を設定する
    top3.loc[top3['le_place'] == '札幌', 'min_exp'] = 8.4
    top3.loc[top3['le_place'] == '札幌', 'max_exp'] = 11.4
    top3.loc[top3['le_place'] == '小倉', 'min_exp'] = 9.4
    top3.loc[top3['le_place'] == '小倉', 'max_exp'] = 12.4
    top3.loc[top3['le_place'] == '新潟', 'min_exp'] = 7.5
    top3.loc[top3['le_place'] == '新潟', 'max_exp'] = 10.5
    top3.loc[top3['le_place'] == '阪神', 'min_exp'] = 10.2
    top3.loc[top3['le_place'] == '阪神', 'max_exp'] = 13.2
    top3.loc[top3['le_place'] == '中京', 'min_exp'] = 12.2
    top3.loc[top3['le_place'] == '中京', 'max_exp'] = 15.2
    top3.loc[top3['le_place'] == '中山', 'min_exp'] = 11.1
    top3.loc[top3['le_place'] == '中山', 'max_exp'] = 14.1
    top3.loc[top3['le_place'] == '東京', 'min_exp'] = 11.8
    top3.loc[top3['le_place'] == '東京', 'max_exp'] = 14.8
    top3.loc[top3['le_place'] == '福島', 'min_exp'] = 14.9
    top3.loc[top3['le_place'] == '福島', 'max_exp'] = 17.9
    top3.loc[top3['le_place'] == '函館', 'min_exp'] = 5.2
    top3.loc[top3['le_place'] == '函館', 'max_exp'] = 8.2

    top3=top3[(top3['期待値']>top3['min_exp']-range_exp) & (top3['期待値']<top3['max_exp']+range_exp)]
    top3=top3[top3['score_return']>0.7]#足きりスコア
    top3['minオッズ']=round(top3['min_exp']/top3['score_return'],2)
    top3['maxオッズ']=round(top3['max_exp']/top3['score_return'],2)
    top3['max_bet'] = round(1/top3['minオッズ']*return_money,-2)
    top3['min_bet'] = round(1/top3['maxオッズ']*return_money,-2)
    top3['期待値'] = round(top3['期待値'],1)
    top3.drop(columns=['min_exp','max_exp','期待値'],inplace=True)
    return top3.sort_values(['le_place','R','馬番'], ascending=True)
#5_4本命モデルttps://db.netkeiba.com/jockey/result/recent/05621p
def top_model(first_result, range_exp, return_money):
    top2 = first_result[first_result['score_rank'] <= 2].sort_values(by = ['race_id','score_top4'], ascending=False)
    top2 = top2[~(top2['単勝オッズ'].str.contains('-', na=False))]
    top2 = top2.astype({'単勝オッズ': float})
    top2['期待値'] = (top2['score_top4'])*top2['単勝オッズ']
    top2 = top2.sort_values('期待値', ascending=True)
    dropcol = ['score_rank','return_rank','複勝オッズ_min','複勝オッズ_max']
    top2.drop(columns=dropcol,inplace=True)

    #開催場別に期待値を設定する
    top2.loc[top2['le_place'] == '札幌', 'min_exp'] = 8.4
    top2.loc[top2['le_place'] == '札幌', 'max_exp'] = 10.4
    top2.loc[top2['le_place'] == '小倉', 'min_exp'] = 10.6
    top2.loc[top2['le_place'] == '小倉', 'max_exp'] = 12.6
    top2.loc[top2['le_place'] == '新潟', 'min_exp'] = 9.1
    top2.loc[top2['le_place'] == '新潟', 'max_exp'] = 11.1
    top2.loc[top2['le_place'] == '阪神', 'min_exp'] = 9.6
    top2.loc[top2['le_place'] == '阪神', 'max_exp'] = 11.6
    top2.loc[top2['le_place'] == '中京', 'min_exp'] = 7.5
    top2.loc[top2['le_place'] == '中京', 'max_exp'] = 9.5
    top2.loc[top2['le_place'] == '中山', 'min_exp'] = 8
    top2.loc[top2['le_place'] == '中山', 'max_exp'] = 10
    top2.loc[top2['le_place'] == '東京', 'min_exp'] = 8.2
    top2.loc[top2['le_place'] == '東京', 'max_exp'] = 10.2
    top2.loc[top2['le_place'] == '福島', 'min_exp'] = 7.6
    top2.loc[top2['le_place'] == '福島', 'max_exp'] = 9.6
    top2.loc[top2['le_place'] == '函館', 'min_exp'] = 12.6
    top2.loc[top2['le_place'] == '函館', 'max_exp'] = 14.6

    top2=top2[(top2['期待値']>top2['min_exp']-range_exp) & (top2['期待値']<top2['max_exp']+range_exp)]
    top2['minオッズ']=round(top2['min_exp']/top2['score_top4'],2)
    top2['maxオッズ']=round(top2['max_exp']/top2['score_top4'],2)
    top2['max_bet'] = round(1/top2['minオッズ']*return_money,-2)
    top2['min_bet'] = round(1/top2['maxオッズ']*return_money,-2)
    top2['期待値'] = round(top2['期待値'],1)
    top2.drop(columns=['min_exp','max_exp'],inplace=True)
    return top2.sort_values(['le_place','R','馬番'], ascending=True)
#5_5ハイブリットモデル
def hybrid_model(first_result,return_pred,top_pred,range_exp):
    df = first_result[~first_result['race_id'].isin(return_pred['race_id'])]
    df2 = df[df['return_rank'] <= 3].sort_values(by = ['race_id','return_rank'], ascending=False)
    df2 = df2.astype({'単勝オッズ': float})
    df3 = df2[df2['単勝オッズ']>20].value_counts('race_id')==1
    target_race_id = df3[df3].index
    target = first_result[first_result['race_id'].isin(target_race_id)]
    target = target[target['score_rank'] <= 2]
    target = target.astype({'単勝オッズ': float})
    target['期待値'] = target['単勝オッズ']*target['score_top4']
    kekka = target[(target['期待値']>8-range_exp)&(target['期待値']<10+range_exp)].copy()
    kekka['minオッズ']=round(8/kekka['score_top4'],2)
    kekka['maxオッズ']=round(10/kekka['score_top4'],2)
    kekka = kekka[kekka['score_top4']>=1.2]
    # kekka.drop(columns=['min_bet','max_bet','期待値'],inplace=True)
    return kekka.sort_values(['le_place','R','馬番'], ascending=True)
#5_6指数モデル
def index_model(index_df):
    def make3(first_result):
        top3 = first_result[first_result['return_rank'] <= 3].sort_values(by = ['race_id','return_rank'], ascending=False)
        top3 = top3[~(top3['単勝オッズ'].str.contains('-', na=False))]
        top3 = top3.astype({'単勝オッズ': float})
        top3['期待値'] = top3['score_return']*top3['単勝オッズ']
        top3 = top3.sort_values('期待値', ascending=True)
        # ['course_len', 'race_type', 'race_round', 'weather','ground_state']
        dropcol = ['複勝オッズ_min','複勝オッズ_max']
        top3.drop(columns=dropcol,inplace=True)
        return top3
    def make2(first_result):
        top2 = first_result[first_result['score_rank'] <= 2].sort_values(by = ['race_id','score_top4'], ascending=False)
        top2 = top2[~(top2['単勝オッズ'].str.contains('-', na=False))]
        top2 = top2.astype({'単勝オッズ': float})
        top2['期待値'] = (top2['score_top4'])*top2['単勝オッズ']
        top2 = top2.sort_values('期待値', ascending=True)
        dropcol = ['複勝オッズ_min','複勝オッズ_max']
        top2.drop(columns=dropcol,inplace=True)
        return top2
    sumdf = pd.DataFrame()
    top3 = make3(index_df)
    group_re = top3.groupby('race_id').sum()
    mergedf_re = top3.merge(group_re[['単勝オッズ','score_top4','score_return','score_rank','期待値']],on='race_id',how = 'left')
    top2 = make2(index_df)
    group_top = top2.groupby('race_id').sum()
    mergedf = top2.merge(group_top[['単勝オッズ','score_top4','score_return','score_rank','期待値']],on='race_id',how = 'left')
    sumdf = group_re.merge(group_top,how='outer',right_index=True,left_index=True,suffixes=['_re','_top'])
    race_id = sumdf[(sumdf['score_return_re']>sumdf['score_top4_top'])].index
    hits = top3[top3['race_id'].isin(race_id)].sort_values(['race_id','le_place','R','score_return']).copy()
    target = hits[hits['単勝オッズ']<=20].copy()
    target = target[target['score_return'].isin(target.groupby('race_id')['score_return'].max())]
    target = target[(target['score_rank']>=8)&(target['score_rank']<=10)]
    return target.sort_values(['le_place','R','馬番'], ascending=True)
#5_7ハイブリットモデル2(not_use_odds)
def hybrid2_model(first_result):
    def make_return(df,rank):
        rank_df = first_result[first_result['return_rank'] == rank].sort_values(by = ['race_id','return_rank'], ascending=False)
        rank_df = rank_df[~(rank_df['単勝オッズ'].str.contains('-', na=False))]
        rank_df = rank_df.astype({'単勝オッズ': float})
        # rank_df['期待値'] = rank_df['score_return']*rank_df['単勝オッズ']
        # rank_df = rank_df.sort_values('期待値', ascending=True)
        # ['course_len', 'race_type', 'race_round', 'weather','ground_state']
        dropcol = ['複勝オッズ_min','複勝オッズ_max']
        rank_df.drop(columns=dropcol,inplace=True)
        return rank_df
    return_top4 = pd.DataFrame()
    return6 = make_return(first_result,6)
    return7 = make_return(first_result,7)
    return8 = make_return(first_result,8)
    return9 = make_return(first_result,20)
    return6 = return6[return6['score_top4']>1.6]
    return7 = return7[return7['score_top4']>1.4]
    return8 = return8[return8['score_top4']>1.7]
    return9 = return9[return9['score_top4']>1.6]
    return_top4=pd.concat([return_top4,return6])
    return_top4=pd.concat([return_top4,return7])
    return_top4=pd.concat([return_top4,return8])
    return_top4=pd.concat([return_top4,return9])
    return return_top4

#6_スケジューラーの終了処理
def end():
    print("テスト終わり")
    global isWorking
    isWorking = False
#7_すべての最新オッズを取得する
def get_all_race_odds(driver,date):
    
    new_path = 'pickle/test/{0}/'.format(date)
    if not os.path.exists(new_path):#ディレクトリがなかったら
        os.mkdir(new_path)#作成したいフォルダ名を作成
    else:
        print('既に "'+new_path+'" は存在します。')

    driver.find_element(By.CLASS_NAME, 'ico_regular').click()#通常投票
    time.sleep(2)

    #開催選択画面
    places = []
    place_element = driver.find_elements(By.CLASS_NAME,'ui-link')
    for place_obj in place_element:
        if place_obj != place_element[0] and place_obj != place_element[-1]:
            print(place_obj.text)
            places.append(place_obj.text)

            time.sleep(2)
            driver.find_element(By.PARTIAL_LINK_TEXT,places[-1]).click()#競馬場
            # driver.find_element(By.PARTIAL_LINK_TEXT,place_obj.text).click()#競馬場
            time.sleep(2)

            #開催回がない場合の処理
            try:
                Alert(driver).accept()
                continue
            except NoAlertPresentException:
                pass

            # レース選択画面
            race_nums = []
            race_element = driver.find_elements(By.CLASS_NAME,'raceNum')
            for racenum_obj in race_element:
                print(racenum_obj.text)
                race_nums.append(racenum_obj.text)

                time.sleep(2)
                driver.find_element(By.PARTIAL_LINK_TEXT,race_nums[-1]).click()#何Rか
                # driver.find_element(By.PARTIAL_LINK_TEXT,racenum_obj.text).click()#何Rか
                time.sleep(2)
                driver.find_element(By.PARTIAL_LINK_TEXT,'単勝').click()
                time.sleep(2)
                b=driver.find_element(By.CLASS_NAME,'selectHorse').find_elements(By.CLASS_NAME,'ui-link')
                time.sleep(2)

                odds_df = pd.DataFrame()
                horseNumber=[]#馬番
                horseName=[]#馬名
                odds=[]#オッズ
                torikeshi=False#出走取り消しが存在するか

                for n in range(len(b)):
                    x=b[n].text.split('\n')
                    horseNumber.append(x[0])
                    horseName.append(x[1])
                    if x[2]!='取消':
                        odds.append(x[2])
                    else:
                        odds.append('-')
                        torikeshi=True

                odds_df = pd.DataFrame([horseNumber,horseName,odds],index=["馬番", "馬名", "単勝オッズ"] ).T
                odds_df.to_pickle('pickle/test/{0}/latest_odds_{1}_{2}.pickle'.format(date,places[-1],race_nums[-1]))

                driver.find_element(By.PARTIAL_LINK_TEXT,'式別').click()
                time.sleep(2)
                driver.find_element(By.PARTIAL_LINK_TEXT,'レース').click()
                time.sleep(2)

            driver.find_element(By.PARTIAL_LINK_TEXT,'競馬場名').click()
            time.sleep(2)

        else:
            time.sleep(2)
            continue

    driver.find_element(By.LINK_TEXT,'トップメニュー').click()#通常投票画面に返る
    print('完了しました。')

## resultの作成

In [ ]:
"""
rdataの作成19-22最新まで(常に直前まで更新が望ましい)
SPI算出にr.data_pを使用
カテゴリデータ作成にr.data_peを使用
"""
r = Results.read_pickle(['pickle/results/results_2021.pickle'])
r.data_p = pd.read_pickle('pickle/r_data19_21/r_data_p.pickle')
r.data_pe = pd.read_pickle('pickle/r_data19_21/r_data_pe.pickle')
r22_data_p= pd.read_pickle('pickle/r_data19_21/r_data_p22.pickle')
r22_data_pe  = pd.read_pickle('pickle/r_data19_21/r_data_pe22.pickle')
r.data_p= pd.concat([r.data_p,r22_data_p])
r.data_pe= pd.concat([r.data_pe,r22_data_pe])
del r22_data_p
del r22_data_pe
hr = HorseResults.read_pickle(['pickle/syutuba/latest/horse_results_latest.pickle'])
hr.preprocessing()
hr_p_backup = hr.horse_results_p.copy() #毎回作成する必要がないようにbuckupとして保持

## 開始前準備

In [ ]:
#race_idの作成(自動取得検討)
date = '2022年12月4日'
racenum = 12
race_id_list1 = ['2022060502'+'{}'.format(str(i).zfill(2)) for i in range(1, racenum+1, 1)]
race_id_list2 = ['2022090602'+'{}'.format(str(i).zfill(2)) for i in range(1, racenum+1, 1)]
race_id_list3 = ['2022070602'+'{}'.format(str(i).zfill(2)) for i in range(1, racenum+1, 1)]
all_race_id = race_id_list1 + race_id_list2 + race_id_list3
# all_race_id = race_id_list1 + race_id_list2

In [ ]:
making_dir(date)

In [ ]:
race_schedule = get_racetime(date)
day_schedule = create_day_schedule(all_race_id,race_schedule,3)
day_schedule

In [ ]:
scrape_time = day_schedule['get_race_time'].unique()
get_odds_time = day_schedule['get_odds_time'].unique()
print(scrape_time)
print(get_odds_time)

## 実行メイン処理

In [ ]:
schedule.clear()
endTime = "16:30"
isWorking = True
#レース情報のスクレイピング
for n in range(len(scrape_time)):
    try:
        race_id_list = day_schedule[day_schedule.index==n]['race_id']
        date_list = [date]*len(race_id_list)
        race_id_day_dict = dict(zip(race_id_list,date_list))
        schedule.every().day.at(scrape_time[n]).do(create_race_pred,race_id_day_dict,date,hr_p_backup,r)
    except  Exception as e:
        print(e)
        ws.PlaySound( 'SystemExclamation', ws.SND_ALIAS )
#最新オッズの取得と購入
for i in range(len(get_odds_time)):
    try:
        race_id = day_schedule[day_schedule['get_odds_time']==get_odds_time[i]]['race_id'].values[0]
        kaisai = day_schedule[day_schedule['get_odds_time']==get_odds_time[i]]['place'].values[0]
        R = day_schedule[day_schedule['get_odds_time']==get_odds_time[i]]['R'].values[0]
        schedule.every().day.at(get_odds_time[i]).do(Forecast_calculation,race_id,kaisai,R,date)
    except  Exception as e:
        print(e)
        ws.PlaySound( 'SystemExclamation', ws.SND_ALIAS )
schedule.every().day.at(endTime).do(end)
while isWorking:
    time.sleep(3)
    schedule.run_pending()
    time.sleep(10)#何秒ごとに処理を実行するかを入力

# 当日運用

## 保存先ディレクトリの作成

In [ ]:
# date = '2022年10月29日'
# race_num1 = '2022050408'
# race_num2 = '2022090408'
# race_num3 = '2022040405'
# os.makedirs('pickle/syutuba/race/'+date)
# os.makedirs('pickle/syutuba/race/'+date+'/std')
# os.makedirs('pickle/syutuba/race/'+date+'/first_result')
# os.makedirs('pickle/syutuba/生予測値/stdata_c/'+date)

## レースIDの作成

#### 複数レースの場合

In [ ]:
#開催場所に合わせて設定
racenum = 12
race_id_list1 = [race_num1+'{}'.format(str(i).zfill(2)) for i in range(1, racenum+1, 1)]
race_id_list2 = [race_num2+'{}'.format(str(i).zfill(2)) for i in range(1, racenum+1, 1)]
race_id_list3 = [race_num3+'{}'.format(str(i).zfill(2)) for i in range(1, racenum+1, 1)]
race_id_list = race_id_list1 + race_id_list2 + race_id_list3
# race_id_list =  race_id_list1 + race_id_list2

print(len(race_id_list))
date_list = [date]*len(race_id_list)
race_id_day_dict = dict(zip(race_id_list,date_list))
race_id_day_dict

#### 中間レースから複数レースを対象とする場合

In [ ]:
race_id_day_dict=[]
race_id_list=[]
start = 8
end = 12
for i in range(start,end+1,1):
    print(i)
    race_id_list.append(race_num1+str(i).zfill(2))
    race_id_list.append(race_num2+str(i).zfill(2))
    race_id_list.append(race_num3+str(i).zfill(2))
date_list = [date]*len(race_id_list)
race_id_day_dict = dict(zip(race_id_list,date_list))
race_id_day_dict

#### 1レースの場合

In [ ]:
#日付とレースIDと何レース目か
# date = '2022年10月23日'
race_num1 = '2022060501'
race_num2 = '2022090601'
race_num3 = '2022070601'
num = '10'
# race_id_list = [race_num1+num,race_num2+num,race_num3+num]
race_id_list = [race_num1+num,race_num2+num]
print(len(race_id_list))
date_list = [date]*len(race_id_list)
race_id_day_dict = dict(zip(race_id_list,date_list))
race_id_day_dict

## レーススクレイピングと予想

In [ ]:
%%time
first_result = create_race_pred(race_id_day_dict,date,hr_p_backup,r)

In [ ]:
first_result

In [ ]:
return_pred = return_model(first_result, 0, 90000)
return_pred

In [ ]:
top_pred = top_model(first_result, 0, 90000)
top_pred

In [ ]:
hybrid_model(first_result,return_pred,top_pred,100)

In [ ]:
index_model(first_result)

## 予想表のオッズを直近オッズに入れ替える

In [ ]:
first_result=pd.DataFrame()
path_list=[]
#指定したレースのみの場合
# for num in race_id_day_dict.keys():
#     path_list.append('pickle/syutuba/race/'+date+'/first_result/first_result_{}.pickle'.format(num))
#全体の場合
files = glob.glob('pickle/syutuba/race/'+date+'/first_result/*.pickle')
for file in files:
    path_list.append(file)    
first_result = pd.read_pickle(path_list[0])
for path in path_list[1:]:
    first_result = pd.concat([first_result, pd.read_pickle(path)])

In [ ]:
path_list=[]
files = glob.glob('pickle/syutuba/race/'+date+'/latest_odds/*.pickle')
for file in files:
    path_list.append(file)
latest_odds = pd.read_pickle(path_list[0])
for path in path_list[1:]:
    latest_odds = pd.concat([latest_odds, pd.read_pickle(path)])

In [ ]:
latest_odds[latest_odds['馬名']=='オーソレミオ']

In [ ]:
first_result.rename(columns = {'馬名_x':'馬名'},inplace=True)
first_result['馬番'] = first_result['馬番'].astype(str)
first_result = first_result.merge(latest_odds,how='left',on=['馬番','馬名'])#マージする場合
first_result.drop(columns='単勝オッズ_x',inplace=True)
first_result.rename(columns = {'単勝オッズ_y':'単勝オッズ'},inplace=True)

# 12R終了後全レースを一括でスクレイプして予測値を保存する

## 作成

In [ ]:
print(date)
os.makedirs('pickle/syutuba/race/'+date+'/total')
os.makedirs('pickle/syutuba/race/'+date+'/total/first_result')
os.makedirs('pickle/syutuba/生予測値/Label_Encoder/'+date)

In [ ]:
"""
rdataの作成19-22最新まで(常に直前まで更新が望ましい)
SPI算出にr.data_pを使用
カテゴリデータ作成にr.data_peを使用
"""
r = Results.read_pickle(['pickle/results/results_2021.pickle'])
r.data_p = pd.read_pickle('pickle/r_data19_21/r_data_p.pickle')
r.data_pe = pd.read_pickle('pickle/r_data19_21/r_data_pe.pickle')
r22_data_p= pd.read_pickle('pickle/r_data19_21/r_data_p22.pickle')
r22_data_pe  = pd.read_pickle('pickle/r_data19_21/r_data_pe22.pickle')
r.data_p= pd.concat([r.data_p,r22_data_p])
r.data_pe= pd.concat([r.data_pe,r22_data_pe])
del r22_data_p
del r22_data_pe

In [ ]:
#開催場所に合わせて設定
racenum = 12
race_num1 = '2022060502'
race_num2 = '2022090602'
race_num3 = '2022070602'
race_id_list1 = [race_num1+'{}'.format(str(i).zfill(2)) for i in range(1, racenum+1, 1)]
race_id_list2 = [race_num2+'{}'.format(str(i).zfill(2)) for i in range(1, racenum+1, 1)]
race_id_list3 = [race_num3+'{}'.format(str(i).zfill(2)) for i in range(1, racenum+1, 1)]
race_id_list = race_id_list1 + race_id_list2 + race_id_list3
# race_id_list =  race_id_list1 + race_id_list2
print(len(race_id_list))
date_list = [date]*len(race_id_list)
race_id_day_dict = dict(zip(race_id_list,date_list))
race_id_day_dict

In [ ]:
%%time
#学習モデル更新済み
def day_after_create_race_pred(race_id_day_dict,date,hr_p_backup,r,already=False):
    now = datetime.now()
    current_time = now.strftime("%Y%m%d%H%M%S")
    #初めての時はスクレイピングを実行
    if already==False:
        st = ShutubaTable.scrape(race_id_day_dict)
        for rid,date in race_id_day_dict.items():
            st.data[st.data.index==rid].to_pickle('pickle/syutuba/race/{0}/total/syutuba_{1}.pickle'.format(date,rid))
    else:
        path_list=[]
        files = glob.glob('pickle/syutuba/race/'+date+'/total/*.pickle')
        # files = glob.glob('pickle/syutuba/race/'+date+'/*.pickle')#検証のために実施path
        for file in files:
            path_list.append(file)
        path_list
        st = ShutubaTable.read_pickle(path_list)
        
    #不足データがある場合、スクレイピングされ常に更新されていく
    hi = HorseInfo.read_pickle(['pickle/syutuba/latest/h_info_latest.pickle'])
    st.merge_horse_info(hi)
    jr = jockey_results.read_pickle(['pickle/syutuba/latest/jockey_id_latest.pickle'])
    st.merge_jockey_info(jr,5)
    st.preprocessing()
    hr.horse_results_p = hr_p_backup.copy()
    past_race_num=10
    st.merge_horse_results(hr, n_samples_list=[5,9,'all'] ,rt = r.data_p)
    st.concat_past_data(hr,past_race_num)
    p = Peds.read_pickle(['pickle/syutuba/latest/peds_id_latest.pickle'])
    p.encode(st)
    st.merge_peds(p.peds_e)

    def Read_Label_Encorder(r):
        with open('pickle/LabelEncoder/le_horse.pickle', 'rb') as f:
            r.le_horse = pickle.load(f)
        with open('pickle/LabelEncoder/le_jockey.pickle', 'rb') as f:
            r.le_jockey = pickle.load(f)
        with open('pickle/LabelEncoder/le_class.pickle', 'rb') as f:
            r.le_class = pickle.load(f)
        with open('pickle/LabelEncoder/le_place.pickle', 'rb') as f:
            r.le_place = pickle.load(f)
        with open('pickle/LabelEncoder/le_trainer.pickle', 'rb') as f:
            r.le_trainer = pickle.load(f)
        with open('pickle/LabelEncoder/le_horseowner.pickle', 'rb') as f:
            r.le_horseowner = pickle.load(f)
        with open('pickle/LabelEncoder/le_producer.pickle', 'rb') as f:
            r.le_producer = pickle.load(f)
        with open('pickle/LabelEncoder/le_relative1.pickle', 'rb') as f:
            r.le_relative1 = pickle.load(f)
        with open('pickle/LabelEncoder/le_relative2.pickle', 'rb') as f:
            r.le_relative2 = pickle.load(f)
        with open('pickle/LabelEncoder/le_kai.pickle', 'rb') as f:
            r.le_kai = pickle.load(f)
        with open('pickle/LabelEncoder/le_day.pickle', 'rb') as f:
            r.le_day = pickle.load(f)
    Read_Label_Encorder(r)

    st.process_categorical\
    (r.le_horse, r.le_jockey, r.le_class, r.le_place, r.data_pe, r.le_trainer, r.le_horseowner,\
     r.le_producer, r.le_relative1, r.le_relative2, r.le_kai, r.le_day, past_race_num)

    latest_colcategory = pd.read_pickle('pickle/st_data/st_data_c_latest_categories.pickle')
    # latest_colcategory = pd.read_pickle('pickle/st_data/category_std/st_data20220903.pickle')

    for index ,name in latest_colcategory.head(0).dtypes.iteritems():
        if name=='category':
            # print(index)
            new = list(st.data_c[index].cat.categories)
            moto = list(latest_colcategory[index].cat.categories)
            number = 0
            befor = 0
            changeFLG=False
            A,B,C,D=[],[],[],[]
            D = list(set(new)-set(moto))#完全に新しく出てきたもの
            # if len(D)>0:
            #     print('new_label_encode',index,D)
            for i in moto:
                if befor <= i:
                    befor = i
                    number+=1
                else:
                    changeFLG = True
                    A = moto[:number]#前半 直前まで出たもの
                    B = moto[number:]#後半 2021まで出ていたもの
                    C = list(set(B)&set(new))#今回新たに出ているが過去にも出たことのあるもの
                    # for s in C:
                    #     B.remove(s)
                    # B = [s for s in B if s not in C]
                    list(map(lambda x: B.remove(x), C))
            if changeFLG==False:
                A = moto[:number]#前半 直前まで出たもの
                B = moto[number:]#後半 2021まで出ていたもの
                C = list(set(B)&set(new))#今回新たに出ているが過去にも出たことのあるもの
                list(map(lambda x: B.remove(x), C))
    
            setlist = (sorted(A+C)+sorted(D)+(B))
            # time_protcol()

            if len(setlist)==0:#何も更新がなかった場合
                # print('更新なし',index)
                st.data_c[index] =st.data_c[index].cat.set_categories((moto))
            else:
                #前半部分に新しいリストを合わせて昇順にする
                st.data_c[index] =st.data_c[index].cat.set_categories((setlist))

    def std_data_proccesing(st):
        #並び替えとindexのsetとタイトル削除
        st.data_c=st.data_c.sort_values(['race_id','馬番'])
        st.data_c.set_index('race_id',inplace = True)
        st.data_c =st.data_c.rename_axis(index=None)

        #特徴量を削除
        # st.data_c.drop(columns =['time','final_to_rank_my_allR','jockey_id'],inplace = True)
        st.data_c.drop(columns =['time'],inplace = True)

        #レースIDごとに標準偏差に変更する
        std_tar_col =['枠番', 'first_to_final_course_len_5R', 'first_corner_race_type_9R', 'final_to_rank_8p', '体重_8p']
        # std_tar_col =['枠番', 'first_to_final_course_len_5R', 'first_corner_race_type_9R']
        standard_scaler = lambda x: (x - x.mean()) / x.std()
        st.data_c[list(map(lambda x:  x+'_std', std_tar_col))] = st.data_c[std_tar_col].groupby(level=0).transform(standard_scaler)

        #学習モデルの特徴量を合わせるために、保存したデータを読み込み
        col_df = pd.read_pickle('pickle/syutuba/columns/r_columns.pickle')
        print(set(col_df.columns.tolist())-set(st.data_c.columns.tolist()))
        print(set(st.data_c.columns.tolist())-set(col_df.columns.tolist()))

        #不足分をリスト化して追加する
        col = set(col_df.columns)-set(st.data_c.columns)
        st.data_c[list(col)]=0
    std_data_proccesing(st)
    
    if already==False:
        st.data_c.to_pickle('pickle/syutuba/生予測値/stdata_c/{}/stdata_c_std.pickle'.format(date))
    # else:#検証のために実施
    #     st.data_c.to_pickle('pickle/syutuba/生予測値/stdata_c/{}/stdata_c_std_実施値.pickle'.format(date))

    def Created_predscore(st, r, returnModel, top4Model):
        stX_test = st.data_c.drop(['着順', '人気','date','生年月日'], axis=1)
        # stX_test = test.drop(['着順', '人気','date','生年月日'], axis=1)

        #学習モデルの読み込み
        lgb_clf_return = pickle.load(open('./pickle/syutuba/学習モデル/'+returnModel+'.pkl', 'rb'))
        lgb_clf_top4 = pickle.load(open('./pickle/syutuba/学習モデル/'+top4Model+'.pkl', 'rb'))

        #ModelEvaluatorクラスのオブジェクトを作成
        #予想時にはreturn情報はないので、適当によみこむ
        me_st_return = ModelEvaluator(lgb_clf_return, ['pickle/syutuba/latest/return_table_latest.pickle'])
        me_st_top4 = ModelEvaluator(lgb_clf_top4, ['pickle/syutuba/latest/return_table_latest.pickle'])

        #predの算出
        scores = me_st_return.predict_proba(stX_test)
        pred = stX_test[['馬番','horse_id','class','place','jockey_id']].copy()
        #ラベルデコード
        pred['le_horse_id']=r.le_horse.inverse_transform(pred['horse_id'])
        pred['le_class']=r.le_class.inverse_transform(pred['class'])
        pred['le_place']=r.le_place.inverse_transform(pred['place'])
        pred['le_jockey_id']=r.le_jockey.inverse_transform(pred['jockey_id'])
        pred['score'] = scores
        pred_return = pred.copy()
        #predの算出
        scores = me_st_top4.predict_proba(stX_test)
        pred = stX_test[['馬番','horse_id','class','place','jockey_id']].copy()
        #ラベルデコード
        pred['le_horse_id']=r.le_horse.inverse_transform(pred['horse_id'])
        pred['le_class']=r.le_class.inverse_transform(pred['class'])
        pred['le_place']=r.le_place.inverse_transform(pred['place'])
        pred['le_jockey_id']=r.le_jockey.inverse_transform(pred['jockey_id'])
        pred['score'] = scores
        pred_top4 = pred.copy()

        #mergeするためにindexをカラムにして、merge後元に戻す
        st.data.reset_index(inplace=True)
        st.data.rename(columns={'index': 'race_id'},inplace=True)
        pred_return.reset_index(inplace=True)
        pred_return.rename(columns={'index': 'race_id'},inplace=True)
        pred_top4.reset_index(inplace=True)
        pred_top4.rename(columns={'index': 'race_id'},inplace=True)

        AI_return = pred_return.merge(st.data[['horse_id','馬名']].drop_duplicates(),how='left',left_on=['le_horse_id'],right_on=['horse_id'])
        AI_top4 = pred_top4.merge(st.data[['horse_id','馬名']].drop_duplicates(),how='left',left_on=['le_horse_id'],right_on=['horse_id'])
        AI_top4 = AI_top4.merge(st.data[['race_id','date']].drop_duplicates(),how='left',left_on=['race_id'],right_on=['race_id'])

        st.data.set_index('race_id',inplace = True)
        st.data.index.name = None
        pred_return.set_index('race_id',inplace = True)
        pred_return.index.name = None
        pred_top4.set_index('race_id',inplace = True)
        pred_top4.index.name = None

        AI_return.drop(columns=['horse_id_x','class','le_horse_id','horse_id_y'],inplace=True)
        AI_return = AI_return.reindex(columns=['race_id','le_place','馬番','馬名','le_jockey_id','le_class','score'])

        AI_top4.drop(columns=['horse_id_x','class','le_horse_id','horse_id_y'],inplace=True)
        AI_top4 = AI_top4.reindex(columns=['race_id','le_place','馬番','馬名','le_jockey_id','le_class','score'])

        place_dict = {
        '01':'札幌',  '02':'函館',  '03':'福島',  '04':'新潟',  '05':'東京', 
        '06':'中山',  '07':'中京',  '08':'京都',  '09':'阪神',  '10':'小倉'
        }
        AI_return = AI_return.replace(place_dict)
        AI_top4 = AI_top4.replace(place_dict)

        AI = AI_top4.merge(AI_return[['race_id','馬名','score']],how='left',left_on=['race_id','馬名'],right_on=['race_id','馬名'])
        AI.rename(columns={'馬名_x':'馬名', 'score_x': 'score_top4', 'score_y': 'score_return'},inplace=True)
        AI.insert(loc=3,column='R',value=AI['race_id'].str[-2:])
        return AI
    
    # returnModel = '19-21学習モデル_750'
    # top4Model = '19-21的中型上位4位モデル'
    returnModel = '221101return_model'
    top4Model = '221101top4_model'
    AI = Created_predscore(st, r, returnModel, top4Model)
    
    def concate_odds(race_id_day_dict,current_time,AI):
        if already==False:
            st_odds = ST_odds.scrape(race_id_day_dict.keys())
            st_odds.data.to_pickle('pickle/syutuba/race/odds/st_odds_{}.pickle'.format(current_time))
            STR=ST_odds.read_pickle(['pickle/syutuba/race/odds/st_odds_latest.pickle','pickle/syutuba/race/odds/st_odds_{}.pickle'.format(current_time)])
            STR.data.to_pickle('pickle/syutuba/race/odds/st_odds_latest.pickle')
        else:
            STR=ST_odds.read_pickle(['pickle/syutuba/race/odds/st_odds_latest.pickle'])
        STR.data.reset_index(inplace=True)
        STR.data.rename(columns={'index': 'race_id'},inplace=True)
        add_odds = AI.merge(STR.data,how='left',left_on=['race_id','馬番'],right_on=['race_id','馬番'])
        add_odds.drop(columns = '馬名_y' ,inplace = True)
        rankdf = add_odds.copy()
        rankdf['score_rank'] = add_odds.groupby(by = 'race_id')['score_top4'].rank(ascending=False)
        rankdf['return_rank'] = add_odds.groupby(by = 'race_id')['score_return'].rank(ascending=False)
        first_result = rankdf.sort_values(['le_place','R','馬番'], ascending=True).copy()
        return first_result
    first_result = concate_odds(race_id_day_dict, current_time, AI)
    
    #予測値の保存
    if already==False:
        for rid,date in race_id_day_dict.items():
            first_result[first_result['race_id']==rid].to_pickle('pickle/syutuba/race/{0}/total/first_result/first_result_{1}.pickle'.format(date,rid))
    return first_result
# first_result = day_after_create_race_pred(race_id_day_dict, date, hr_p_backup, r ,False)
# return_pred = return_model(first_result, 0, 90000)
# top_pred = top_model(first_result, 0, 90000)
# hybrid = hybrid_model(first_result,return_pred,top_pred,0)

In [ ]:
first_result = day_after_create_race_pred(race_id_day_dict, date, hr_p_backup, r ,False)

In [ ]:
#ラベルエンコーディングを保存
def Save_Label_Encoder(r,date):
    with open('pickle/syutuba/生予測値/Label_Encoder/{}/le_horse.pickle'.format(date), 'wb') as f:
        pickle.dump(r.le_horse , f)
    with open('pickle/syutuba/生予測値/Label_Encoder/{}/le_jockey.pickle'.format(date), 'wb') as f:
        pickle.dump(r.le_jockey , f)
    with open('pickle/syutuba/生予測値/Label_Encoder/{}/le_class.pickle'.format(date), 'wb') as f:
        pickle.dump(r.le_class , f)
    with open('pickle/syutuba/生予測値/Label_Encoder/{}/le_place.pickle'.format(date), 'wb') as f:
        pickle.dump(r.le_place , f)
    with open('pickle/syutuba/生予測値/Label_Encoder/{}/le_trainer.pickle'.format(date), 'wb') as f:
        pickle.dump(r.le_trainer , f)
    with open('pickle/syutuba/生予測値/Label_Encoder/{}/le_horseowner.pickle'.format(date), 'wb') as f:
        pickle.dump(r.le_horseowner , f)
    with open('pickle/syutuba/生予測値/Label_Encoder/{}/le_producer.pickle'.format(date), 'wb') as f:
        pickle.dump(r.le_producer , f)
    with open('pickle/syutuba/生予測値/Label_Encoder/{}/le_relative1.pickle'.format(date), 'wb') as f:
        pickle.dump(r.le_relative1 , f)
    with open('pickle/syutuba/生予測値/Label_Encoder/{}/le_relative2.pickle'.format(date), 'wb') as f:
        pickle.dump(r.le_relative2 , f)
    with open('pickle/syutuba/生予測値/Label_Encoder/{}/le_kai.pickle'.format(date), 'wb') as f:
        pickle.dump(r.le_kai , f)
    with open('pickle/syutuba/生予測値/Label_Encoder/{}/le_day.pickle'.format(date), 'wb') as f:
        pickle.dump(r.le_day , f)
Save_Label_Encoder(r,date)

## 予測値の確認

In [ ]:
return_pred = return_model(first_result, 0, 90000)
return_pred.sort_values(by = ['le_place','R'], ascending=True)

In [ ]:
top_pred = top_model(first_result, 0, 0)
top_pred.sort_values(by = ['le_place','R'], ascending=True)

In [ ]:
hybrid_model(first_result,return_pred,top_pred,0)
# hybrid.sort_values(by = ['le_place','R'], ascending=False)

In [ ]:
index_model(first_result)

In [ ]:
hybrid2_model(first_result)

## 生データでの対象馬確認

In [ ]:
date = '2022年11月27日'

In [ ]:
#毎レーススクレイピング時のpred
raw_pred=pd.DataFrame()
path_list=[]
#指定したレースのみの場合
# for num in race_id_day_dict.keys():
#     path_list.append('pickle/syutuba/race/'+date+'/first_result/first_result_{}.pickle'.format(num))
#全体の場合
files = glob.glob('pickle/syutuba/race/'+date+'/first_result/*.pickle')
for file in files:
    path_list.append(file)    
raw_pred = pd.read_pickle(path_list[0])
for path in path_list[1:]:
    raw_pred = pd.concat([raw_pred, pd.read_pickle(path)])

In [ ]:
#毎レースの直前オッズ
latest_odds=pd.DataFrame()
path_list=[]
files = glob.glob('pickle/syutuba/race/'+date+'/latest_odds/*.pickle')
for file in files:
    path_list.append(file)    
latest_odds = pd.read_pickle(path_list[0])
for path in path_list[1:]:
    latest_odds = pd.concat([latest_odds, pd.read_pickle(path)])
latest_odds['馬番'] = latest_odds['馬番'].astype(int)

In [ ]:
raw = raw_pred.merge(latest_odds,how = 'left',left_on=['馬番','馬名_x'],right_on=['馬番','馬名'])
raw = raw.drop(['馬名','単勝オッズ_x'],axis=1)
raw = raw.rename(columns={"単勝オッズ_y": "単勝オッズ"})

In [ ]:
return_model(raw, 0, 90000)

In [ ]:
top_model(raw, 0, 90000)

In [ ]:
hybrid_model(raw,return_pred,top_pred,0)

In [ ]:
index_model(raw)

In [ ]:
hybrid2_model(raw)

# 当日終了後、出馬表の結果からシミュレーションする

## 出馬表の再更新

In [ ]:
race_id_day_dict

In [ ]:
""" 
#その日に開催されるレースIDと日付けを指定する
#日付は予測値に影響を与えるので注意(hrのマージの日付になる)
#レース終了後日付に注意して再取得し、latestを更新する
"""
date = '2022年11月27日'
#開催場所に合わせて設定
racenum = 12
race_id_list1 = ['2022050508{}'.format(str(i).zfill(2)) for i in range(1, racenum+1, 1)]
race_id_list2 = ['2022090508{}'.format(str(i).zfill(2)) for i in range(1, racenum+1, 1)]
# race_id_list3 = ['2022030306{}'.format(str(i).zfill(2)) for i in range(1, racenum+1, 1)]
# race_id_list = race_id_list1 + race_id_list2 + race_id_list3
race_id_list = race_id_list1 + race_id_list2

print(len(race_id_list))
date_list = [date]*len(race_id_list)
race_id_day_dict = dict(zip(race_id_list,date_list))
race_id_day_dict

In [ ]:
#スクレイピングして保存
st = ShutubaTable.scrape(race_id_day_dict,False)
st.data.to_pickle('pickle/syutuba/race/syutuba_{}.pickle'.format(date))

In [ ]:
#latestを更新
st = ShutubaTable.read_pickle(['pickle/syutuba/race/syutuba_latest.pickle','pickle/syutuba/race/syutuba_{}.pickle'.format(date)])
st.data.to_pickle('pickle/syutuba/race/syutuba_latest.pickle')

In [ ]:
st = ShutubaTable.read_pickle(['pickle/syutuba/race/syutuba_latest.pickle'])

In [ ]:
st.data['date'].unique()

## 結果表の再更新

In [ ]:
st=ShutubaTable.read_pickle(['pickle/syutuba/race/syutuba_latest.pickle'])
STR=ST_results.read_pickle(['pickle/syutuba/race/STR/STR_latest.pickle'])
diff_race_id = set(st.data.index.unique())-set(STR.data['race_id'].unique())
print(len(diff_race_id))

In [ ]:
#日付はSTRにないため関係ないが保存名になる
# date = '2022年11月20日'
print(date)
#出馬表からのオッズの取得
STR = ST_results.scrape(diff_race_id)
#STRの保存
STR.data.to_pickle('pickle/syutuba/race/STR/STR_{}.pickle'.format(date))
STR=ST_results.read_pickle(['pickle/syutuba/race/STR/STR_latest.pickle','pickle/syutuba/race/STR/STR_{}.pickle'.format(date)])

In [ ]:
#STRの更新
STR.data.to_pickle('pickle/syutuba/race/STR/STR_latest.pickle')
STR=ST_results.read_pickle(['pickle/syutuba/race/STR/STR_latest.pickle'])

## AIの作成

In [ ]:
#2021まででカテゴリデータを作成すること
#22以降はstで1日ずつ作成する必要があるため
r = Results.read_pickle(['pickle/results/results_2021.pickle'])
r.data_p = pd.read_pickle('pickle/r_data19_21/r_data_p.pickle')
r.data_pe = pd.read_pickle('pickle/r_data19_21/r_data_pe.pickle')
past_race_num=10
r.process_categorical(past_race_num)

r22_data_p= pd.read_pickle('pickle/r_data19_21/r_data_p22.pickle')
r22_data_pe  = pd.read_pickle('pickle/r_data19_21/r_data_pe22.pickle')
r.data_p= pd.concat([r.data_p,r22_data_p])
r.data_pe= pd.concat([r.data_pe,r22_data_pe])
del r22_data_p
del r22_data_pe

In [ ]:
hr = HorseResults.read_pickle(['pickle/syutuba/latest/horse_results_latest.pickle'])
hr.preprocessing()
hr_p_backup = hr.horse_results_p.copy()

In [ ]:
st = ShutubaTable.read_pickle(['pickle/syutuba/race/syutuba_latest.pickle'])#26/18hukumu
#不足データがある場合、スクレイピングされ常に更新されていく
hi = HorseInfo.read_pickle(['pickle/syutuba/latest/h_info_latest.pickle'])
st.merge_horse_info(hi)
jr = jockey_results.read_pickle(['pickle/syutuba/latest/jockey_id_latest.pickle'])
st.merge_jockey_info(jr,5)
st.preprocessing()

In [ ]:
"""stのスクレイプで日にちを間違えた場合にrとの整合性が取れずデータが
重複してしまっている可能性があるためチェックする"""
for i in st.data[['date','horse_id','馬番']].value_counts().tolist():
    if i != 1:
        print(i)

In [ ]:
st.data['date'].unique()

In [ ]:
copydf = st.data_p.copy()
backup = st.data_p.copy()
hr_p_backup = hr.horse_results_p.copy()

In [ ]:
#一日ごとにSPIを算出する
#stのその日以前のrを使用してSPI算出している
stdata_days = pd.DataFrame()
daylist = copydf.sort_values('date',ascending = True)['date'].unique()
for day in daylist:
    st.data_p = copydf[copydf['date']==day]
    st.merge_horse_results(hr, n_samples_list=[5,9,'all'] ,rt = r.data_p)
    hr.horse_results_p = hr_p_backup
    st.data_h[st.data_h['date']==day].to_pickle('pickle/syutuba/SPI/st_data_h_{}.pickle'.format(day.astype('M8[D]')))
    stdata_days = pd.concat([stdata_days,st.data_h[st.data_h['date']==day]])
clear_output()
print(len(st.data_h))

In [ ]:
# spidatas = pd.DataFrame()
# daylist = copydf.sort_values('date',ascending = True)['date'].unique()
# for day in daylist[:-1]:
#     spidata = pd.read_pickle('pickle/syutuba/SPI/st_data_h_{}.pickle'.format(day.astype('M8[D]')))
#     spidatas = pd.concat([spidatas,spidata])
# #最新日のみ計算して結合
# st.data_p = copydf[copydf['date']==daylist[-1]]
# st.merge_horse_results(hr, n_samples_list=[5,9,'all'] ,rt = r.data_p)
# hr.horse_results_p = hr_p_backup
# st.data_h[st.data_h['date']==daylist[-1]].to_pickle('pickle/syutuba/SPI/st_data_h_{}.pickle'.format(daylist[-1].astype('M8[D]')))
# spidatas = pd.concat([spidatas,st.data_h[st.data_h['date']==daylist[-1]]])

In [ ]:
#日ごとに作成した場合バックアップでhrがリセットされているため下記処理実行
hr.horse_results_p = hr.horse_results_p.reset_index()
jockey_df = r.data_p[['騎手','jockey_id']].drop_duplicates()
hr.horse_results_p = hr.horse_results_p.merge(jockey_df, how='left', on = '騎手')
hr.horse_results_p = hr.horse_results_p.set_index('horse_id')

In [ ]:
past_race_num=10
# st.merge_horse_results(hr, n_samples_list=[5,9,'all'] ,rt = r.data_p)
st.data_h = stdata_days.copy()
st.concat_past_data(hr,past_race_num)

#5世代分の血統データの追加
#pedsが存在しない場合は自動更新
p = Peds.read_pickle(['pickle/syutuba/latest/peds_id_latest.pickle'])
p.encode(st)
st.merge_peds(p.peds_e)

In [ ]:
#カテゴリデータ更新の時に行う
#日ごとにカテゴライズしてラベルエンコーダーを作成する
#古い日から一日ごとにエンコードしないとカテゴリに振られる採番が異なってしまう
#ラベルエンコーディングとは、アルゴリズムに基づいて全データを順に並べ0から採番して識別すること
# date = '2022年11月27日'#ここが必要か確認すること
print(date)
def create_dir(date,mainday=True):
    if mainday==True:
        path = 'pickle/LabelEncoder/backup/'+date
    else:
        path = 'pickle/LabelEncoder/backup/'+mainday+'/'+date
    os.makedirs(path)
mainday = date#更新日
create_dir(mainday)
def Backup_Label_Encoder(r,mainday,date):
    with open('pickle/LabelEncoder/backup/'+mainday+'/'+date+'/le_horse.pickle', 'wb') as f:
        pickle.dump(r.le_horse , f)
    with open('pickle/LabelEncoder/backup/'+mainday+'/'+date+'/le_jockey.pickle', 'wb') as f:
        pickle.dump(r.le_jockey , f)
    with open('pickle/LabelEncoder/backup/'+mainday+'/'+date+'/le_class.pickle', 'wb') as f:
        pickle.dump(r.le_class , f)
    with open('pickle/LabelEncoder/backup/'+mainday+'/'+date+'/le_place.pickle', 'wb') as f:
        pickle.dump(r.le_place , f)
    with open('pickle/LabelEncoder/backup/'+mainday+'/'+date+'/le_trainer.pickle', 'wb') as f:
        pickle.dump(r.le_trainer , f)
    with open('pickle/LabelEncoder/backup/'+mainday+'/'+date+'/le_horseowner.pickle', 'wb') as f:
        pickle.dump(r.le_horseowner , f)
    with open('pickle/LabelEncoder/backup/'+mainday+'/'+date+'/le_producer.pickle', 'wb') as f:
        pickle.dump(r.le_producer , f)
    with open('pickle/LabelEncoder/backup/'+mainday+'/'+date+'/le_relative1.pickle', 'wb') as f:
        pickle.dump(r.le_relative1 , f)
    with open('pickle/LabelEncoder/backup/'+mainday+'/'+date+'/le_relative2.pickle', 'wb') as f:
        pickle.dump(r.le_relative2 , f)
    with open('pickle/LabelEncoder/backup/'+mainday+'/'+date+'/le_kai.pickle', 'wb') as f:
        pickle.dump(r.le_kai , f)
    with open('pickle/LabelEncoder/backup/'+mainday+'/'+date+'/le_day.pickle', 'wb') as f:
        pickle.dump(r.le_day , f)
st_data_pe_bk = st.data_pe.copy()
condf = pd.DataFrame()
past_race_num=10
print('le_horse',len(r.le_horse.classes_))
for date in tqdm(st_data_pe_bk.sort_values('date',ascending = True)['date'].dt.strftime("%Y%m%d").unique()):
    st.data_pe=st_data_pe_bk[st_data_pe_bk['date']<=date]
    # print(len(st.data_pe))
    # print('le_horse',len(r.le_horse.classes_))
    st.process_categorical\
    (r.le_horse,r.le_jockey,r.le_class,r.le_place,r.data_pe,r.le_trainer,r.le_horseowner,\
     r.le_producer,r.le_relative1,r.le_relative2,r.le_kai,r.le_day,past_race_num)
    # print('st_le_horse',len(st.le_horse.classes_))
    #concatする場合
    # condf = pd.concat([condf,st.data_c[st.data_c['date']==date]])
    #一日ごとに保存
    st.data_c[st.data_c['date']==date].to_pickle('pickle/st_data/category/st_data{}.pickle'.format(date))
    create_dir(date,mainday)
    Backup_Label_Encoder(r,mainday,date)
print('st_le_horse',len(st.le_horse.classes_))

In [ ]:
print('le_horse',len(r.le_horse.classes_))

## ラベルエンコーダーの保存

In [ ]:
def Save_Label_Encoder(r):
    with open('pickle/LabelEncoder/le_horse.pickle', 'wb') as f:
        pickle.dump(r.le_horse , f)
    with open('pickle/LabelEncoder/le_jockey.pickle', 'wb') as f:
        pickle.dump(r.le_jockey , f)
    with open('pickle/LabelEncoder/le_class.pickle', 'wb') as f:
        pickle.dump(r.le_class , f)
    with open('pickle/LabelEncoder/le_place.pickle', 'wb') as f:
        pickle.dump(r.le_place , f)
    with open('pickle/LabelEncoder/le_trainer.pickle', 'wb') as f:
        pickle.dump(r.le_trainer , f)
    with open('pickle/LabelEncoder/le_horseowner.pickle', 'wb') as f:
        pickle.dump(r.le_horseowner , f)
    with open('pickle/LabelEncoder/le_producer.pickle', 'wb') as f:
        pickle.dump(r.le_producer , f)
    with open('pickle/LabelEncoder/le_relative1.pickle', 'wb') as f:
        pickle.dump(r.le_relative1 , f)
    with open('pickle/LabelEncoder/le_relative2.pickle', 'wb') as f:
        pickle.dump(r.le_relative2 , f)
    with open('pickle/LabelEncoder/le_kai.pickle', 'wb') as f:
        pickle.dump(r.le_kai , f)
    with open('pickle/LabelEncoder/le_day.pickle', 'wb') as f:
        pickle.dump(r.le_day , f)
Save_Label_Encoder(r)

## for文によるカテゴリデータの作成とAIの作成
　これで作るとカテゴリデータの順番が一定になりpredがぶれなくなる

In [ ]:
#上記stデータで2022年データのカテゴリデータを作成したので
#それに2021年のカテゴリデータをくっつける作業
from pandas.api.types import CategoricalDtype
# std_df = st.data_c.copy()
#21年までのカテゴリデータ情報
col_data = pd.read_pickle('pickle/st_data/st_data_c_19-21_categories.pickle')
#学習モデルの特徴量(カラム数)を合わせるために、保存したデータを読み込み
col_df = pd.read_pickle('pickle/syutuba/columns/r_columns.pickle')

for date in tqdm(st_data_pe_bk.sort_values('date',ascending = True)['date'].dt.strftime("%Y%m%d").unique()):
    std_df = pd.read_pickle('pickle/st_data/category/st_data{}.pickle'.format(date))
    #並び替えとindexのsetとタイトル削除
    std_df=std_df.sort_values(['race_id','馬番'])
    std_df.set_index('race_id',inplace = True)
    std_df =std_df.rename_axis(index=None)

    #特徴量を削除
    # st0618.drop(columns =['time','final_to_rank_my_allR','jockey_id'],inplace = True)
    std_df.drop(columns =['time'],inplace = True)

    #レースIDごとに標準偏差に変更する
    std_tar_col =['枠番', 'first_to_final_course_len_5R', 'first_corner_race_type_9R', 'final_to_rank_8p', '体重_8p']
    # std_tar_col =['枠番', 'first_to_final_course_len_5R', 'first_corner_race_type_9R']
    standard_scaler = lambda x: (x - x.mean()) / x.std()
    std_df[list(map(lambda x:  x+'_std', std_tar_col))] = std_df[std_tar_col].groupby(level=0).transform(standard_scaler)

    #不足分をリスト化して追加する
    col = set(col_df.columns)-set(std_df.columns)
    std_df[list(col)]=0
    
    #21年のカテゴリデータを追加して統一する
    for index ,name in col_data.head(0).dtypes.iteritems():
        if name=='category':
            diff_category = list(set(col_data[index].cat.categories)-set(std_df[index].cat.categories))
            std_df[index] = std_df[index].cat.add_categories(diff_category)
            #ソートしてカテゴリを作成する場合
            # std_df[index] = std_df[index].cat.set_categories(sorted(list(std_df[index].cat.categories)))
    
    std_df.to_pickle('pickle/st_data/category_std/st_data{}.pickle'.format(date))

In [ ]:
#最新日のカテゴリデータ順で当日カテゴリデータの順を合わせるため保存
print(date)
no = pd.read_pickle('pickle/st_data/category_std/st_data{}.pickle'.format(date))
no.head(0).to_pickle('pickle/st_data/st_data_c_latest_categories.pickle')
no.head(0)['horse_id'].cat.categories

In [ ]:
#学習モデルの読み込み
lgb_clf_return = pickle.load(open('./pickle/syutuba/学習モデル/19-21学習モデル_750.pkl', 'rb'))
lgb_clf_top4 = pickle.load(open('./pickle/syutuba/学習モデル/19-21的中型上位4位モデル.pkl', 'rb'))
me_st_return = ModelEvaluator(lgb_clf_return, ['pickle/syutuba/latest/return_table_latest.pickle'])
me_st_top4 = ModelEvaluator(lgb_clf_top4, ['pickle/syutuba/latest/return_table_latest.pickle'])

for date in tqdm(st_data_pe_bk.sort_values('date',ascending = True)['date'].dt.strftime("%Y%m%d").unique()):
    std_df = pd.read_pickle('pickle/st_data/category_std/st_data{}.pickle'.format(date))
    #テストデータの作成と保存
    stX_test = std_df.drop(['着順', '人気','date','生年月日'], axis=1)
    scores = me_st_return.predict_proba(stX_test)
    pred = stX_test[['馬番','horse_id','class','place']].copy()
    #ラベルデコード
    pred['le_horse_id']=r.le_horse.inverse_transform(pred['horse_id'])
    pred['le_class']=r.le_class.inverse_transform(pred['class'])
    pred['le_place']=r.le_place.inverse_transform(pred['place'])
    pred['score'] = scores
    pred_return = pred.copy()
    #predの算出
    scores = me_st_top4.predict_proba(stX_test)
    pred = stX_test[['馬番','horse_id','class','place']].copy()
    #ラベルデコード
    pred['le_horse_id']=r.le_horse.inverse_transform(pred['horse_id'])
    pred['le_class']=r.le_class.inverse_transform(pred['class'])
    pred['le_place']=r.le_place.inverse_transform(pred['place'])
    pred['score'] = scores
    pred_top4 = pred.copy()
    #mergeするためにindexをカラムにして、merge後元に戻す
    st.data.reset_index(inplace=True)
    st.data.rename(columns={'index': 'race_id'},inplace=True)
    pred_return.reset_index(inplace=True)
    pred_return.rename(columns={'index': 'race_id'},inplace=True)
    pred_top4.reset_index(inplace=True)
    pred_top4.rename(columns={'index': 'race_id'},inplace=True)
    AI_return = pred_return.merge(st.data[['horse_id','馬名']].drop_duplicates(),how='left',left_on=['le_horse_id'],right_on=['horse_id'])
    AI_top4 = pred_top4.merge(st.data[['horse_id','馬名']].drop_duplicates(),how='left',left_on=['le_horse_id'],right_on=['horse_id'])
    # AI_top4 = AI_top4.merge(st.data[['race_id','date']].drop_duplicates(),how='left',left_on=['race_id'],right_on=['race_id'])
    AI_top4 = AI_top4.merge(st.data[['race_id','date','course_len','race_type','race_round','weather','ground_state']].drop_duplicates(),how='left',left_on=['race_id'],right_on=['race_id'])

    st.data.set_index('race_id',inplace = True)
    st.data.index.name = None
    pred_return.set_index('race_id',inplace = True)
    pred_return.index.name = None
    pred_top4.set_index('race_id',inplace = True)
    pred_top4.index.name = None

    AI_return.drop(columns=['horse_id_x','class','le_horse_id','horse_id_y'],inplace=True)
    AI_return = AI_return.reindex(columns=['race_id','le_place','馬番','馬名','le_class','score'])
    AI_top4.drop(columns=['horse_id_x','class','le_horse_id','horse_id_y'],inplace=True)
    # AI_top4 = AI_top4.reindex(columns=['race_id','date','le_place','馬番','馬名','le_class','score'])
    AI_top4 = AI_top4.reindex(columns=['race_id','date','le_place','馬番','馬名','le_class','score','course_len','race_type','race_round','weather','ground_state'])

    place_dict = {
    '01':'札幌',  '02':'函館',  '03':'福島',  '04':'新潟',  '05':'東京', 
    '06':'中山',  '07':'中京',  '08':'京都',  '09':'阪神',  '10':'小倉'
    }
    AI_return = AI_return.replace(place_dict)
    AI_top4 = AI_top4.replace(place_dict)
    AI = AI_top4.merge(AI_return[['race_id','馬名','score']],how='left',left_on=['race_id','馬名'],right_on=['race_id','馬名'])
    AI.rename(columns={'score_x': 'score_top4','score_y': 'score_return',},inplace=True)
    AI.insert(loc=3,column='R',value=AI['race_id'].str[-2:])
    
    AI.to_pickle('pickle/st_data/AI/AI{}.pickle'.format(date))

In [ ]:
#AIデータの結合処理
AI = pd.DataFrame()
for date in tqdm(st_data_pe_bk.sort_values('date',ascending = True)['date'].dt.strftime("%Y%m%d").unique()):
    read = pd.read_pickle('pickle/st_data/AI/AI{}.pickle'.format(date))
    AI = pd.concat([AI,read])

# 回収率の確認

## 前処理済みのAIを結合する場合

In [ ]:
aist = ShutubaTable.read_pickle(['pickle/syutuba/race/syutuba_latest.pickle'])
aist.data["date"] = pd.to_datetime(aist.data["date"], format="%Y年%m月%d日")
#AIデータの結合処理

AI = pd.DataFrame()
for date in tqdm(aist.data.sort_values('date',ascending = True)['date'].dt.strftime("%Y%m%d").unique()):
    read = pd.read_pickle('pickle/st_data/AI/AI{}.pickle'.format(date))
    AI = pd.concat([AI,read])
del aist    

## 確定オッズによる回収率の算出

In [ ]:
STR=ST_results.read_pickle(['pickle/syutuba/race/STR/STR_latest.pickle'])
add_odds = AI.merge(STR.data,how='left',left_on=['race_id','馬番'],right_on=['race_id','馬番'])
rankdf = add_odds.copy()
rankdf['score_rank'] = add_odds.groupby(by = 'race_id')['score_top4'].rank(ascending=False)
rankdf['return_rank'] = add_odds.groupby(by = 'race_id')['score_return'].rank(ascending=False)
first_result = rankdf.copy()
first_result['着順'] = pd.to_numeric(first_result['着順'], errors='coerce')
first_result.dropna(subset=['着順'], inplace=True)
first_result['着順'] = first_result['着順'].astype(int)

## 回収値保存先の作成

In [ ]:
date='20221204'
def make_dir(date):
    dir_path1 = 'note/回収率推移/'+date+'/score_return'
    dir_path2 = 'note/回収率推移/'+date+'/score_top4'
    dir_path3 = 'note/回収率推移/'+date+'/score_return/期待値変動型'
    dir_path4 = 'note/回収率推移/'+date+'/score_top4/期待値変動型'
    path_list = [dir_path1,dir_path2,dir_path3,dir_path4]
    for path in path_list:
        try:
            os.makedirs(path)
        except OSError as ex:
            print(ex)
            continue
    return [dir_path1,dir_path2,dir_path3,dir_path4]
path_list = make_dir(date)

## returnモデル期待値変動型での回収率

In [ ]:
def making_top3(first_result,flg=True):
    top3 = first_result[first_result['return_rank'] <= 3].sort_values(by = ['race_id','return_rank'], ascending=False)
    top3['期待値'] = top3['score_return']*top3['単勝オッズ']
    top3 = top3.sort_values('期待値', ascending=True)
    dropcol =['course_len', 'race_type', 'race_round', 'weather','ground_state']
    # dropcol = ['score_rank','return_rank','複勝オッズ_min','複勝オッズ_max']
    if flg:
        top3.drop(columns=dropcol,inplace=True)
    return top3
def set_expreturn(top3):
    #開催場別に期待値を設定する
    top3.loc[top3['le_place'] == '札幌', 'min_exp'] = 8.4
    top3.loc[top3['le_place'] == '札幌', 'max_exp'] = 11.4
    top3.loc[top3['le_place'] == '小倉', 'min_exp'] = 9.4
    top3.loc[top3['le_place'] == '小倉', 'max_exp'] = 12.4
    top3.loc[top3['le_place'] == '新潟', 'min_exp'] = 7.5
    top3.loc[top3['le_place'] == '新潟', 'max_exp'] = 10.5
    top3.loc[top3['le_place'] == '阪神', 'min_exp'] = 10.2
    top3.loc[top3['le_place'] == '阪神', 'max_exp'] = 13.2
    
    # top3.loc[top3['le_place'] == '中京', 'min_exp'] = 9.6
    # top3.loc[top3['le_place'] == '中京', 'max_exp'] = 11.6
    # top3.loc[top3['le_place'] == '中山', 'min_exp'] = 12.3
    # top3.loc[top3['le_place'] == '中山', 'max_exp'] = 14.3
    
    top3.loc[top3['le_place'] == '中京', 'min_exp'] = 12.2
    top3.loc[top3['le_place'] == '中京', 'max_exp'] = 15.2
    top3.loc[top3['le_place'] == '中山', 'min_exp'] = 11.1
    top3.loc[top3['le_place'] == '中山', 'max_exp'] = 14.1
    top3.loc[top3['le_place'] == '東京', 'min_exp'] = 11.8
    top3.loc[top3['le_place'] == '東京', 'max_exp'] = 14.8
    top3.loc[top3['le_place'] == '福島', 'min_exp'] = 14.9
    top3.loc[top3['le_place'] == '福島', 'max_exp'] = 17.9
    top3.loc[top3['le_place'] == '函館', 'min_exp'] = 5.2
    top3.loc[top3['le_place'] == '函館', 'max_exp'] = 8.2
    return top3
def return_expmodel(top3,return_money=10000):
    top3 = set_expreturn(top3)
    top3=top3[(top3['期待値']>=top3['min_exp']) & (top3['期待値']<=top3['max_exp'])].copy()
    top3=top3[top3['score_return']>0.7]#足きりスコア
    top3['minオッズ']=round(top3['min_exp']/top3['score_return'],2)
    top3['maxオッズ']=round(top3['max_exp']/top3['score_return'],2)
    top3['賭け金']=1/top3['単勝オッズ']*return_money
    top3['期待値'] = round(top3['期待値'],1)
    return top3

In [ ]:
return_money=10000
top3 = making_top3(first_result)
top3 = return_expmodel(top3,return_money)
print('回収率',top3[top3['着順']<=1]['単勝オッズ'].sum()/len(top3))
print('対象レース',len(top3))
print('的中レース',len(top3[top3['着順']<=1]))
print('全レース',first_result['race_id'].nunique())
print('対象レース割合',top3['race_id'].nunique()/first_result['race_id'].nunique())
print('的中率',(len(top3[top3['着順']<=1])/len(top3)))
print('単勝適正回収値',len(top3[top3['着順']==1])*return_money/top3['賭け金'].sum())
print(top3[top3['着順']==1]['単勝オッズ'].std())
with open(path_list[2]+'/overview.txt', 'w', encoding='sjis') as f:
    print('回収率',top3[top3['着順']<=1]['単勝オッズ'].sum()/len(top3),file=f)
    print('対象レース',len(top3),file=f)
    print('的中レース',len(top3[top3['着順']<=1]),file=f)
    print('全レース',first_result['race_id'].nunique(),file=f)
    print('対象レース割合',top3['race_id'].nunique()/first_result['race_id'].nunique(),file=f)
    print('的中率',(len(top3[top3['着順']<=1])/len(top3)),file=f)
    print('単勝適正回収値',len(top3[top3['着順']==1])*return_money/top3['賭け金'].sum(),file=f)
    seting_exp = top3[['le_place','min_exp','max_exp']]
    print('設定オッズ',seting_exp[~seting_exp.duplicated()],file=f)

In [ ]:
# 確率が１/１００の場合は、
# 　・１万回行えば、１/８０～１/１２０の誤差
# 的中率10%は1/10
# その100倍は1000回つまり、1000レースの予想を行うと95%の確率で±20%以内の確立になる
#よって1/8～1/12の確率(12.5％～0.83%)
# 母数の400倍の試行回数をこなすと95%の確率で誤差±10%以内の確率になる

In [ ]:
#scoreによる回収値
first_result['賭け金']=1/first_result['単勝オッズ']
a,x,y,z = [],[],[],[]
score_name = 'score_return'
smin = int(round(first_result[score_name].min(),1)*10)
smax = int(round(first_result[score_name].max(),1)*10)
min_odds = 5
max_odds = 30
for score in range(smin,smax):
    try:
        score = score/10
        df = first_result[first_result[score_name]>score]
        df = df[df['単勝オッズ']>=min_odds]
        df = df[df['単勝オッズ']<=max_odds]
        recovery_rate = round(df[df['着順']==1]['単勝オッズ'].sum()/len(df),2)
        hit_rate = round(len(df[df['着順']==1])/len(df),2)
        odds_std = round(df[df['着順']==1]['単勝オッズ'].std(),2)
        odds_max = round(df[df['着順']==1]['単勝オッズ'].max(),2)
        race_num = len(df)
        s_odds = len(df[df['着順']==1])/df['賭け金'].sum()
        x.append(score)
        y.append(recovery_rate)
        z.append(len(df)/35000)
        a.append(hit_rate)
        # if recovery_rate >1.2 and odds_std <20 and race_num>10 and hit_rate>0.1:
        if recovery_rate >1.2:
            print('score',score,'----------------------')
            print('回収率',recovery_rate)
            print('最大オッズ',odds_max)
            print('中央オッズ',df[df['着順']==1]['単勝オッズ'].median())
            print('標準偏差',odds_std)
            print('レース数',race_num)
            print('的中率',hit_rate)
            print('単勝適正回収値',s_odds)
    except ZeroDivisionError as e:
        continue
fig = plt.figure(facecolor="white")
ax = fig.add_subplot(111, xlabel="second_score", ylabel='recovery_rate',title=score_name)
ax.grid(color = "gray", linestyle="--")
ax.plot(x,y, label="recovery_rate",alpha=0.9)
ax.plot(x,z, label="race_len",alpha=0.9)
ax.plot(x,a, label="hit_rate",alpha=0.9)
ax.legend() 

In [ ]:
top3 = making_top3(first_result)
top3 = return_expmodel(top3,return_money)

taisyo = top3.copy()
date1 = datetime(2022, 12, 3)
date2 = date1 + relativedelta(days=3)  #年をまたいだ1ヶ月加算
DAY = taisyo[pd.to_datetime(taisyo['date'], format="%Y年%m月%d日")>=datetime.strftime(date1, format="%Y%m%d")]
DAY = DAY[pd.to_datetime(DAY['date'], format="%Y年%m月%d日")<datetime.strftime(date2, format="%Y%m%d")]
DAY

### クラスによる的中率

In [ ]:
def class_return(top3):
    class_hikaku = pd.DataFrame()
    class_hikaku['taisyo_class'] = top3['le_class'].value_counts()
    class_hikaku['hit_class'] = top3[top3['着順']<=1]['le_class'].value_counts()
    class_hikaku['hitrate_class'] = round(class_hikaku['hit_class']/class_hikaku['taisyo_class']*100,2)
    class_hikaku = class_hikaku.sort_values(['hitrate_class'],ascending=False)    
    return class_hikaku
top3 = making_top3(first_result)
top3 = return_expmodel(top3)
with open(path_list[2]+'/クラス別的中率.txt', 'w', encoding='sjis') as f:
    # print(class_return(top3),file=f)
    print('レース目と当選数\n',top3[top3['着順']<=1]['R'].value_counts(),file=f)
    print('クラス別当選率\n',class_return(top3),file=f)
class_return(top3)

In [ ]:
def place_return(top3):
    place_hikaku = pd.DataFrame()
    place_hikaku['taisyo_place'] = top3['le_place'].value_counts()
    place_hikaku['hit_place'] = top3[top3['着順']<=1]['le_place'].value_counts()
    place_hikaku['hitrate_place'] = round(place_hikaku['hit_place']/place_hikaku['taisyo_place']*100,2)
    place_hikaku = place_hikaku.sort_values(['hitrate_place'],ascending=False)    
    return place_hikaku
top3 = making_top3(first_result)
top3 = return_expmodel(top3)
place_return(top3)

In [ ]:
def r_return(top3):
    r_hikaku = pd.DataFrame()
    r_hikaku['hazure_R'] = top3['R'].value_counts()
    r_hikaku['hit_R'] = top3[top3['着順']<=1]['R'].value_counts()
    r_hikaku['hitrate_R'] = round(r_hikaku['hit_R']/r_hikaku['hazure_R']*100,2)
    r_hikaku = r_hikaku.sort_values(['hitrate_R' ],ascending=False)
    return r_hikaku
r_return(top3)

### 月間回収率

In [ ]:
top3 = making_top3(first_result)
top3 = return_expmodel(top3)
kakekin = 10000
summoney = 0
a,b,c,d,e,f,g = [],[],[],[],[],[],[]
for mon in range(1,datetime.now().month+1):
    date1 = datetime(2022, mon, 1)
    date2 = date1 + relativedelta(months=1)  #年をまたいだ1ヶ月加算
    DAY = top3[pd.to_datetime(top3['date'], format="%Y年%m月%d日")>=datetime.strftime(date1, format="%Y%m%d")]
    DAY = DAY[pd.to_datetime(DAY['date'], format="%Y年%m月%d日")<datetime.strftime(date2, format="%Y%m%d")]

    syuppi = kakekin*len(DAY)
    race_num = len(DAY)
    profit = round(kakekin*DAY[DAY['着順']==1]['単勝オッズ'].sum()-syuppi)
    a.append(mon)
    b.append(syuppi)
    c.append(round(kakekin*DAY[DAY['着順']==1]['単勝オッズ'].sum()))
    d.append(profit)
    e.append(round(len(DAY[DAY['着順']==1])/race_num*100,1))
    f.append(round(DAY[DAY['着順']==1]['単勝オッズ'].sum()/len(DAY),2))
    g.append(race_num)
    summoney = summoney+profit
monthdf = pd.DataFrame(
        data={'月': a, '出費': b, '当金': c, '利益': d, '当率': e, '回収率': f, '対象レース': g},
        columns=['月', '出費', '当金', '利益', '当率', '回収率', '対象レース'])
with open(path_list[2]+'/一ケ月当たりの回収率.txt', 'w', encoding='sjis') as f:
    print('１レース当たりの掛け金',kakekin,file=f)
    print('合計損益',summoney,file=f)
    print('回収率',round(monthdf['当金'].sum()/monthdf['出費'].sum(),2))
    print('回収率',monthdf,file=f)
print('１レース当たりの掛け金',kakekin)
print('合計損益',summoney)
print('回収率',round(monthdf['当金'].sum()/monthdf['出費'].sum(),2))
monthdf

### 週間回収率

In [ ]:
today = datetime.now()-datetime(2022,1,1)
top3 = making_top3(first_result)
top3 = return_expmodel(top3)
kakekin = 10000
date1 = datetime(2022, 1, 1)
sumprofit,sumprofit_rate,sum_syuppi,sum_hitmoney,divnum=0,0,0,0,0
progress_date,progress_rate,progress_profit=[],[],[]
a,b,c,d,e,f,g,h = [],[],[],[],[],[],[],[]
hit_df = pd.DataFrame()
bet_df = pd.DataFrame()
for week in range(1,today.days,7):
    date2 = date1 + relativedelta(weeks=1)  #年をまたいだ1ヶ月加算
    DAY = top3[pd.to_datetime(top3['date'], format="%Y年%m月%d日")>=datetime.strftime(date1, format="%Y%m%d")]
    DAY = DAY[pd.to_datetime(DAY['date'], format="%Y年%m月%d日")<datetime.strftime(date2, format="%Y%m%d")]

    syuppi = kakekin*len(DAY)
    race_num = len(DAY)
    profit = round(kakekin*DAY[DAY['着順']==1]['単勝オッズ'].sum()-syuppi)
    profit_rate = round(DAY[DAY['着順']==1]['単勝オッズ'].sum()/len(DAY),2)
    hit_money = round(kakekin*DAY[DAY['着順']==1]['単勝オッズ'].sum())
    hit_rate = round(len(DAY[DAY['着順']==1])/race_num*100,1)
    hit_num = len(DAY[DAY['着順']==1])
    hit_df = pd.concat([hit_df,DAY[DAY['着順']==1]])
    bet_df = pd.concat([bet_df,DAY[DAY['着順']!=1]])
    a.append(str(date1.date())[5:]+' ~ '+str((date1 + relativedelta(days=1)).date())[8:])
    b.append(syuppi)
    c.append(hit_money)
    d.append(profit)
    e.append(hit_rate)
    f.append(profit_rate)
    g.append(race_num)
    h.append(hit_num)
    
    date1 = date2
    sumprofit = sumprofit+profit
    sum_syuppi = sum_syuppi+syuppi
    sum_hitmoney = sum_hitmoney+hit_money
    progress_date.append(date2)
    progress_rate.append(sum_hitmoney/sum_syuppi)
    progress_profit.append(sumprofit)
weekdf = pd.DataFrame(
        data={'週': a, '出費': b, '当金': c, '利益': d, '当率': e, '回収率': f, '対象レース': g, '的中レース': h},
        columns=['週', '出費', '当金', '利益', '当率', '回収率', '対象レース','的中レース'])
with open(path_list[2]+'/一週間当たりの回収率.txt', 'w', encoding='sjis') as f:
    print('１レース当たりの掛け金',kakekin,file=f)
    print('合計損益',weekdf['利益'].sum(),file=f)
    print('合計損益',weekdf,file=f)
print('１レース当たりの掛け金',kakekin)
print('合計損益',weekdf['利益'].sum())
weekdf

### グラフ

In [ ]:
#利益推移
fig = plt.figure(facecolor="white")
plt.plot(progress_date, progress_profit, marker="o")
fig.savefig(path_list[2]+"/利益推移グラフ.png", bbox_inches="tight")

In [ ]:
#回収率推移
fig = plt.figure(facecolor="white")
plt.plot(progress_date, progress_rate, marker="o")
fig.savefig(path_list[2]+"/回収率推移グラフ.png", bbox_inches="tight")

## returnモデルの回収率

In [ ]:
def making_top3(first_result,flg=True):
    top3 = first_result[first_result['return_rank'] <= 3].sort_values(by = ['race_id','return_rank'], ascending=False)
    top3['期待値'] = top3['score_return']*top3['単勝オッズ']
    top3 = top3.sort_values('期待値', ascending=True)
    dropcol =['course_len', 'race_type', 'race_round', 'weather','ground_state']
    # dropcol = ['score_rank','return_rank','複勝オッズ_min','複勝オッズ_max']
    if flg:
        top3.drop(columns=dropcol,inplace=True)
    return top3
def make_retrunmodel(first_result,minexp,maxexp):
    top3 = making_top3(first_result,False)
    top3=top3[(top3['期待値']>minexp) & (top3['期待値']<maxexp)]
    top3['賭け金']=1/top3['単勝オッズ']
    top3['minオッズ']=minexp/top3['score_return']
    top3['maxオッズ']=maxexp/top3['score_return']
    return top3

### クラスによる的中率

In [ ]:
hikaku = class_return(top3)
with open(path_list[0]+'/当選傾向.txt', 'w', encoding='sjis') as f:
    print('レース目と当選数\n',top3[top3['着順']<=1]['R'].value_counts(),file=f)
    print('クラス別当選率\n',hikaku,file=f)
hikaku

In [ ]:
minexp,maxexp = 12.5,14.5
# minexp,maxexp = 11.4,13.4
top3 = make_retrunmodel(first_result,minexp,maxexp)
kakekin = 10000
summoney = 0
a,b,c,d,e,f,g = [],[],[],[],[],[],[]
for mon in range(1,datetime.now().month+1):
    date1 = datetime(2022, mon, 1)
    date2 = date1 + relativedelta(months=1)  #年をまたいだ1ヶ月加算
    DAY = top3[pd.to_datetime(top3['date'], format="%Y年%m月%d日")>=datetime.strftime(date1, format="%Y%m%d")]
    DAY = DAY[pd.to_datetime(DAY['date'], format="%Y年%m月%d日")<datetime.strftime(date2, format="%Y%m%d")]

    syuppi = kakekin*len(DAY)
    race_num = len(DAY)
    profit = round(kakekin*DAY[DAY['着順']==1]['単勝オッズ'].sum()-syuppi)
    a.append(mon)
    b.append(syuppi)
    c.append(round(kakekin*DAY[DAY['着順']==1]['単勝オッズ'].sum()))
    d.append(profit)
    e.append(round(len(DAY[DAY['着順']==1])/race_num*100,1))
    f.append(round(DAY[DAY['着順']==1]['単勝オッズ'].sum()/len(DAY),2))
    g.append(race_num)
    summoney = summoney+profit
monthdf = pd.DataFrame(
        data={'月': a, '出費': b, '当金': c, '利益': d, '当率': e, '回収率': f, '対象レース': g},
        columns=['月', '出費', '当金', '利益', '当率', '回収率', '対象レース'])
with open(path_list[0]+'/一ケ月当たりの回収率.txt', 'w', encoding='sjis') as f:
    print('１レース当たりの掛け金',kakekin,file=f)
    print('合計損益',summoney,file=f)
    print('回収率',round(monthdf['当金'].sum()/monthdf['出費'].sum(),2),file=f)
    print('期待値設定',minexp,'~',maxexp,file=f)
    print('回収率',monthdf,file=f)
print('１レース当たりの掛け金',kakekin)
print('合計損益',summoney)
print('回収率',round(monthdf['当金'].sum()/monthdf['出費'].sum(),2))
monthdf

In [ ]:
#利益推移
fig = plt.figure(facecolor="white")
plt.plot(progress_date, progress_profit, marker="o")
fig.savefig(path_list[0]+"/利益推移グラフ.png", bbox_inches="tight")

In [ ]:
#回収率推移
fig = plt.figure(facecolor="white")
plt.plot(progress_date, progress_rate, marker="o")
fig.savefig(path_list[0]+"/回収率推移グラフ.png", bbox_inches="tight")

### 期待値のスイートスポット

In [ ]:
# 期待値ごとの回収値変化
top3 = making_top3(first_result)
x = np.arange(1, 30, 0.1) #1から20までを0.1きざみ
y,z,a=[],[],[]
hani=2
tansyo_pro,old_tansyo = 0,0
with open(path_list[0]+'/期待値ごとの回収値変化.txt', 'w', encoding='sjis') as f:
    for num in x:
        try:
            plotdf = top3.sort_values('期待値', ascending=True).copy()
            plotdf=plotdf[(plotdf['期待値']>num) & (plotdf['期待値']<num+hani)]
            kaisyu = plotdf[plotdf['着順']<=1]['単勝オッズ'].sum()/len(plotdf)
            tekityu = len(plotdf[plotdf['着順']<=1])/len(plotdf)
            race_num = len(plotdf)
            plotdf['賭け金']=1/plotdf['単勝オッズ']*10000
            tansyo_pro = len(plotdf[plotdf['着順']==1])*10000/plotdf['賭け金'].sum()
            if kaisyu>1.2 and tekityu>0.1 and race_num>150:
                print('回収率',round(kaisyu,2),'期待値',round(num,1),'~',round(num+hani,1),'対象レース',race_num,'的中率',round(tekityu*100,1),'単勝適正回収値',round(tansyo_pro,2),file=f)
                print('回収率',round(kaisyu,2),'期待値',round(num,1),'~',round(num+hani,1),'対象レース',race_num,'的中率',round(tekityu*100,1),'単勝適正回収値',round(tansyo_pro,2))
                if tansyo_pro>old_tansyo:
                    old_tansyo = tansyo_pro#更新
                    print_text = '回収率',round(kaisyu,2),'期待値',round(num,1),'~',round(num+hani,1),'対象レース',race_num,'的中率',round(tekityu*100,1),'単勝適正回収値',round(tansyo_pro,2)
            y.append(kaisyu)
            z.append(tansyo_pro)
            a.append(tekityu)
        except ZeroDivisionError:
            # print('skip_num',num)
            y.append(0)
            z.append(0)
            a.append(0)
            continue
# print(print_text)
fig = plt.figure(facecolor="white")
plt.grid(color = "gray", linestyle="--")
plt.plot(x,y)
plt.plot(x,z)
plt.plot(x,a)
fig.savefig(path_list[0]+"/期待値ごとの回収値変化.png", bbox_inches="tight")

### 月別回収率

In [ ]:
top3 = making_top3(first_result)
for month in range(1,datetime.now().month+1):
    date1 = datetime(2022, month, 1)# 期待値ごとの回収値変化
    date2 = datetime(2022, month, calendar.monthrange(2022, month)[1])
    x = np.arange(1, 20, 0.5) #1から20までを0.1きざみ
    y,z,a=[],[],[]
    hani=2
    tansyo_pro,old_tansyo = 0,0
    # with open(path_list[0]+'/月別回収値変化{}.txt'.format(month), 'w', encoding='sjis') as f:
    for num in x:
        try:
            plotdf = top3[pd.to_datetime(top3['date'], format="%Y年%m月%d日")>=datetime.strftime(date1, format="%Y%m%d")]
            plotdf = plotdf[pd.to_datetime(plotdf['date'], format="%Y年%m月%d日")<=datetime.strftime(date2, format="%Y%m%d")]
            place = plotdf['le_place'].unique()
            plotdf = plotdf.sort_values('期待値', ascending=True)
            plotdf=plotdf[(plotdf['期待値']>num) & (plotdf['期待値']<num+hani)]
            kaisyu = plotdf[plotdf['着順']<=1]['単勝オッズ'].sum()/len(plotdf)
            tekityu = len(plotdf[plotdf['着順']<=1])/len(plotdf)
            race_num = len(plotdf)
            plotdf['賭け金']=1/plotdf['単勝オッズ']*10000
            tansyo_pro = len(plotdf[plotdf['着順']==1])*10000/plotdf['賭け金'].sum()
            # if kaisyu>1.2:
                # print('回収率',round(kaisyu,2),'期待値',round(num,1),'~',round(num+hani,1),'対象レース',race_num,'的中率',round(tekityu*100,1),'単勝適正回収値',round(tansyo_pro,2),file=f)
                # print('回収率',round(kaisyu,2),'期待値',round(num,1),'~',round(num+hani,1),'対象レース',race_num,'的中率',round(tekityu*100,1),'単勝適正回収値',round(tansyo_pro,2))
            if tansyo_pro>old_tansyo:
                old_tansyo = tansyo_pro#更新
                print_text = '回収率',round(kaisyu,2),'期待値',round(num,1),'~',round(num+hani,1),'対象レース',race_num,'的中率',round(tekityu*100,1),'単勝適正回収値',round(tansyo_pro,2)
            y.append(kaisyu)
            z.append(tansyo_pro)
            a.append(tekityu)
        except ZeroDivisionError:
            # print('skip_num',num)
            y.append(0)
            z.append(0)
            a.append(0)
            continue
    print('month',date1,'~',date2,'\n',place,'\n',print_text)
    fig = plt.figure(facecolor="white")
    plt.grid(color = "gray", linestyle="--")
    plt.title(str(month)+str(place)+'\n'+str(print_text))
    plt.plot(x,y)
    plt.plot(x,z)
    plt.plot(x,a)
    fig.savefig(path_list[0]+"/月別の回収値変化{}月.png".format(month), bbox_inches="tight")

### 開催地別回収率

In [ ]:
top3 = making_top3(first_result)
top3 = return_expmodel(top3,return_money)
seting_exp = top3[['le_place','min_exp','max_exp']]
seting_exp[~seting_exp.duplicated()]

In [ ]:
# 期待値ごとの回収値変化(最低対象レース数30の場合)
top3 = making_top3(first_result)
place = top3['le_place'].unique()
for p in place:
    print(p)
    x = np.arange(1, 15, 0.1) #1から20までを0.1きざみ
    y,z,a=[],[],[]
    hani=3
    tansyo_pro,old_tansyo = 0,0
    for num in x:
        try:
            plotdf = top3.sort_values('期待値', ascending=True)
            plotdf = top3[top3['le_place']==p]
            all_race_num = plotdf['race_id'].nunique()
            plotdf=plotdf[(plotdf['期待値']>num) & (plotdf['期待値']<num+hani)]
            kaisyu = plotdf[plotdf['着順']<=1]['単勝オッズ'].sum()/len(plotdf)
            tekityu = len(plotdf[plotdf['着順']<=1])/len(plotdf)
            race_num = len(plotdf)
            plotdf['賭け金']=1/plotdf['単勝オッズ']*10000
            tansyo_pro = len(plotdf[plotdf['着順']==1])*10000/plotdf['賭け金'].sum()
            # if kaisyu>1.2 and tekityu>0.1 and race_num>150:
            # print('回収率',round(kaisyu,2),'期待値',round(num,1),'~',round(num+hani,1),'対象レース',race_num,'的中率',round(tekityu*100,1),'単勝適正回収値',round(tansyo_pro,2),'レース数割合',round(race_num/all_race_num,2))
            if tansyo_pro>old_tansyo and race_num>30:
                old_tansyo = tansyo_pro#更新
                print_text = '回収率',round(kaisyu,2),'期待値',round(num,1),'~',round(num+hani,1),'対象レース',race_num,'的中率',round(tekityu*100,1),'単勝適正回収値',round(tansyo_pro,2),'レース数割合',round(race_num/all_race_num,2)
            y.append(kaisyu)
            z.append(tansyo_pro)
            a.append(tekityu)
        except ZeroDivisionError:
            # print('skip_num',num)
            y.append(0)
            z.append(0)
            a.append(0)
            continue
    print(print_text)
    fig = plt.figure(facecolor="white")
    plt.grid(color = "gray", linestyle="--")
    plt.title(str(p)+'\n'+str(print_text))
    plt.plot(x,y)
    plt.plot(x,z)
    plt.plot(x,a)
    fig.savefig(path_list[0]+"/開催地別の回収値変化_{}.png".format(p), bbox_inches="tight")

### 開催地別の詳細期待値分布

In [ ]:
# 期待値ごとの回収値変化
p='福島'
print(p)
x = np.arange(1, 15, 0.1) #1から20までを0.1きざみ
y,z,a=[],[],[]
hani,tansyo_pro,old_tansyo = 3,0,0
for num in x:
    try:
        plotdf = top3.sort_values('期待値', ascending=True)
        plotdf = top3[top3['le_place']==p]
        all_race_num = plotdf['race_id'].nunique()
        plotdf=plotdf[(plotdf['期待値']>num) & (plotdf['期待値']<num+hani)]
        kaisyu = plotdf[plotdf['着順']<=1]['単勝オッズ'].sum()/len(plotdf)
        tekityu = len(plotdf[plotdf['着順']<=1])/len(plotdf)
        race_num = len(plotdf)
        plotdf['賭け金']=1/plotdf['単勝オッズ']*10000
        tansyo_pro = len(plotdf[plotdf['着順']==1])*10000/plotdf['賭け金'].sum()
        if kaisyu>1.2 and tekityu>0.1 and race_num>15:
            print('回収率',round(kaisyu,2),'期待値',round(num,1),'~',round(num+hani,1),'対象レース',race_num,'的中率',round(tekityu*100,1),'単勝適正回収値',round(tansyo_pro,2),'レース数割合',round(race_num/all_race_num,2))
        if tansyo_pro>old_tansyo:
            old_tansyo = tansyo_pro#更新
            print_text = '回収率',round(kaisyu,2),'期待値',round(num,1),'~',round(num+hani,1),'対象レース',race_num,'的中率',round(tekityu*100,1),'単勝適正回収値',round(tansyo_pro,2),'レース数割合',round(race_num/all_race_num,2)
        y.append(kaisyu)
        z.append(tansyo_pro)
        a.append(tekityu)
    except ZeroDivisionError:
        # print('skip_num',num)
        y.append(0)
        z.append(0)
        a.append(0)
        continue
print(print_text)
fig = plt.figure(facecolor="white")
plt.grid(color = "gray", linestyle="--")
plt.plot(x,y)
plt.plot(x,z)
plt.plot(x,a)

## top4モデルでの回収率

In [ ]:
#topモデルで使用する期待値表を作成する
def maiking_top2(first_result,flg=True):
    top2 = first_result[first_result['score_rank'] <= 2].sort_values(by = ['race_id','score_top4'], ascending=False)
    top2['期待値'] = ((top2['score_top4']))*top2['単勝オッズ']
    top2 = top2.sort_values('期待値', ascending=True)
    dropcol =['course_len', 'race_type', 'race_round', 'weather','ground_state']
    # dropcol = ['score_rank','return_rank','複勝オッズ_min','複勝オッズ_max']
    if flg:
        top2.drop(columns=dropcol,inplace=True)
    return top2

In [ ]:
def make_scoremodel(first_result,minexp,maxexp):
    top2 = maiking_top2(first_result,False)
    top2=top2[(top2['期待値']>minodds) & (top2['期待値']<maxodds)]
    top2['賭け金']=1/top2['単勝オッズ']
    top2['minオッズ']=minodds/top2['score_return']
    top2['maxオッズ']=maxodds/top2['score_return']
    return top2

In [ ]:
#順張りの場合
minodds = 8.1
maxodds = 10.1
top2 = make_scoremodel(first_result,minodds,maxodds)
print('回収率',top2[top2['着順']<=1]['単勝オッズ'].sum()/len(top2))
print('対象レース',len(top2))
print('的中レース',len(top2[top2['着順']<=1]))
print('全レース',first_result['race_id'].nunique())
print('対象レース割合',len(top2)/first_result['race_id'].nunique())
print('的中率',(len(top2[top2['着順']<=1])/len(top2)))
print('単勝適正回収値',len(top2[top2['着順']==1])/top2['賭け金'].sum())
# ファイルへの書き込み
with open(path_list[1]+'/overview.txt', 'a', encoding='sjis') as f:
    print('回収率',top2[top2['着順']<=1]['単勝オッズ'].sum()/len(top2),file=f)
    print('対象レース',len(top2),file=f)
    print('的中レース',len(top2[top2['着順']<=1]),file=f)
    print('全レース',first_result['race_id'].nunique(),file=f)
    print('対象レース割合',len(top2)/first_result['race_id'].nunique(),file=f)
    print('的中率',(len(top2[top2['着順']<=1])/len(top2)),file=f)
    print('単勝適正回収値',len(top2[top2['着順']==1])*10000/top2['賭け金'].sum(),file=f)
    print('設定オッズ',minodds,'~',maxodds,file=f)

In [ ]:
hikaku = class_return(top2)
with open(path_list[1]+'当選傾向.txt', 'w', encoding='sjis') as f:
    print('レース目と当選数\n',top2[top2['着順']<=1]['R'].value_counts(),file=f)
    print('クラス別当選率\n',hikaku,file=f)
hikaku

In [ ]:
# 期待値ごとの回収値変化
top2=maiking_top2(first_result)
x = np.arange(1, 20, 0.1) #1から20までを0.1きざみ
y,z,a=[],[],[]
hani,tansyo_pro,old_tansyo = 2,0,0
for num in x:
    try:
        plotdf = top2.sort_values('期待値', ascending=True)
        plotdf=plotdf[(plotdf['期待値']>num) & (plotdf['期待値']<num+hani)]
        kaisyu = plotdf[plotdf['着順']<=1]['単勝オッズ'].sum()/len(plotdf)
        tekityu = len(plotdf[plotdf['着順']<=1])/len(plotdf)
        race_num = len(plotdf)
        plotdf['賭け金']=1/plotdf['単勝オッズ']*10000
        tansyo_pro = len(plotdf[plotdf['着順']==1])*10000/plotdf['賭け金'].sum()
        if kaisyu>1 and tekityu>0.1 and race_num>100:
            print('回収率',round(kaisyu,2),'期待値',round(num,1),'~',round(num+hani,1),'対象レース',race_num,'的中率',round(tekityu*100,1),'単勝適正回収値',round(tansyo_pro,2))
            if tansyo_pro>old_tansyo:
                old_tansyo = tansyo_pro#更新
                print_text = '回収率',round(kaisyu,2),'期待値',round(num,1),'~',round(num+hani,1),'対象レース',race_num,'的中率',round(tekityu*100,1),'単勝適正回収値',round(tansyo_pro,2)
        y.append(kaisyu)
        z.append(tansyo_pro)
        a.append(tekityu)
    except ZeroDivisionError:
        # print('skip_num',num)
        y.append(0)
        z.append(0)
        a.append(0)
        continue
print(print_text)
fig = plt.figure(facecolor="white")
plt.grid(color = "gray", linestyle="--")
plt.plot(x,y)
plt.plot(x,z)
plt.plot(x,a)
fig.savefig(path_list[1]+"/期待値ごとの回収値変化.png", bbox_inches="tight")

### 開催地別期待値グラフ

In [ ]:
place = top2['le_place'].unique()
for p in place:
    print(p)
    x = np.arange(1, 15, 0.1) #1から20までを0.1きざみ
    y,z,a=[],[],[]
    hani,tansyo_pro,old_tansyo = 2,0,0
    for num in x:
        try:
            plotdf = top2.sort_values('期待値', ascending=True)
            plotdf = top2[top2['le_place']==p]
            all_race_num = len(plotdf['race_id'].unique())
            plotdf=plotdf[(plotdf['期待値']>num) & (plotdf['期待値']<num+hani)]
            kaisyu = plotdf[plotdf['着順']<=1]['単勝オッズ'].sum()/len(plotdf)
            tekityu = len(plotdf[plotdf['着順']<=1])/len(plotdf)
            race_num = len(plotdf)
            plotdf['賭け金']=1/plotdf['単勝オッズ']*10000
            tansyo_pro = len(plotdf[plotdf['着順']==1])*10000/plotdf['賭け金'].sum()
            # if kaisyu>1.2 and tekityu>0.1 and race_num>150:
            # print('回収率',round(kaisyu,2),'期待値',round(num,1),'~',round(num+hani,1),'対象レース',race_num,'的中率',round(tekityu*100,1),'単勝適正回収値',round(tansyo_pro,2),'レース数割合',round(race_num/all_race_num,2))
            if tansyo_pro>old_tansyo:
                old_tansyo = tansyo_pro#更新
                print_text = '回収率',round(kaisyu,2),'期待値',round(num,1),'~',round(num+hani,1),'対象レース',race_num,'的中率',round(tekityu*100,1),'単勝適正回収値',round(tansyo_pro,2),'レース数割合',round(race_num/all_race_num,2)
            y.append(kaisyu)
            z.append(tansyo_pro)
            a.append(tekityu)
        except ZeroDivisionError:
            # print('skip_num',num)
            y.append(0)
            z.append(0)
            a.append(0)
            continue
    print(print_text)
    fig = plt.figure(facecolor="white")
    plt.grid(color = "gray", linestyle="--")
    plt.plot(x,y)
    plt.plot(x,z)
    plt.plot(x,a)
    fig.savefig(path_list[1]+"/開催地別の回収値変化_{}.png".format(p), bbox_inches="tight")

In [ ]:
top2=maiking_top2(first_result)
for month in range(1,datetime.now().month+1):
    date = datetime(2022, month, 1)# 期待値ごとの回収値変化
    x = np.arange(1, 12, 0.5) #1から20までを0.1きざみ
    y,z,a=[],[],[]
    hani,tansyo_pro,old_tansyo = 2,0,0
    # with open(dir_path1+'/期待値ごとの回収値変化.txt', 'w', encoding='sjis') as f:
    for num in x:
        try:
            plotdf = top2[pd.to_datetime(top2['date'], format="%Y年%m月%d日")>=datetime.strftime(date, format="%Y%m%d")]
            # plotdf = top2[top2['le_place']=='札幌']
            # print((plotdf))
            plotdf = plotdf.sort_values('期待値', ascending=True)
            plotdf=plotdf[(plotdf['期待値']>num) & (plotdf['期待値']<num+hani)]
            kaisyu = plotdf[plotdf['着順']<=1]['単勝オッズ'].sum()/len(plotdf)
            tekityu = len(plotdf[plotdf['着順']<=1])/len(plotdf)
            race_num = len(plotdf)
            plotdf['賭け金']=1/plotdf['単勝オッズ']*10000
            tansyo_pro = len(plotdf[plotdf['着順']==1])*10000/plotdf['賭け金'].sum()
            # if kaisyu>1.2:
                # print('回収率',round(kaisyu,2),'期待値',round(num,1),'~',round(num+hani,1),'対象レース',race_num,'的中率',round(tekityu*100,1),'単勝適正回収値',round(tansyo_pro,2),file=f)
                # print('回収率',round(kaisyu,2),'期待値',round(num,1),'~',round(num+hani,1),'対象レース',race_num,'的中率',round(tekityu*100,1),'単勝適正回収値',round(tansyo_pro,2))
            if tansyo_pro>old_tansyo:
                old_tansyo = tansyo_pro#更新
                print_text = '回収率',round(kaisyu,2),'期待値',round(num,1),'~',round(num+hani,1),'対象レース',race_num,'的中率',round(tekityu*100,1),'単勝適正回収値',round(tansyo_pro,2)
            y.append(kaisyu)
            z.append(tansyo_pro)
            a.append(tekityu)
        except ZeroDivisionError:
            # print('skip_num',num)
            y.append(0)
            z.append(0)
            a.append(0)
            continue
    print('month',month,'\n',print_text)
    fig = plt.figure(facecolor="white")
    plt.grid(color = "gray", linestyle="--")
    plt.plot(x,y)
    plt.plot(x,z)
    plt.plot(x,a)
    fig.savefig(path_list[1]+"/月別の回収値変化{}月.png".format(month), bbox_inches="tight")

### 月別回収率

In [ ]:
top2 = make_scoremodel(top2,minodds,maxodds)
kakekin = 10000
summoney = 0
a,b,c,d,e,f,g = [],[],[],[],[],[],[]
for mon in range(1,datetime.now().month+1):
    date1 = datetime(2022, mon, 1)
    date2 = date1 + relativedelta(months=1)  #年をまたいだ1ヶ月加算
    DAY = top2[pd.to_datetime(top2['date'], format="%Y年%m月%d日")>=datetime.strftime(date1, format="%Y%m%d")]
    DAY = DAY[pd.to_datetime(DAY['date'], format="%Y年%m月%d日")<datetime.strftime(date2, format="%Y%m%d")]

    syuppi = kakekin*len(DAY)
    race_num = len(DAY)
    profit = round(kakekin*DAY[DAY['着順']==1]['単勝オッズ'].sum()-syuppi)
    a.append(mon)
    b.append(syuppi)
    c.append(round(kakekin*DAY[DAY['着順']==1]['単勝オッズ'].sum()))
    d.append(profit)
    e.append(round(len(DAY[DAY['着順']==1])/race_num*100,1))
    f.append(round(DAY[DAY['着順']==1]['単勝オッズ'].sum()/len(DAY),2))
    g.append(race_num)
    summoney = summoney+profit
monthdf = pd.DataFrame(
        data={'月': a, '出費': b, '当金': c, '利益': d, '当率': e, '回収率': f, '対象レース': g},
        columns=['月', '出費', '当金', '利益', '当率', '回収率', '対象レース'])
with open(path_list[1]+'/一ケ月当たりの回収率.txt', 'w', encoding='sjis') as f:
    print('１レース当たりの掛け金',kakekin,file=f)
    print('合計損益',summoney,file=f)
    print('回収率',round(monthdf['回収率'].mean(),2),file=f)
    print('回収率',monthdf,file=f)
print('１レース当たりの掛け金',kakekin,'\n合計損益',summoney,'\n回収率',round(monthdf['当金'].sum()/monthdf['出費'].sum(),2))
monthdf

### 週別回収率

In [ ]:
top2 = make_scoremodel(top2,minodds,maxodds)
today = datetime.now()-datetime(2022,1,1)
kakekin = 10000
date1 = datetime(2022, 1, 1)
sumprofit,sumprofit_rate,sum_syuppi,sum_hitmoney,divnum=0,0,0,0,0
progress_date,progress_rate,progress_profit=[],[],[]
a,b,c,d,e,f,g = [],[],[],[],[],[],[]
for week in range(1,today.days,7):
    try:
    # print(week,'週------')
    # date1 = datetime(2022, 1, 1)
        date2 = date1 + relativedelta(weeks=1)  #年をまたいだ1ヶ月加算
        # print(date1,date2)
        DAY = top2[pd.to_datetime(top2['date'], format="%Y年%m月%d日")>=datetime.strftime(date1, format="%Y%m%d")]
        DAY = DAY[pd.to_datetime(DAY['date'], format="%Y年%m月%d日")<datetime.strftime(date2, format="%Y%m%d")]

        syuppi = kakekin*len(DAY)
        race_num = len(DAY)
        # print(date2,race_num)
        profit = round(kakekin*DAY[DAY['着順']==1]['単勝オッズ'].sum()-syuppi)
        profit_rate = round(DAY[DAY['着順']==1]['単勝オッズ'].sum()/len(DAY),2)
        hit_money = round(kakekin*DAY[DAY['着順']==1]['単勝オッズ'].sum())
        hit_rate = round(len(DAY[DAY['着順']==1])/race_num*100,1)
        # print('出費',syuppi)
        # print('当金',hit_money)
        # print('利益',profit)
        # print('当率',hit_rate)
        # print('回収率',profit_rate)
        # print('対象レース',race_num)
        a.append(str(date1.date())[5:]+' ~ '+str(date2.date())[5:])
        b.append(syuppi)
        c.append(hit_money)
        d.append(profit)
        e.append(hit_rate)
        f.append(profit_rate)
        g.append(race_num)

        date1 = date2
        sumprofit = sumprofit+profit
        sum_syuppi = sum_syuppi+syuppi
        sum_hitmoney = sum_hitmoney+hit_money
        progress_date.append(date2)
        progress_rate.append(sum_hitmoney/sum_syuppi)
        progress_profit.append(sumprofit)
    except:
        continue
weekdf = pd.DataFrame(
        data={'週': a, '出費': b, '当金': c, '利益': d, '当率': e, '回収率': f, '対象レース': g},
        columns=['週', '出費', '当金', '利益', '当率', '回収率', '対象レース'])
with open(path_list[1]+'/一週間当たりの回収率.txt', 'w', encoding='sjis') as f:
    print('１レース当たりの掛け金',kakekin,file=f)
    print('合計損益',weekdf['利益'].sum(),file=f)
    print('合計損益',weekdf,file=f)
print('１レース当たりの掛け金',kakekin,'\n合計損益',weekdf['利益'].sum())
weekdf

### 収益のグラフ化

In [ ]:
#利益推移
fig = plt.figure(facecolor="white")
plt.plot(progress_date, progress_profit, marker="o")
fig.savefig(path_list[1]+"/利益推移グラフ.png", bbox_inches="tight")

In [ ]:
#回収率推移
fig = plt.figure(facecolor="white")
plt.plot(progress_date, progress_rate, marker="o")
fig.savefig(path_list[1]+"/回収率推移グラフ.png", bbox_inches="tight")

## top4モデル期待値変動型での回収率

In [ ]:
#開催地別の期待値設定
def set_exptop(top2):
    #開催場別に期待値を設定する
    top2.loc[top2['le_place'] == '札幌', 'min_exp'] = 8.4
    top2.loc[top2['le_place'] == '札幌', 'max_exp'] = 10.4
    top2.loc[top2['le_place'] == '小倉', 'min_exp'] = 14.8
    top2.loc[top2['le_place'] == '小倉', 'max_exp'] = 16.8
    top2.loc[top2['le_place'] == '新潟', 'min_exp'] = 9.1
    top2.loc[top2['le_place'] == '新潟', 'max_exp'] = 11.1
    # top2.loc[top2['le_place'] == '新潟', 'min_exp'] = 14.3
    # top2.loc[top2['le_place'] == '新潟', 'max_exp'] = 17.3
    top2.loc[top2['le_place'] == '阪神', 'min_exp'] = 9.6
    top2.loc[top2['le_place'] == '阪神', 'max_exp'] = 11.6
    top2.loc[top2['le_place'] == '中京', 'min_exp'] = 7.5
    top2.loc[top2['le_place'] == '中京', 'max_exp'] = 9.5
    top2.loc[top2['le_place'] == '中山', 'min_exp'] = 8
    top2.loc[top2['le_place'] == '中山', 'max_exp'] = 10
    top2.loc[top2['le_place'] == '東京', 'min_exp'] = 8.2
    top2.loc[top2['le_place'] == '東京', 'max_exp'] = 10.2
    top2.loc[top2['le_place'] == '福島', 'min_exp'] = 7.6
    top2.loc[top2['le_place'] == '福島', 'max_exp'] = 9.6
    top2.loc[top2['le_place'] == '函館', 'min_exp'] = 12.6
    top2.loc[top2['le_place'] == '函館', 'max_exp'] = 14.6
    return top2
def top_expmodel(top2,return_money=10000):
    top2 = set_exptop(top2)
    top2=top2[(top2['期待値']>top2['min_exp']) & (top2['期待値']<top2['max_exp'])].copy()
    top2['minオッズ']=round(top2['min_exp']/top2['score_top4'],2)
    top2['maxオッズ']=round(top2['max_exp']/top2['score_top4'],2)
    top2['賭け金']=1/top2['単勝オッズ']*return_money
    top2['期待値'] = round(top2['期待値'],1)
    return top2

In [ ]:
return_money=10000
top2 = maiking_top2(first_result)
top2 = top_expmodel(top2,return_money)
print('回収率',top2[top2['着順']<=1]['単勝オッズ'].sum()/len(top2)),print('対象レース',len(top2)),print('的中レース',len(top2[top2['着順']<=1])),print('全レース',first_result['race_id'].nunique()),print('対象レース割合',top2['race_id'].nunique()/first_result['race_id'].nunique()),print('的中率',(len(top2[top2['着順']<=1])/len(top2))),print('単勝適正回収値',len(top2[top2['着順']==1])*return_money/top2['賭け金'].sum())
with open(path_list[3]+'/overview.txt', 'a', encoding='sjis') as f:
    print('回収率',top2[top2['着順']<=1]['単勝オッズ'].sum()/len(top2),file=f)
    print('対象レース',len(top2),file=f)
    print('的中レース',len(top2[top2['着順']<=1]),file=f)
    print('全レース',first_result['race_id'].nunique(),file=f)
    print('対象レース割合',len(top2)/first_result['race_id'].nunique(),file=f)
    print('的中率',(len(top2[top2['着順']<=1])/len(top2)),file=f)
    print('単勝適正回収値',len(top2[top2['着順']==1])*return_money/top2['賭け金'].sum(),file=f)
    seting_exp = top2[['le_place','min_exp','max_exp']]
    print('設定オッズ',seting_exp[~seting_exp.duplicated()],file=f)

In [ ]:
taisyo = top2.copy()
date1 = datetime(2022, 12, 3)
date2 = date1 + relativedelta(days=3)  #年をまたいだ1ヶ月加算
DAY = taisyo[pd.to_datetime(taisyo['date'], format="%Y年%m月%d日")>=datetime.strftime(date1, format="%Y%m%d")]
DAY = DAY[pd.to_datetime(DAY['date'], format="%Y年%m月%d日")<datetime.strftime(date2, format="%Y%m%d")]
DAY

In [ ]:
#順張りの場合
kakekin = 10000
summoney = 0
a,b,c,d,e,f,g = [],[],[],[],[],[],[]
for mon in range(1,datetime.now().month+1):
    date1 = datetime(2022, mon, 1)
    date2 = date1 + relativedelta(months=1)  #年をまたいだ1ヶ月加算
    DAY = top2[pd.to_datetime(top2['date'], format="%Y年%m月%d日")>=datetime.strftime(date1, format="%Y%m%d")]
    DAY = DAY[pd.to_datetime(DAY['date'], format="%Y年%m月%d日")<datetime.strftime(date2, format="%Y%m%d")]

    syuppi = kakekin*len(DAY)
    race_num = len(DAY)
    profit = round(kakekin*DAY[DAY['着順']==1]['単勝オッズ'].sum()-syuppi)
    a.append(mon)
    b.append(syuppi)
    c.append(round(kakekin*DAY[DAY['着順']==1]['単勝オッズ'].sum()))
    d.append(profit)
    e.append(round(len(DAY[DAY['着順']==1])/race_num*100,1))
    f.append(round(DAY[DAY['着順']==1]['単勝オッズ'].sum()/len(DAY),2))
    g.append(race_num)
    summoney = summoney+profit
monthdf = pd.DataFrame(
        data={'月': a, '出費': b, '当金': c, '利益': d, '当率': e, '回収率': f, '対象レース': g},
        columns=['月', '出費', '当金', '利益', '当率', '回収率', '対象レース'])
with open(path_list[3]+'/一ケ月当たりの回収率.txt', 'w', encoding='sjis') as f:
    print('１レース当たりの掛け金',kakekin,file=f)
    print('合計損益',summoney,file=f)
    print('回収率',round(monthdf['当金'].sum()/monthdf['出費'].sum(),2),file=f)
    print('回収率',monthdf,file=f)
print('１レース当たりの掛け金',kakekin)
print('合計損益',summoney)
print('回収率',round(monthdf['当金'].sum()/monthdf['出費'].sum(),2))
monthdf

In [ ]:
#週間回収率
kakekin = 10000
date1 = datetime(2022, 1, 1)
sumprofit,sumprofit_rate,sum_syuppi,sum_hitmoney,divnum=0,0,0,0,0
progress_date,progress_rate,progress_profit=[],[],[]
a,b,c,d,e,f,g,h = [],[],[],[],[],[],[],[]
hit_df = pd.DataFrame()
bet_df = pd.DataFrame()
for week in range(1,today.days,7):
    # try:
    # print(week,'週------')
    # date1 = datetime(2022, 1, 1)
    date2 = date1 + relativedelta(weeks=1)  #年をまたいだ1ヶ月加算
    # print(date1,date2)
    DAY = top2[pd.to_datetime(top2['date'], format="%Y年%m月%d日")>=datetime.strftime(date1, format="%Y%m%d")]
    DAY = DAY[pd.to_datetime(DAY['date'], format="%Y年%m月%d日")<datetime.strftime(date2, format="%Y%m%d")]

    syuppi = kakekin*len(DAY)
    race_num = len(DAY)
    profit = round(kakekin*DAY[DAY['着順']==1]['単勝オッズ'].sum()-syuppi)
    profit_rate = round(DAY[DAY['着順']==1]['単勝オッズ'].sum()/len(DAY),2)
    hit_money = round(kakekin*DAY[DAY['着順']==1]['単勝オッズ'].sum())
    hit_rate = round(len(DAY[DAY['着順']==1])/race_num*100,1)
    hit_num = len(DAY[DAY['着順']==1])
    hit_df = pd.concat([hit_df,DAY[DAY['着順']==1]])
    bet_df = pd.concat([bet_df,DAY[DAY['着順']!=1]])
    # print('出費',syuppi)
    # print('当金',hit_money)
    # print('利益',profit)
    # print('当率',hit_rate)
    # print('回収率',profit_rate)
    # print('対象レース',race_num)
    # a.append(str(date1.date())[5:]+' ~ '+str(date2.date())[5:])
    a.append(str(date1.date())[5:]+' ~ '+str((date1 + relativedelta(days=1)).date())[8:])
    b.append(syuppi)
    c.append(hit_money)
    d.append(profit)
    e.append(hit_rate)
    f.append(profit_rate)
    g.append(race_num)
    h.append(hit_num)
    
    date1 = date2
    sumprofit = sumprofit+profit
    sum_syuppi = sum_syuppi+syuppi
    sum_hitmoney = sum_hitmoney+hit_money
    progress_date.append(date2)
    progress_rate.append(sum_hitmoney/sum_syuppi)
    progress_profit.append(sumprofit)
weekdf = pd.DataFrame(
        data={'週': a, '出費': b, '当金': c, '利益': d, '当率': e, '回収率': f, '対象レース': g, '的中レース': h},
        columns=['週', '出費', '当金', '利益', '当率', '回収率', '対象レース','的中レース'])
with open(path_list[3]+'/一週間当たりの回収率.txt', 'w', encoding='sjis') as f:
    print('１レース当たりの掛け金',kakekin,file=f)
    print('合計損益',weekdf['利益'].sum(),file=f)
    print('合計損益',weekdf,file=f)
print('１レース当たりの掛け金',kakekin)
print('合計損益',weekdf['利益'].sum())
weekdf

In [ ]:
#利益推移
fig = plt.figure(facecolor="white")
plt.plot(progress_date, progress_profit, marker="o")
fig.savefig(path_list[3]+"/利益推移グラフ.png", bbox_inches="tight")

In [ ]:
#回収率推移
fig = plt.figure(facecolor="white")
plt.plot(progress_date, progress_rate, marker="o")
fig.savefig(path_list[3]+"/回収率推移グラフ.png", bbox_inches="tight")

## ハイブリット形式

In [ ]:
#top3の期待値範囲に該当せず20倍以上のオッズが付いている馬が1頭でもいる場合のレースで
#top2の期待値範囲8-11に該当する場合の回収値
#要約すると穴馬モデルが来ない可能性の高いレース(期待値範囲外かつ20倍以上一頭)で
top2 = maiking_top2(first_result)
top3 = making_top3(first_result)
top3 = return_expmodel(top3,return_money)
first_result['race_id']
df = first_result[~first_result['race_id'].isin(top3['race_id'])]
df2 = df[df['return_rank'] <= 3].sort_values(by = ['race_id','return_rank'], ascending=False)
df3 = df2[df2['単勝オッズ']>20].value_counts('race_id')==1
target_race_id = df3[df3].index
target = top2[top2['race_id'].isin(target_race_id)]

In [ ]:
kekka = target[(target['期待値']>8)&(target['期待値']<11)]
kekka = kekka[kekka['score_top4']>=1.2]
print('回収率',kekka[kekka['着順']<=1]['単勝オッズ'].sum()/len(kekka))
print('平均的中オッズ',kekka[kekka['着順']<=1]['単勝オッズ'].mean())
print('対象レース',len(kekka))
print('対象レース割合',kekka['race_id'].nunique()/first_result['race_id'].nunique())
print('的中レース',len(kekka[kekka['着順']<=1]))
print('的中率',(len(kekka[kekka['着順']<=1])/len(kekka)))
print('出現確率',kekka['date'].nunique()/first_result['date'].nunique())

In [ ]:
date1 = datetime(2022, 12, 3)
date2 = date1 + relativedelta(days=1000)  #年をまたいだ1ヶ月加算
DAY = kekka[pd.to_datetime(kekka['date'], format="%Y年%m月%d日")>=datetime.strftime(date1, format="%Y%m%d")]
DAY = DAY[pd.to_datetime(DAY['date'], format="%Y年%m月%d日")<datetime.strftime(date2, format="%Y%m%d")]
print('回収率',DAY[DAY['着順']<=1]['単勝オッズ'].sum()/len(DAY))
DAY.sort_values(['le_place','R','馬番'])

In [ ]:
#月間回収率
kakekin = 10000
summoney = 0
a,b,c,d,e,f,g = [],[],[],[],[],[],[]
for mon in range(1,datetime.now().month+1):
    date1 = datetime(2022, mon, 1)
    date2 = date1 + relativedelta(months=1)  #年をまたいだ1ヶ月加算
    DAY = kekka[pd.to_datetime(kekka['date'], format="%Y年%m月%d日")>=datetime.strftime(date1, format="%Y%m%d")]
    DAY = DAY[pd.to_datetime(DAY['date'], format="%Y年%m月%d日")<datetime.strftime(date2, format="%Y%m%d")]

    syuppi = kakekin*len(DAY)
    race_num = len(DAY)
    if race_num == 0:
        race_num=1
    profit = round(kakekin*DAY[DAY['着順']==1]['単勝オッズ'].sum()-syuppi)
    a.append(mon)
    b.append(syuppi)
    c.append(round(kakekin*DAY[DAY['着順']==1]['単勝オッズ'].sum()))
    d.append(profit)
    e.append(round(len(DAY[DAY['着順']==1])/race_num*100,1))
    f.append(round(DAY[DAY['着順']==1]['単勝オッズ'].sum()/len(DAY),2))
    g.append(race_num)
    summoney = summoney+profit
monthdf = pd.DataFrame(
        data={'月': a, '出費': b, '当金': c, '利益': d, '当率': e, '回収率': f, '対象レース': g},
        columns=['月', '出費', '当金', '利益', '当率', '回収率', '対象レース'])
print('１レース当たりの掛け金',kakekin)
print('合計損益',summoney)
print('回収率',round(monthdf['当金'].sum()/monthdf['出費'].sum(),2))
print(monthdf[7:].mean())
monthdf

In [ ]:
DAY = kekka[pd.to_datetime(kekka['date'], format="%Y年%m月%d日")>=datetime.strftime(date1, format="%Y%m%d")]
kekka['date'].unique()

In [ ]:
class_return(kekka)

In [ ]:
place_return(kekka)

In [ ]:
# 期待値ごとの回収値変化
x = np.arange(1, 15, 0.1) #1から20までを0.1きざみ
y,z,a=[],[],[]
hani,tansyo_pro,old_tansyo = 2,0,0
for num in x:
    try:
        plotdf = target.sort_values('期待値', ascending=True)
        plotdf=plotdf[(plotdf['期待値']>num) & (plotdf['期待値']<num+hani)]
        kaisyu = plotdf[plotdf['着順']<=1]['単勝オッズ'].sum()/len(plotdf)
        tekityu = len(plotdf[plotdf['着順']<=1])/len(plotdf)
        race_num = len(plotdf)
        plotdf['賭け金']=1/plotdf['単勝オッズ']*10000
        tansyo_pro = len(plotdf[plotdf['着順']==1])*10000/plotdf['賭け金'].sum()
        if kaisyu>1 and tekityu>0.1 and race_num>50:
            print('回収率',round(kaisyu,2),'期待値',round(num,1),'~',round(num+hani,1),'対象レース',race_num,'的中率',round(tekityu*100,1),'単勝適正回収値',round(tansyo_pro,2))
            if tansyo_pro>old_tansyo:
                old_tansyo = tansyo_pro#更新
                print_text = '回収率',round(kaisyu,2),'期待値',round(num,1),'~',round(num+hani,1),'対象レース',race_num,'的中率',round(tekityu*100,1),'単勝適正回収値',round(tansyo_pro,2)
        y.append(kaisyu)
        z.append(tansyo_pro)
        a.append(tekityu)
    except ZeroDivisionError:
        # print('skip_num',num)
        y.append(0)
        z.append(0)
        a.append(0)
        continue
print(print_text)
fig = plt.figure(facecolor="white")
plt.grid(color = "gray", linestyle="--")
plt.plot(x,y)
plt.plot(x,z)
plt.plot(x,a)

In [ ]:
#該当馬が存在せずtop2から単勝馬券を購入した場合
print('回収率',kekka[kekka['着順']<=1]['単勝オッズ'].sum()/len(kekka))
print('対象レース',len(kekka))
print('的中レース',len(kekka[kekka['着順']<=1]))
print('的中率',(len(kekka[kekka['着順']<=1])/len(kekka)))
# print('単勝適正回収値',len(target[target['着順']==1])*return_money/target['賭け金'].sum())

In [ ]:
#さらにtop2のscore1位から購入した場合
sc1 = kekka[kekka['score_rank']==1]
print('回収率',sc1[sc1['着順']<=1]['単勝オッズ'].sum()/len(sc1))
print('対象レース',len(sc1))
print('的中レース',len(sc1[sc1['着順']<=1]))
print('的中率',(len(sc1[sc1['着順']<=1])/len(sc1)))
# print('単勝適正回収値',len(sc1[sc1['着順']==1])*return_money/sc1['賭け金'].sum())

In [ ]:
#さらにtop2のscore２位から購入した場合
sc2 = kekka[kekka['score_rank']==2]
print('回収率',sc2[sc2['着順']<=1]['単勝オッズ'].sum()/len(sc2))
print('対象レース',len(sc2))
print('的中レース',len(sc2[sc2['着順']<=1]))
print('的中率',(len(sc2[sc2['着順']<=1])/len(sc2)))
# print('単勝適正回収値',len(sc2[sc2['着順']==1])*return_money/sc2['賭け金'].sum())

## 指数モデル

In [ ]:
def index_model_simyu(index_df):
    def make3(first_result):
        top3 = first_result[first_result['return_rank'] <= 3].sort_values(by = ['race_id','return_rank'], ascending=False)
        # top3 = top3[~(top3['単勝オッズ'].str.contains('-', na=False))]
        top3 = top3.astype({'単勝オッズ': float})
        top3['期待値'] = top3['score_return']*top3['単勝オッズ']
        top3 = top3.sort_values('期待値', ascending=True)
        # ['course_len', 'race_type', 'race_round', 'weather','ground_state']
        # dropcol = ['複勝オッズ_min','複勝オッズ_max']
        # top3.drop(columns=dropcol,inplace=True)
        return top3
    def make2(first_result):
        top2 = first_result[first_result['score_rank'] <= 2].sort_values(by = ['race_id','score_top4'], ascending=False)
        # top2 = top2[~(top2['単勝オッズ'].str.contains('-', na=False))]
        top2 = top2.astype({'単勝オッズ': float})
        top2['期待値'] = (top2['score_top4'])*top2['単勝オッズ']
        top2 = top2.sort_values('期待値', ascending=True)
        # dropcol = ['複勝オッズ_min','複勝オッズ_max']
        # top2.drop(columns=dropcol,inplace=True)
        return top2
    sumdf = pd.DataFrame()
    top3 = make3(index_df)
    group_re = top3.groupby('race_id').sum()
    mergedf_re = top3.merge(group_re[['単勝オッズ','score_top4','score_return','score_rank','期待値']],on='race_id',how = 'left')
    top2 = make2(index_df)
    group_top = top2.groupby('race_id').sum()
    mergedf = top2.merge(group_top[['単勝オッズ','score_top4','score_return','score_rank','期待値']],on='race_id',how = 'left')
    sumdf = group_re.merge(group_top,how='outer',right_index=True,left_index=True,suffixes=['_re','_top'])
    race_id = sumdf[(sumdf['score_return_re']>sumdf['score_top4_top'])].index
    hits = top3[top3['race_id'].isin(race_id)].sort_values(['race_id','le_place','R','score_return']).copy()
    target = hits[hits['単勝オッズ']<=20].copy()
    target = target[target['score_return'].isin(target.groupby('race_id')['score_return'].max())]
    target = target[(target['score_rank']>=9)&(target['score_rank']<=10)]
    dropcol = ['course_len','race_type','race_round','weather','ground_state']
    target.drop(columns=dropcol,inplace=True)
    return target.sort_values(['le_place','R','馬番'], ascending=True)

In [ ]:
indf = index_model_simyu(first_result)

In [ ]:
print('回収率',indf[indf['着順']<=1]['単勝オッズ'].sum()/len(indf))
print('平均的中オッズ',indf[indf['着順']<=1]['単勝オッズ'].mean())
print('対象レース',len(indf))
print('対象レース割合',indf['race_id'].nunique()/first_result['race_id'].nunique())
print('的中レース',len(indf[indf['着順']<=1]))
print('的中率',(len(indf[indf['着順']<=1])/len(indf)))
print('出現確率',indf['date'].nunique()/first_result['date'].nunique())
indf[indf['着順']<=1]['単勝オッズ'].std()

In [ ]:
kakekin = 10000
summoney = 0
a,b,c,d,e,f,g = [],[],[],[],[],[],[]
for mon in range(1,datetime.now().month+1):
    date1 = datetime(2022, mon, 1)
    date2 = date1 + relativedelta(months=1)  #年をまたいだ1ヶ月加算
    DAY = indf[pd.to_datetime(indf['date'], format="%Y年%m月%d日")>=datetime.strftime(date1, format="%Y%m%d")]
    DAY = DAY[pd.to_datetime(DAY['date'], format="%Y年%m月%d日")<datetime.strftime(date2, format="%Y%m%d")]

    syuppi = kakekin*len(DAY)
    race_num = len(DAY)
    if race_num == 0:
        race_num=1
    profit = round(kakekin*DAY[DAY['着順']==1]['単勝オッズ'].sum()-syuppi)
    a.append(mon)
    b.append(syuppi)
    c.append(round(kakekin*DAY[DAY['着順']==1]['単勝オッズ'].sum()))
    d.append(profit)
    e.append(round(len(DAY[DAY['着順']==1])/race_num*100,1))
    f.append(round(DAY[DAY['着順']==1]['単勝オッズ'].sum()/len(DAY),2))
    g.append(race_num)
    summoney = summoney+profit
monthdf = pd.DataFrame(
        data={'月': a, '出費': b, '当金': c, '利益': d, '当率': e, '回収率': f, '対象レース': g},
        columns=['月', '出費', '当金', '利益', '当率', '回収率', '対象レース'])
print('１レース当たりの掛け金',kakekin)
print('合計損益',summoney)
print('回収率',round(monthdf['当金'].sum()/monthdf['出費'].sum(),2))
print(monthdf[7:].mean())
monthdf

## ハイブリット２形式

In [ ]:
test = first_result.copy()
test['単勝オッズ'] = test['単勝オッズ'].astype(str)
test[['複勝オッズ_min', '複勝オッズ_max']] = 0
hybrid2 = hybrid2_model(test)

In [ ]:
print('回収率',hybrid2[hybrid2['着順']<=1]['単勝オッズ'].sum()/len(hybrid2))
print('平均的中オッズ',hybrid2[hybrid2['着順']<=1]['単勝オッズ'].mean())
print('対象レース',len(hybrid2))
print('対象レース割合',hybrid2['race_id'].nunique()/first_result['race_id'].nunique())
print('的中レース',len(hybrid2[hybrid2['着順']<=1]))
print('的中率',(len(hybrid2[hybrid2['着順']<=1])/len(hybrid2)))
print('出現確率',hybrid2['date'].nunique()/first_result['date'].nunique())
hybrid2[hybrid2['着順']<=1]['単勝オッズ'].std()

# 後日作業

In [ ]:
""" 
#その日に開催されるレースIDと日付けを指定する
#日付は予測値に影響を与えるので注意(hrのマージの日付になる)
"""
date = '2022年10月30日'
#開催場所に合わせて設定
racenum = 12
race_id_list1 = ['2022060401{}'.format(str(i).zfill(2)) for i in range(1, racenum+1, 1)]
race_id_list2 = ['2022070501{}'.format(str(i).zfill(2)) for i in range(1, racenum+1, 1)]
# race_id_list3 = ['2022010203{}'.format(str(i).zfill(2)) for i in range(1, racenum+1, 1)]
# race_id_list = race_id_list1 + race_id_list2 + race_id_list3
race_id_list = race_id_list1 + race_id_list2

print(len(race_id_list))
date_list = [date]*len(race_id_list)
race_id_day_dict = dict(zip(race_id_list,date_list))
race_id_day_dict

In [ ]:
st = ShutubaTable.read_pickle(['pickle/syutuba/race/syutuba_latest.pickle','pickle/syutuba/race/syutuba_{}.pickle'.format(date)])
st.data.to_pickle('pickle/syutuba/race/syutuba_latest.pickle')

In [ ]:
st=ShutubaTable.read_pickle(['pickle/syutuba/race/syutuba_latest.pickle'])
#不足データがある場合、スクレイピングされ常に更新されていく
hi = HorseInfo.read_pickle(['pickle/syutuba/latest/h_info_latest.pickle'])
st.merge_horse_info(hi)
jr = jockey_results.read_pickle(['pickle/syutuba/latest/jockey_id_latest.pickle'])
st.merge_jockey_info(jr,5)
st.preprocessing()
r = Results.read_pickle(['pickle/syutuba/r_data/r_data_latest.pickle'])

In [ ]:
print(len(st.data.index[~st.data.index.isin(r.data.index)].unique()))
st.data.index[~st.data.index.isin(r.data.index)].unique()

## resultsの更新

In [ ]:
#rデータのstとの差分をスクレイプし更新する
date = '2022年11月27日'
r = Results.read_pickle(['pickle/syutuba/r_data/r_data_latest.pickle'])
diff_race_id_list = st.data.index[~st.data.index.isin(r.data.index)].unique()
results = Results.scrape(diff_race_id_list)
results.to_pickle('pickle/syutuba/r_data/r_data_{}.pickle'.format(date))

In [ ]:
diff_race_id_list

In [ ]:
results['date'].unique()

In [ ]:
del r
r=Results.read_pickle(['pickle/syutuba/r_data/r_data_latest.pickle','pickle/syutuba/r_data/r_data_{}.pickle'.format(date)])
r.data.to_pickle('pickle/syutuba/r_data/r_data_latest.pickle')

## return_tablesの更新

In [ ]:
#stとの差分
rt_st = Return.read_pickle(['pickle/syutuba/latest/return_table_latest.pickle'])
my_list = st.data.index[~st.data.index.isin(rt_st.return_tables.index)].unique()

#不足があった場合、適当な日付で取得
my_dict = {}
day = '2022年11月27日'
day_list = [day]*len(my_list)
my_dict = dict(zip(my_list,day_list))
len(my_dict)

In [ ]:
rt_st = Return.scrape(my_dict)
rt_st.to_pickle('pickle/syutuba/return/return_table_{}.pickle'.format(date))

# #最新に更新
del rt_st
rt_st = Return.read_pickle(['pickle/syutuba/latest/return_table_latest.pickle','pickle/syutuba/return/return_table_{}.pickle'.format(date)])
rt_st.return_tables.to_pickle('pickle/syutuba/latest/return_table_latest.pickle')

## resultsの更新データを反映

In [ ]:
%%time
#testデータ作成用
#最新データかつSTデータと同様でなければシミュレーションとして成り立たないため
r = Results.read_pickle(['pickle/syutuba/r_data/r_data_latest.pickle'])
jr = jockey_results.read_pickle(['pickle/syutuba/latest/jockey_id_latest.pickle'])
hr = HorseResults.read_pickle(['pickle/syutuba/latest/horse_results_latest.pickle'])
hi = HorseInfo.read_pickle(['pickle/syutuba/latest/h_info_latest.pickle'])
p = Peds.read_pickle(['pickle/syutuba/latest/peds_id_latest.pickle'])

past_race_num=10
t = time.time()
r.merge_horse_info(hi)
r.merge_jockey_info(jr,5)
time_protcol()
r.preprocessing()
time_protcol()
hr.preprocessing()
hr_p_backup = hr.horse_results_p.copy() #毎回作成する必要がないようにbuckupとして保持
time_protcol()
r.merge_horse_results(hr, n_samples_list=[5,9,'all'])
# r.merge_horse_results(hr, n_samples_list=[5])
r.concat_past_data(hr,past_race_num)
t = time.time()
p.encode()
r.merge_peds(p.peds_e)
r.process_categorical(past_race_num)
time_protcol()

In [ ]:
r.data_p['date'].unique()

In [ ]:
r.data_p.to_pickle('pickle/r_data19_21/r_data_p22.pickle')
r.data_pe.to_pickle('pickle/r_data19_21/r_data_pe22.pickle')